<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/ons_study_v35_base_fair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# @title [CELL 1] Environment Setup & Configuration
print("="*60)
print("      [CELL 1] SYSTEM WAKE-UP & ENVIRONMENT SETUP...")
print("="*60)

# 1. Install required packages (ensures Colab has what we need)
!pip install -q pandas pytz

# 2. Import standard libraries used across the entire pipeline
import os
import re
import csv
import json
import math
import shutil
import pandas as pd
import pytz
from datetime import datetime
from ftplib import FTP_TLS

# 3. Securely load FTP Credentials from Colab Secrets
try:
    from google.colab import userdata
    print("\n[+] Google Colab Environment Detected. Loading secrets...")

    # Check and set FTP Host
    HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
    if HOST:
        os.environ["FTP_HOST"] = HOST
        print("    ✅ FTP_HOST loaded.")
    else:
        print("    ⚠️ WARNING: FTP_HOST not found in secrets.")

    # Check and set FTP User
    USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
    if USER:
        os.environ["FTP_USER"] = USER
        print("    ✅ FTP_USER loaded.")
    else:
        print("    ⚠️ WARNING: FTP_USER not found in secrets.")

    # Check and set FTP Password
    PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
    if PASS:
        os.environ["FTP_PASS"] = PASS
        print("    ✅ FTP_PASS loaded.")
    else:
        print("    ⚠️ WARNING: FTP_PASS not found in secrets.")

except ImportError:
    print("\n[!] Running outside of Google Colab. Make sure environment variables are set manually.")
except userdata.SecretNotFoundError as e:
    print(f"\n❌ SECRET ERROR: {e}")
    print("    Please click the '🔑 Keys' icon on the left sidebar and ensure FTP_HOST, FTP_USER, and FTP_PASS are saved.")

print("\n✅ Cell 1 (Environment Setup) Complete. The system is ready.")
print("="*60)

      [CELL 1] SYSTEM WAKE-UP & ENVIRONMENT SETUP...

[+] Google Colab Environment Detected. Loading secrets...
    ✅ FTP_HOST loaded.
    ✅ FTP_USER loaded.
    ✅ FTP_PASS loaded.

✅ Cell 1 (Environment Setup) Complete. The system is ready.


In [35]:
# @title [CELL 2] The Data & Math Engine (Pure Analytics)
print("="*60)
print("      [CELL 2] DATA & MATH ENGINE STARTING...")
print("="*60)

import os, sys, re, csv, json, math, shutil
import pandas as pd
from ftplib import FTP_TLS
try:
    from google.colab import userdata
except ImportError:
    pass

CSV_DB = "engine_database.csv"
JSON_DB = "compiled_database.json"
KEY_FILE = "match_to_unmasked.csv"
PROCESSED_GED = "_processed_unmasked.ged"

try:
    HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
    USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
    PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
except: pass

# --- STEP 1: FETCH FILES ---
if not os.path.exists(KEY_FILE):
    print(f"    [+] Fetching {KEY_FILE} via FTP...")
    try:
        ftps = FTP_TLS(); ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()
        with open(KEY_FILE, "wb") as f: ftps.retrbinary(f"RETR /ons-study/{KEY_FILE}", f.write)
        ftps.quit()
    except Exception as e: print(f"    ⚠️ FTP fetch failed: {e}")

ged_files = [f for f in os.listdir('.') if f.lower().endswith('.ged') and "_processed" not in f.lower()]
if not ged_files:
    print("❌ No GEDCOM found. Please upload one.")
else:
    DEFAULT_GEDCOM = sorted(ged_files, key=lambda x: os.path.getmtime(x), reverse=True)[0]
    shutil.copyfile(DEFAULT_GEDCOM, PROCESSED_GED)
    print(f"    [+] Active GEDCOM: {DEFAULT_GEDCOM}")

# --- STEP 2: LOAD AUTH & PARSE GEDCOM ---
csv_auth = {}
if os.path.exists(KEY_FILE):
    with open(KEY_FILE, 'r', errors='replace') as f:
        for i, row in enumerate(csv.reader(f)):
            if i > 0 and len(row) >= 2:
                code, name = row[0].strip().lower(), row[1].strip()
                tid = "I" + re.sub(r'[^0-9]', '', row[2]) if len(row)>2 and row[2].strip() else ""
                sort_key = row[3].strip().lower() if len(row)>3 else ""
                csv_auth[code] = {"name": name, "id": tid, "sort_key": sort_key}

individuals = {}; families = {}; persons_data = {}
current_id = None; current_fam = None; mode = None

def clean_name(n):
    if not n: return "findme"
    s = n.replace("/", "").strip()
    if s.lower() in ["unknown", "missing", "searching", "living", "private", "nee", "wife"] or "?" in s: return "findme"
    return s

with open(PROCESSED_GED, "r", encoding="utf-8", errors="replace") as f:
    for line in f:
        line = line.strip(); parts = line.split(" ", 2)
        if len(parts) < 2: continue
        lvl, tag, val = parts[0], parts[1], parts[2] if len(parts)>2 else ""

        if lvl == "0" and "INDI" in val:
            current_id = tag.replace("@", "")
            individuals[current_id] = {"name": "findme", "famc": None, "fams": [], "code": "", "cm": 0}
            persons_data[current_id] = {'name': f'ID {current_id}', 'bdate': '', 'bplace': '', 'ddate': '', 'dplace': '', 'sources_count': 0, 'citations_count': 0}
            current_fam = None; mode = None
        elif current_id and lvl != "0":
            if tag == "NAME" and lvl == "1":
                individuals[current_id]["name"] = clean_name(val)
                persons_data[current_id]["name"] = val.replace('/', '').strip()
            elif tag == "FAMC" and lvl == "1": individuals[current_id]["famc"] = val.replace("@", "")
            elif tag == "FAMS" and lvl == "1": individuals[current_id]["fams"].append(val.replace("@", ""))
            elif tag == "NPFX" and lvl == "2":
                m_code = re.search(r'(\d+)\s*&?\s*([^ \t\n\r\f\v]+)', val)
                if m_code: individuals[current_id]["code"] = m_code.group(2).lower()
                m_cm = re.search(r'^(\d+)|(\d+)\s*cM', val, re.IGNORECASE)
                if m_cm: individuals[current_id]["cm"] = int(m_cm.group(1) or m_cm.group(2))
            elif tag == "BIRT": mode = "BIRT"
            elif tag == "DEAT": mode = "DEAT"
            elif tag == "SOUR" and lvl == "1": persons_data[current_id]['sources_count'] += 1; mode = None
            elif tag == "SOUR" and lvl == "2": persons_data[current_id]['citations_count'] += 1
            elif tag == "DATE" and mode == "BIRT": persons_data[current_id]['bdate'] = val
            elif tag == "DATE" and mode == "DEAT": persons_data[current_id]['ddate'] = val
            elif tag == "PLAC" and mode == "BIRT": persons_data[current_id]['bplace'] = val
            elif tag == "PLAC" and mode == "DEAT": persons_data[current_id]['dplace'] = val
            elif lvl == "1": mode = None

        elif lvl == "0" and "FAM" in val:
            current_fam = tag.replace("@", "")
            families[current_fam] = {"husb": None, "wife": None}
            current_id = None
        elif current_fam and lvl != "0":
            if tag == "HUSB": families[current_fam]["husb"] = val.replace("@", "")
            elif tag == "WIFE": families[current_fam]["wife"] = val.replace("@", "")

# --- STEP 3: BUILD BASE CSV ROWS ---
print("    [+] Tracing Lineages & Generating CSV...")
def get_parents(pid):
    if not pid or pid not in individuals: return None, None
    famc = individuals[pid]["famc"]
    if not famc or famc not in families: return None, None
    return families[famc]["husb"], families[famc]["wife"]

yates_memo = {}
def has_yates(pid):
    if not pid or pid not in individuals: return False
    if pid in yates_memo: return yates_memo[pid]
    n = individuals[pid]["name"].lower()
    if "yates" in n or "yeates" in n: yates_memo[pid] = True; return True
    d, m = get_parents(pid)
    res = has_yates(d) or has_yates(m)
    yates_memo[pid] = res; return res

def climb(start_id):
    curr = start_id; lin = []
    while curr:
        p = individuals.get(curr)
        if not p: break
        spouse_name = "findme"; sp_id = None
        if p["fams"]:
            fid = p["fams"][0]
            if fid in families:
                f = families[fid]
                sp_id = f["wife"] if f["husb"] == curr else f["husb"]
                if sp_id and sp_id in individuals: spouse_name = individuals[sp_id]["name"]
        lin.append({"name": p["name"], "id": curr, "spouse": spouse_name, "sp_id": sp_id})
        d, m = get_parents(curr)
        if not d and not m: break
        dy, my = has_yates(d), has_yates(m)
        curr = d if dy and not my else (m if my and not dy else (d if d else m))
    return lin

for code, td in csv_auth.items():
    if td["id"] and td["id"] in individuals:
        full = list(reversed(climb(td["id"])))
        td["lin_str"] = " -> ".join([x["name"] for x in full])
        td["pids"] = ",".join([x["id"] for x in full])
    else:
        td["lin_str"] = ""; td["pids"] = ""

rows = []
for uid, p in individuals.items():
    if p["code"]:
        kc = p["code"]
        td = csv_auth.get(kc, {"name": kc, "id": "", "sort_key": "", "lin_str": "", "pids": ""})
        t_disp = f"{td['name']} [{td['id']}]" if td['id'] else f"{td['name']} [{kc}]"

        lin = climb(uid)
        if not lin: continue
        full = list(reversed(lin))

        gen1 = full[0]
        top_name = gen1["name"]; sp_name = gen1["spouse"]
        pair_simp = f"{top_name} & {sp_name}" if sp_name != "findme" else top_name

        sur = top_name.split()[-1] if top_name.split() else ""
        firsts = re.sub(f"{re.escape(sur)}$", "", top_name).strip() if sur else top_name
        b_yr = re.search(r'\d{4}', persons_data[gen1['id']]['bdate'])
        d_yr = re.search(r'\d{4}', persons_data[gen1['id']]['ddate'])
        dates = f"({b_yr.group(0) if b_yr else 'findme'} - {d_yr.group(0) if d_yr else 'findme'})"
        dir_lbl = f"{sur}, {firsts} {dates}" + (f" & {sp_name}" if sp_name != "findme" else "")

        rows.append({
            "Tester_Code": kc, "Tester_Name": td["name"], "Tester_ID": td["id"], "Tester_Display": t_disp,
            "Tester_Sort_Key": td["sort_key"], "Tester_Lineage": td["lin_str"], "Tester_Path_IDs": td["pids"],
            "Match_Name": p["name"], "Match_ID": uid, "cM": p["cm"],
            "Match_Lineage": " -> ".join([pair_simp] + [x["name"] for x in full[1:]]),
            "Match_Path_IDs": ",".join([x["id"] for x in full]),
            "Authority_Directory_Label": dir_lbl
        })

df = pd.DataFrame(rows)
df.to_csv(CSV_DB, index=False, encoding="iso-8859-15", quoting=csv.QUOTE_ALL)
df.rename(columns={"Authority_Directory_Label": "Dir_Label", "Tester_Code": "Kit_Code", "Match_Lineage": "Lineage", "Match_Path_IDs": "s_ids", "Tester_Display": "Kit_Name"}, inplace=True)
df['search_ids'] = df['s_ids']; df['search_names'] = df['Lineage'].astype(str).str.replace(' -> ', '|')
df['t_names'] = df['Tester_Lineage'].astype(str).str.replace(' -> ', '|'); df['t_ids'] = df['Tester_Path_IDs'].astype(str).str.replace(',', '|')

# --- STEP 4: MATHEMATICAL ENGINE (ANCHOR/CSS) ---
print("    [+] Calculating Math Matrices & Audits...")
def clean_num(s): return re.sub(r'[^0-9]', '', str(s))
def norm_log(val, cap): return min(1.0, math.log(1 + max(0, val or 0)) / math.log(1 + cap))

id_to_kits = {}
for _, r in df.iterrows():
    for i in [clean_num(x) for x in str(r['s_ids']).split(',') if x]:
        id_to_kits.setdefault(i, set()).add(r['Kit_Name'])

participant_scores = []
for p_name, grp in df.groupby('Kit_Name'):
    val_grp = grp[grp['Dir_Label'] != 'No Matches']
    pm = len(val_grp)
    if pm == 0: continue

    dc = val_grp['Dir_Label'].value_counts()
    hc_t, hc_2 = (int(dc.iloc[0]), int(dc.iloc[1])) if len(dc)>1 else (int(dc.iloc[0]), 0) if len(dc)>0 else (0,0)
    target_anc = str(dc.index[0]) if len(dc)>0 else "Unknown"

    hh, target_id = 0, None
    for _, r in val_grp.iterrows():
        for i in [clean_num(x) for x in str(r['s_ids']).split(',') if x]:
            if len(id_to_kits.get(i, set())) > hh: hh, target_id = len(id_to_kits[i]), i

    spine_ids = []; br = 0; tb = 0
    if target_id:
        tb = len(id_to_kits.get(target_id, set()))
        col = df[df['s_ids'].apply(lambda x: target_id in [clean_num(y) for y in str(x).split(',') if y])]
        b_set = set()
        for _, r in col.iterrows():
            ids = [clean_num(x) for x in str(r['s_ids']).split(',') if x]
            nms = str(r['search_names']).split('|')
            if target_id in ids:
                idx = ids.index(target_id)
                b_set.add(nms[idx+1].replace('findme', '?').replace('FINDME', '?').split(' (')[0].strip() if idx+1 < len(nms) else "Direct")
        br = len(b_set)
        spine_ids = [clean_num(x) for x in str(val_grp[val_grp['s_ids'].apply(lambda x: target_id in str(x))].iloc[0]['s_ids']).split(',') if x]

    # Math formulas
    dr = float(hc_t / max(1, hc_2))
    w_sum = norm_log(pm, 150) + norm_log(hc_t, 100) + (norm_log(dr, 10)*1.5) + norm_log(tb, 40) + norm_log(hh, 150)
    w_sum += 2.0 * (1.0 if br>=6 else 0.85 if br==5 else 0.70 if br==4 else 0.50 if br==3 else 0.25 if br==2 else 0)
    st_str, st_val = ("PASS", 1.0) if pm>=15 and br>=3 and dr>=1.5 else ("PARTIAL", 0.85) if pm>=15 and br>=2 else ("FAIL", 0.6)
    css = float(100 * (w_sum / 7.5) * st_val)

    # Doc Audit
    tp_viol = 0; tp_checks = 0; diag_log = []
    for pid in spine_ids:
        d = persons_data.get(pid, {})
        if d.get('sources_count',0) + d.get('citations_count',0) == 0: diag_log.append(f"Missing Sources: {d.get('name')} (I{pid})")

    docs_score = 88.5 if len(diag_log)==0 else max(35.0, 85.0 - (len(diag_log)*5))
    anchor = min(100.0, (0.65 * css) + (0.35 * docs_score) + (10.0 * min(css, docs_score)/100.0))

    participant_scores.append({'pName': str(p_name), 'targetAnc': target_anc, 'targetID': str(target_id or ''), 'PM': int(pm), 'HC_T': int(hc_t), 'DR': float(dr), 'BR': int(br), 'NS': int(hh), 'ST_str': st_str, 'cssFinal': float(css), 'DOCS': float(docs_score), 'ANCHOR': float(anchor), 'diagLog': diag_log, 'AX': 1, 'CC': 1.0, 'BC': 1.0, 'DC': 1.0, 'TP': 1.0, 'isGroup': False, 'GD': len(spine_ids)})

anc_data = {}; part_data = {}
for lbl, grp in df.groupby('Dir_Label'):
    if str(lbl).strip() == 'No Matches' or not str(lbl).strip(): continue
    u_t = len(grp['Kit_Name'].unique())
    anc_data[str(lbl)] = {"name": str(lbl), "matches": len(grp), "cm": int(grp['cM'].sum()), "badge": "Platinum" if len(grp)>=30 else "Gold", "integrity": min(100, (len(grp)*2) + (u_t*10)), "testers": u_t}

for kname, grp in df.groupby('Kit_Name'):
    val_grp = grp[grp['Dir_Label'] != 'No Matches']
    p_score = next((i for i in participant_scores if i["pName"] == str(kname)), None)
    part_data[str(kname)] = {
        "name": str(kname), "sort_key": str(grp.iloc[0].get('Tester_Sort_Key', str(kname).lower())).strip(),
        "matches": len(val_grp), "cm": int(val_grp['cM'].sum()) if not val_grp.empty else 0,
        "badge": "Keystone Tester" if len(val_grp)>=15 else "Study Participant",
        "css_status": p_score['ST_str'] if p_score else "UNSCORED", "ns": int(p_score['NS']) if p_score else 0,
        "br": int(p_score['BR']) if p_score else 0, "target_id": p_score['targetID'] if p_score else '',
        "docs_score": float(p_score['DOCS']) if p_score else 0.0, "diag_log": p_score['diagLog'] if p_score else [],
        "kit_code": str(grp.iloc[0]['Kit_Code']), "integrity": 95
    }

# --- STEP 5: SAVE GLOBALS TO DISK ---
print("    [+] Saving Compiled JS_GLOBALS to Disk...")
db_json = df[['Dir_Label', 'Kit_Name', 'cM', 'Match_ID', 'Lineage', 'search_ids', 'search_names', 't_names', 't_ids', 'Tester_ID', 'Kit_Code']].rename(columns={'Dir_Label':'ancestor', 'Kit_Name':'participant', 'cM':'cm', 'Match_ID':'id', 'Lineage':'lineage', 'Tester_ID':'tester_id', 'Kit_Code':'kit_code'}).to_dict(orient='records')
final_json = {"PRECOMPUTED": participant_scores, "DATA": {"ancestors": anc_data, "participants": part_data, "persons": persons_data}, "DB": db_json}

# ✨ THE FIX: A custom encoder that forcefully translates any Pandas math types into safe Python types!
def safe_json_encoder(obj):
    if pd.isna(obj): return None
    if hasattr(obj, 'item'): return obj.item()
    return str(obj)

with open(JSON_DB, "w") as f:
    json.dump(final_json, f, default=safe_json_encoder)

print(f"\n[SUCCESS] Cell 2 Complete. Data successfully crunched and saved to {JSON_DB}.")

      [CELL 2] DATA & MATH ENGINE STARTING...
    [+] Active GEDCOM: yates_study_2025.ged
    [+] Tracing Lineages & Generating CSV...
    [+] Calculating Math Matrices & Audits...
    [+] Saving Compiled JS_GLOBALS to Disk...

[SUCCESS] Cell 2 Complete. Data successfully crunched and saved to compiled_database.json.


In [36]:
# @title [CELL 3] The Template Vault (Pure HTML/CSS)
print("="*60)
print("      [CELL 3] TEMPLATE VAULT LOADING...")
print("="*60)

# 🌟 0. CATCH-ALL CSS (SAFELY SECURED) 🌟
CONTENTS_CSS = ""
SHARE_CSS = ""
GLOSS_CSS = ""
GLOSSARY_CSS = ""
SUBSCRIBE_CSS = ""
THEORY_CSS = ""
TREE_CSS = ""
ADMIN_CSS = "<style>.dashboard-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(200px,1fr));gap:20px;margin:30px auto;max-width:1200px}.dash-card{background:white;padding:20px;border-radius:8px;text-align:center;box-shadow:0 4px 6px rgba(0,0,0,0.1);text-decoration:none;color:#333;border:1px solid #ddd; transition:transform 0.2s;}.dash-card:hover{transform:translateY(-5px);border-color:#006064;background:#e0f7fa}.dash-icon{font-size:40px;margin-bottom:10px;display:block}.dash-title{font-weight:bold;font-size:1.1em;color:#006064}.audit-table-wrapper{background:white;padding:25px;border-radius:8px;box-shadow:0 4px 6px rgba(0,0,0,0.1);max-width:1400px;margin:0 auto} td{padding:10px;border-bottom:1px solid #eee}</style>"
DOSS_CSS = ""
REGISTER_CSS = "<style>.table-scroll-wrapper{overflow-x:auto; background:white; padding:20px; border-radius:8px; box-shadow:0 4px 6px rgba(0,0,0,0.1); max-width:1400px; margin:20px auto; display:flex; justify-content:center; flex-direction:column;} .table-scroll-wrapper table.dataframe { margin: 0 auto; width: 100%; }</style>"

# 🌟 1. GLOBAL CSS & NAV 🌟
NAV_HTML = r"""<style>nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none} nav.oldnav li{display:inline-block} nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px} nav.oldnav a:hover{background-color:#00838f!important} @media print { nav.oldnav, #nav-slot, .no-print { display: none !important; } }</style><nav class="oldnav"><ul><li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li><li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li><li><a href="/ons-study/ons_yates_dna_register.shtml">DNA Register</a></li><li><a href="/ons-study/just-trees.shtml">Trees</a></li><li><a href="/ons-study/dna_network.html">DNA Network</a></li><li><a href="/ons-study/proof_engine.html">Proof Engine</a></li><li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li><li><a href="/ons-study/proof_consolidator.html" style="background-color:#4a148c; color:#fff !important; font-weight:bold; border-left:1px solid #7c43bd; border-right:1px solid #7c43bd;">Report</a></li><li><a href="/ons-study/dna_theory_of_the_case.htm" style="background-color:#d84315; color:#fff !important; font-weight:bold; border-left:1px solid #ffab91; border-right:1px solid #ffab91;">ANCHOR Theory</a></li><li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li><li><a href="/ons-study/gedmatch_integration.shtml" style="color:#81d4fa;">GEDmatch</a></li><li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li><li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40; font-weight:bold;">Subscribe</a></li></ul></nav>"""
SITE_INFO = r"""<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;margin-bottom:0;">This register employs <em>Collateral DNA Saturation</em>—a method blending genealogical reasoning with data-driven logic to prove connections using multiple independent DNA cousins.</p></div>"""
CSS_BASE = r"""body{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px;display:flex;flex-direction:column;min-height:100vh;margin:0;} .wrap{flex:1;} .proof-card{background:white;max-width:1100px;margin:20px auto;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);padding:40px} .badge{padding:5px 10px;border-radius:4px;font-weight:bold;font-size:0.85em;text-transform:uppercase;border:1px solid #ccc;} .badge-platinum{background:#eceff1;color:#263238} .badge-gold{background:#fff8e1;color:#f57f17} .badge-silver{background:#f5f5f5;color:#616161} .badge-bronze{background:#efebe9;color:#5d4037} table{width:100%;border-collapse:collapse;margin-top:15px;margin-bottom:40px;font-family:'Georgia',serif;font-size:15px;} th{background:#eceff1;color:#263238;padding:12px;text-align:left;border-bottom:2px solid #000;} td{padding:12px;border-bottom:1px solid #ddd;vertical-align:top;} @media print{ .no-print{display:none !important;} .only-print{display:block !important;} .proof-card{box-shadow:none;border:none;padding:0;margin:0;} body{background:white;padding:0;display:block;} th{background:#f0f0f0 !important;color:#000 !important;} .badge{border:1px solid #000;color:#000;background:transparent !important;} .legal-footer{background:transparent !important; border-top:2px solid #000 !important; color:#000 !important; page-break-inside:avoid !important; padding:10px 0 !important; margin-top:30px !important;} } .only-print{display:none;}"""
JS_CORE = r"""<script type="text/javascript">(function(){ function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/ +/g,' ').trim().toLowerCase();} function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.-]/g,'')),nB=parseFloat(B.replace(/[^0-9.-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);} function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(' (asc)','').replace(' (desc)','');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}} window.filterTable = function() { var input = document.getElementById("tableSearch"); var filter = input.value.toUpperCase(); var table = document.getElementById("reg-table") || document.querySelector("table.dataframe"); var tr = table.getElementsByTagName("tr"); for (var i = 1; i < tr.length; i++) { var tdArr = tr[i].getElementsByTagName("td"); var found = false; for (var j = 0; j < tdArr.length; j++) { if (tdArr[j]) { var txtValue = tdArr[j].textContent || tdArr[j].innerText; if (txtValue.toUpperCase().indexOf(filter) > -1) { found = true; break; } } } tr[i].style.display = found ? "" : "none"; } } function init(){ var t=document.getElementsByTagName('table'); for(var i=0;i<t.length;i++) if(t[i].className.indexOf('sortable') !== -1) makeSortable(t[i]); } if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init(); })();</script>"""
BTT_BTN = r"""<style>.btt{position:fixed;bottom:20px;right:20px;background:#00838f;color:white;padding:10px 15px;text-decoration:none;border-radius:4px;font-weight:bold;box-shadow:0 2px 5px rgba(0,0,0,0.3);z-index:1000;opacity:0.9;} .btt:hover{opacity:1;background:#006064;} @media print { .btt { display: none !important; } }</style><a href="#top" class="btt no-print">⬆️ Top</a>"""

# 🌟 2. REPORT CONSOLIDATOR (WITH SCALED MATRICES!) 🌟
CONSOLIDATOR_CSS = r"""<style>.consol-panel { background: #f3e5f5; border: 1px solid #ab47bc; padding: 25px; border-radius: 8px; margin-bottom: 25px; font-family: 'Segoe UI', sans-serif; text-align: center; } .consol-panel select { padding: 8px; font-size: 14px; width: 100%; border: 1px solid #7b1fa2; border-radius: 4px; } .consol-btn { background: #4a148c; color: white; border: none; padding: 12px 25px; font-size: 16px; font-weight: bold; border-radius: 4px; cursor: pointer; box-shadow: 0 4px 6px rgba(0,0,0,0.1); margin-top: 10px; } .consol-btn:hover { background: #38006b; } .vg-checkbox-container { height:150px; overflow-y:auto; border:1px solid #7b1fa2; background:white; border-radius:4px; padding:10px; font-size:13px; text-align:left; } .vg-checkbox-container label { display:block; margin-bottom:5px; cursor:pointer; } .vg-checkbox-container label:hover { background-color:#f3e5f5; } .academic-brief { background: white; max-width: 1100px; margin: 0 auto 30px auto; padding: 60px 80px; box-shadow: 0 4px 20px rgba(0,0,0,0.1); font-family: 'Georgia', serif; color: #000; line-height: 1.6; } .title-page { page-break-after: always; display: flex; flex-direction: column; justify-content: center; min-height: 70vh; padding: 20px; } .brief-section-title { font-size: 18px; text-transform: uppercase; border-bottom: 1px solid #ccc; padding-bottom: 5px; margin-top: 40px; margin-bottom: 20px; font-weight: bold; } .brief-table { width: 100%; border-collapse: collapse; margin-top: 15px; font-size: 13px; } .brief-table th { background: #f0f0f0; color: #000; border-bottom: 2px solid #000; border-top: 1px solid #000; padding: 10px; text-align: left; } .brief-table td { padding: 10px; border-bottom: 1px solid #ddd; vertical-align: middle; } @media print { .no-print { display: none !important; } body { background: white; padding: 0; } .academic-brief { box-shadow: none; padding: 0; max-width: 100%; border: none; margin-bottom: 0; } .matrix-page { page-break-before: always; page-break-inside: avoid; width: 100%; } .brief-table { font-size: 11px; width: 100%; table-layout: auto; } .brief-table th, .brief-table td { padding: 4px 6px; word-wrap: break-word; } }</style>"""
CONSOLIDATOR_HTML = f"""<div class="no-print consol-panel"><h2 style="color:#4a148c; margin-top:0;">Virtual Group & White Paper Builder</h2><div style="display:flex; justify-content:center; gap:20px; flex-wrap:wrap; margin-bottom:15px;"><div style="flex:1; max-width:400px; min-width:250px; text-align:left;"><label style="font-size:12px; font-weight:bold; color:#4a148c;">Select Kits for Virtual Group Analysis</label><div id="groupCheckboxes" class="vg-checkbox-container"></div></div><div style="flex:1; max-width:350px; min-width:250px; text-align:left;"><label style="font-size:12px; font-weight:bold; color:#4a148c;">Custom Group Name (e.g. "VA Yates Protocol")</label><input type="text" id="customGroupName" style="width:100%; box-sizing:border-box; padding:8px; border:1px solid #7b1fa2; border-radius:4px; margin-bottom:10px;" placeholder="Optional"><button class="consol-btn" style="width:100%; box-sizing:border-box;" onclick="runConsolidator('matrix')">📄 Generate Academic White Paper</button></div></div></div><div id="report-container"></div>"""
CONSOLIDATOR_JS = r"""<script>__JS_GLOBALS__
document.addEventListener('DOMContentLoaded', function() { const groupDiv = document.getElementById('groupCheckboxes'); if(!groupDiv) return; const validTesters = DB.filter(r => r.t_names && r.t_names.trim() !== ""); const uniqueTesters = [...new Set(validTesters.map(r => r.participant))].sort((a, b) => { let keyA = DATA.participants[a] ? (DATA.participants[a].sort_key || a) : a; let keyB = DATA.participants[b] ? (DATA.participants[b].sort_key || b) : b; return keyA.toLowerCase().localeCompare(keyB.toLowerCase()); }); uniqueTesters.forEach(t => { let kcode = DATA.participants[t] ? DATA.participants[t].kit_code : ''; let displayStr = kcode ? `${t} [${kcode}]` : t; const lbl = document.createElement('label'); lbl.innerHTML = `<input type="checkbox" value="${t}" class="vg-checkbox"> ${displayStr}`; groupDiv.appendChild(lbl); }); });
const getStudyStats = () => { const d = new Date(); return `Study Data Current As Of: ${d.toLocaleDateString('en-US', { month: 'long', day: 'numeric', year: 'numeric' })} ${d.toLocaleTimeString('en-US', { hour: 'numeric', minute: '2-digit', timeZoneName: 'short' })} | Total Autosomal matches: ${DB.length.toLocaleString()}`; };
const getTitlePage = (testerArray, customName) => { const year = new Date().getFullYear(); const titleName = customName || (testerArray.length > 1 ? `Virtual Group (${testerArray.length} Kits)` : testerArray[0]); return `<div class="academic-brief title-page"><div style="font-family: Arial, sans-serif; text-align:center; line-height:1.6;"><h1 style="font-size:36px; border-bottom:none; margin-bottom:5px;">Collateral Saturation</h1><h2 style="font-size:20px; font-weight:normal; color:#444; margin-top:0;">A Quantitative Method for Autosomal Lineage Reconstruction</h2><br><br><br><p style="font-size:18px;"><b>Ronald Eugene Yates, MPH</b><br>University of California, Los Angeles<br>1975</p><br><br><br><p style="font-size:16px;">Yates DNA Study<br>Autosomal Lineage Reconstruction Project</p><br><br><br><h3 style="color:#4a148c;">Analysis Target: ${titleName}</h3><br><br><p style="font-size:16px;">${year}</p><p style="font-size:14px; color:#004d40; margin-top:20px; font-weight:bold;">${getStudyStats()}</p><br><br><br><p style="font-size:14px; color:#555;">&copy; ${year} Ronald Eugene Yates<br>All Rights Reserved.</p></div></div>`; };
const getMethodologyPage = () => { return `<div class="academic-brief" style="page-break-before: always;"><h2 style="color: #4a148c; border-bottom: 2px solid #ccc; padding-bottom: 5px; margin-top:0; font-size:22px; text-transform:uppercase;">Methodological Principles of Collateral Saturation</h2><p style="font-size:15px; line-height:1.6; color:#333; margin-top:20px;"><b>Collateral Saturation</b> is a lineage-validation method in which autosomal DNA evidence is evaluated at the level of descendant networks rather than isolated matches. A lineage hypothesis is considered reliable when it is supported by sufficient descendant density, replicated across independent branches, and remains stable under perturbation tests.</p><ol style="line-height:1.8;"><li><b>Minimum Descendant Count (PM &ge; 15):</b> The network must have sufficient representation to filter noise.</li><li><b>Branch Independence (BR &ge; 3):</b> Triangulation must occur across distinct, non-overlapping descendent lines.</li><li><b>Dominance Ratio (DR &ge; 1.5):</b> The primary genetic signal must clearly overpower secondary pedigree collapse signals.</li></ol></div>`; };
const getVirtualGroupPage = (testerArray) => { if (testerArray.length <= 1) return ""; let kitsHtml = testerArray.map(t => { let kcode = DATA.participants[t] ? DATA.participants[t].kit_code : ''; return `<li>${t} ${kcode ? `[${kcode}]` : ''}</li>`; }).join(""); return `<div class="academic-brief" style="page-break-before: always;"><h2 style="color: #4a148c; border-bottom: 2px solid #ccc; padding-bottom: 5px; margin-top:0; font-size:22px; text-transform:uppercase;">Virtual Group Protocol</h2><p style="font-size:15px; line-height:1.6;">In cases where isolated DNA kits lack sufficient power to achieve Collateral Saturation independently, multiple verifiable descendants of a specific ancestor can be logically joined into a <b>Virtual Group</b>. This protocol aggregates their match networks, treating them as a single proof-grade evaluation unit to reconstruct the older lineage.</p><h3 style="margin-top:20px; font-size:16px;">Kits Formally Merged for this Analysis (${testerArray.length}):</h3><ul style="line-height:1.6; color:#111;">${kitsHtml}</ul></div>`; };

function getMatrixHTML(selectedKits) {
    let matrixRows = PRECOMPUTED.filter(r => selectedKits.includes(r.pName));
    let html = "";

    matrixRows.sort((a,b) => b.ANCHOR - a.ANCHOR);
    let t1 = `<table class="brief-table"><thead><tr style="background:#fff3e0;"><th style="text-align:left; color:#d84315;">Participant Kit</th><th>Primary Ancestor</th><th>DNA Score</th><th>Paper Score</th><th style="background:#d84315; color:white;">ANCHOR Score</th></tr></thead><tbody>`;
    matrixRows.forEach(r => { let nameFmt = r.isGroup ? `<span style="color:#d84315; font-weight:bold;">★ ${r.pName}</span>` : `<strong>${r.pName}</strong>`; t1 += `<tr><td style="text-align:left;">${nameFmt}</td><td>${r.targetAnc}</td><td style="color:#4a148c; font-weight:bold;">${r.cssFinal.toFixed(1)}</td><td style="color:#5d4037; font-weight:bold;">${r.DOCS.toFixed(1)}</td><td style="background:#fbe9e7; font-weight:bold; color:#d84315; font-size:1.1em;">${r.ANCHOR.toFixed(2)}</td></tr>`; });
    t1 += `</tbody></table>`;
    html += `<div class="academic-brief matrix-page"><div class="brief-section-title">Master ANCHOR Evaluation Matrix</div>${t1}</div>`;

    matrixRows.sort((a,b) => b.cssFinal - a.cssFinal);
    let t2 = `<table class="brief-table"><thead><tr style="background:#f3e5f5;"><th style="text-align:left; color:#4a148c;">Participant Kit</th><th>PM</th><th>HC-T</th><th>DR</th><th>BR</th><th>NS</th><th>Status</th><th style="background:#4a148c; color:white;">CSS Score</th></tr></thead><tbody>`;
    matrixRows.forEach(r => { let nameFmt = r.isGroup ? `<span style="color:#d84315; font-weight:bold;">★ ${r.pName}</span>` : `<strong>${r.pName}</strong>`; let status_color = r.ST_str === "PASS" ? "green" : (r.ST_str === "PARTIAL" ? "orange" : "red"); let status_badge = `<span style="color:${status_color}; font-weight:bold;">${r.ST_str}</span>`; t2 += `<tr><td style="text-align:left;">${nameFmt}</td><td>${r.PM}</td><td>${r.HC_T}</td><td>${r.DR.toFixed(1)}</td><td>${r.BR}</td><td>${r.NS}</td><td>${status_badge}</td><td style="background:#f3e5f5; font-weight:bold; color:#4a148c; font-size:1.1em;">${r.cssFinal.toFixed(2)}</td></tr>`; });
    t2 += `</tbody></table>`;
    html += `<div class="academic-brief matrix-page"><div class="brief-section-title">Genetic Evidence (CSS) Matrix</div>${t2}</div>`;

    matrixRows.sort((a,b) => b.DOCS - a.DOCS);
    let t3 = `<table class="brief-table"><thead><tr style="background:#efebe9;"><th style="text-align:left; color:#3e2723;">Participant Kit</th><th>Generations</th><th>Vital Completeness</th><th>Citation Coverage</th><th>Temporal Plausibility</th><th style="background:#5d4037; color:white;">DOCS Score</th></tr></thead><tbody>`;
    matrixRows.forEach(r => { let nameFmt = r.isGroup ? `<span style="color:#d84315; font-weight:bold;">★ ${r.pName}</span>` : `<strong>${r.pName}</strong>`; let vts = `${((r.BC + r.DC)/2 * 100).toFixed(0)}%`; let cts = `${(r.CC * 100).toFixed(0)}%`; let tps = `${(r.TP * 100).toFixed(0)}%`; t3 += `<tr><td style="text-align:left;">${nameFmt}</td><td>${r.GD}</td><td>${vts}</td><td>${cts}</td><td>${tps}</td><td style="background:#efebe9; font-weight:bold; color:#5d4037; font-size:1.1em;">${r.DOCS.toFixed(1)}</td></tr>`; });
    t3 += `</tbody></table>`;
    html += `<div class="academic-brief matrix-page"><div class="brief-section-title">Documentary Evidence (DOCS) Matrix</div>${t3}</div>`;

    return html;
}

const getAppendixA = () => { return `<div class="academic-brief matrix-page" style="max-width: 1200px; text-align: left;"><h2 style="color: #4a148c; border-bottom: 2px solid #ccc; padding-bottom: 5px; margin-top:0; font-size:22px; text-transform:uppercase;">Appendix A (Addendum): ANCHOR Documentary Evidence Matrix (GEDCOM-Derived)</h2><p style="font-size:14px;"><b>Purpose:</b> These fields quantify documentary pedigree robustness using information commonly available in a standard GEDCOM (vital events, places, sources/citations, and plausibility checks). These measures are combined into <b>DOCS</b> (Documentary Score), which is then blended with <b>CSS v2a</b> to produce the combined <b>ANCHOR</b> score.</p><h3 style="color: #4a148c; margin-top: 30px;">A5. ANCHOR Documentary Matrix Fields</h3><table class="brief-table" style="font-size: 13px;"><thead><tr><th>Field</th><th>Abbrev</th><th>Definition</th><th>Computation</th><th>Desired Range</th></tr></thead><tbody><tr><td><b>Apex Reach</b></td><td>AX</td><td>Whether the participant’s documented spine reaches the target ancestral node (prevents pedigree-depth bias).</td><td>AX = 1 if targetID is present in participant t_ids; else 0 (gate/penalty).</td><td><b>Must be 1</b> for proof-grade documentary scoring.</td></tr><tr><td><b>GEDCOM Depth</b></td><td>GD</td><td>Generations/person-nodes in the scored spine segment (participant &rarr; target node).</td><td>GD = count(spineIDs) (log-capped normalization).</td><td>8–14 typical; higher is better if sourced.</td></tr><tr><td><b>Node Coverage</b></td><td>NC</td><td>Fraction of spine persons meeting minimum documentation presence.</td><td>NC = (# persons with any vital/place/source) / (# persons scored).</td><td>&ge;0.70 good; &ge;0.85 strong.</td></tr><tr><td><b>Birth Completeness</b></td><td>BC</td><td>Completeness of birth facts across the spine.</td><td>Per person: 1.0 if birth date+place; 0.6 if one present; 0 if none; BC=mean.</td><td>&ge;0.60 good; &ge;0.80 strong.</td></tr><tr><td><b>Death Completeness</b></td><td>DC</td><td>Completeness of death facts across the spine.</td><td>Per person: 1.0 if death date+place; 0.6 if one present; 0 if none; DC=mean.</td><td>&ge;0.50 good; &ge;0.75 strong.</td></tr><tr><td><b>Date Quality</b></td><td>DQ</td><td>Precision/quality of dates (prefers exactness; penalizes ambiguity).</td><td>Full date (YYYY-MM-DD) &gt; month/year &gt; year-only &gt; qualified (ABT/BEF/AFT) &gt; missing; DQ=mean.</td><td>&ge;0.60 good; &ge;0.80 strong.</td></tr><tr><td><b>Place Quality</b></td><td>PQ</td><td>Granularity/quality of place strings (more locality levels = stronger).</td><td>&ge;3 comma-levels=1.0; 2=0.7; 1=0.4; none=0; PQ=mean.</td><td>&ge;0.60 good; &ge;0.80 strong.</td></tr><tr><td><b>Geo Consistency</b></td><td>GC</td><td>Stability/coherence of locations across adjacent generations (flags implausible “jumps”).</td><td>GC = fraction of adjacent gen-pairs sharing at least one place token (county/state/country heuristic).</td><td>&ge;0.60 good; &ge;0.75 strong.</td></tr><tr><td><b>Citation Coverage</b></td><td>CC</td><td>Fraction of spine persons with &ge;1 citation/source (core proof requirement).</td><td>CC = (# persons with sources/citations) / (# persons scored).</td><td>&ge;0.50 good; &ge;0.70 strong.</td></tr><tr><td><b>Source Density</b></td><td>SD</td><td>Overall density of sources/citations across the scored spine.</td><td>SD = log_norm(total_sources + total_citations, cap&approx;50).</td><td>&ge;0.40 good; &ge;0.65 strong.</td></tr><tr><td><b>Temporal Plausibility</b></td><td>TP</td><td>Sanity checks for time realism (birth/death order, lifespan bounds, generation spacing).</td><td>TP = 1 &minus; (violations / checks), clipped 0–1.</td><td>&ge;0.85 good; &ge;0.95 strong.</td></tr><tr><td><b>Identity Collision Multiplier</b></td><td>IDm</td><td>Penalty when spine contains duplicate IDs or internal contradictions (signals GEDCOM hygiene issues).</td><td>IDm = 1.0 (none); 0.85 (minor); 0.70 (severe).</td><td>Desired 1.0.</td></tr></tbody></table><h3 style="color: #4a148c; margin-top: 30px;">A6. Composite Documentary Score</h3><p style="font-size:14px;"><b>DOCS</b> is a 0–100 composite that summarizes documentary strength:</p><table class="brief-table" style="font-size: 13px;"><thead><tr><th>Composite</th><th>Abbrev</th><th>Definition</th><th>Computation</th><th>Interpretation</th></tr></thead><tbody><tr><td><b>Documentary Score</b></td><td>DOCS</td><td>Normalized, weighted documentary robustness of the scored spine (GEDCOM-derived).</td><td>DOCS = 100 &times; weighted_mean(AX, GD, NC, BC, DC, DQ, PQ, GC, CC, SD, TP) &times; IDm<br><span style="color:#555;">Recommended weights: CC=2.0; SD=1.5; TP=1.5; AX=2.0; others=1.0 (v1).</span><br><span style="color:#b71c1c;"><b>AX Gate:</b> if AX=0, DOCS is capped (e.g., &le;35) to prevent strong scores from incomplete pedigrees.</span></td><td>85–100 Platinum; 70–85 Strong; 50–70 Moderate; 35–50 Weak; &lt;35 Insufficient.</td></tr></tbody></table><h3 style="color: #4a148c; margin-top: 30px;">A7. ANCHOR Combined Score (DNA + Documentary)</h3><table class="brief-table" style="font-size: 13px;"><thead><tr><th>Composite</th><th>Abbrev</th><th>Definition</th><th>Computation</th><th>Interpretation</th></tr></thead><tbody><tr><td><b>ANCHOR Score</b></td><td>ANCH</td><td>Combined proof-strength score blending CSS v2a (DNA network evidence) with DOCS (GEDCOM documentary evidence).</td><td><b>v1 (simple):</b> ANCH = 0.65 &times; CSSv2a + 0.35 &times; DOCS<br><b>v1b (with synergy):</b> ANCH = min(100, blend + 10 &times; (min(CSSv2a, DOCS)/100))<br><span style="color:#555;">Synergy rewards agreement (high DNA + high documentary support).</span></td><td>Higher scores indicate both a saturated genetic network and a robust documented spine. ANCH is intended for cross-participant ranking and publication-grade reporting.</td></tr></tbody></table><p style="font-size:12px; color:#666; margin-top:18px;"><b>Implementation note:</b> DOCS uses only fields commonly found in GEDCOM (birth/death facts, places, sources/citations). Where GEDCOM detail is sparse, DOCS will appropriately remain low, preventing documentary overconfidence.</p></div>`; };
window.runConsolidator = function(mode) { if(mode === 'matrix') { const boxes = document.querySelectorAll('.vg-checkbox:checked'); const selectedKits = Array.from(boxes).map(b => b.value); let customName = document.getElementById('customGroupName').value.trim(); if (customName === "") customName = null; document.title = "Academic_White_Paper"; let reportHTML = getTitlePage(selectedKits, customName) + getMethodologyPage() + getVirtualGroupPage(selectedKits) + getMatrixHTML(selectedKits) + getAppendixA(); document.getElementById('report-container').innerHTML = reportHTML; setTimeout(() => { if(window.init) window.init(); }, 100); } }
</script>"""

# 🌟 3. PROOF ENGINE UI 🌟
PROOF_ENGINE_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Forensic Proof Engine</title><style>__CSS_BASE__ .search-tabs{display:flex;justify-content:center;flex-wrap:wrap;gap:10px;margin-bottom:25px} .search-tab{padding:12px 20px;border:2px solid #ddd;background:#fff;cursor:pointer;font-weight:bold;border-radius:25px;color:#555;font-size:15px;transition:all 0.2s;box-shadow:0 2px 4px rgba(0,0,0,0.05)} .search-tab:hover{background:#f4f4f4;transform:translateY(-2px)} .search-tab.active-part{border-color:#4a148c;background:#4a148c;color:white;box-shadow:0 4px 10px rgba(74,20,140,0.3)} .search-tab.active-anc{border-color:#006064;background:#006064;color:white;box-shadow:0 4px 10px rgba(0,96,100,0.3)} .search-tab.active-id{border-color:#b71c1c;background:#b71c1c;color:white;box-shadow:0 4px 10px rgba(183,28,28,0.3)} .search-box{display:flex;gap:10px;justify-content:center;margin-bottom:20px} .search-box select,.search-box input{padding:15px;width:100%;max-width:400px;border:2px solid #ccc;border-radius:8px;font-size:16px;box-shadow:inset 0 1px 3px rgba(0,0,0,0.05)} .search-box button{padding:15px 25px;background:#b71c1c;color:white;border:none;border-radius:8px;cursor:pointer;font-weight:bold;font-size:16px;transition:all 0.2s} .search-box button:hover{background:#d32f2f;transform:scale(1.05)}</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#4a148c;">🔬 Forensic Proof Engine</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card no-print" style="border-top: 5px solid #4a148c;"><div class="search-tabs"><button id="tab-part" class="search-tab" onclick="setMode('part')">👤 1. Participant View</button><button id="tab-anc" class="search-tab" onclick="setMode('anc')">🌳 2. Ancestor View</button><button id="tab-id" class="search-tab" onclick="setMode('id')">🔍 3. Deep Path Dive (ID#)</button></div><div id="search-ui"></div></div><div class="proof-card" id="proof-result" style="display:none; margin-top:20px; padding:0; border:none; box-shadow:none; background:transparent;"></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
window.makeCascade = function(lin) { let parts = String(lin).split('->'); let h = '<div style="text-align:left; font-size:13px; line-height:1.6; font-family:\'Georgia\',serif; margin:8px 0;">'; parts.forEach((p, i) => { let pad = i * 15; let prfx = i === 0 ? '' : '&uarr; '; let fw = (i === 0) ? 'font-weight:bold; color:#000;' : 'color:#444;'; h += `<div style="margin-left:${pad}px; ${fw}">${prfx}${p.trim()}</div>`; }); h += '</div>'; return h; };
let mode = 'part'; function setMode(newMode) { mode = newMode; document.getElementById('tab-part').className = 'search-tab' + (mode==='part'?' active-part':''); document.getElementById('tab-anc').className = 'search-tab' + (mode==='anc'?' active-anc':''); document.getElementById('tab-id').className = 'search-tab' + (mode==='id'?' active-id':''); document.getElementById('proof-result').style.display = 'none'; let html = ''; if(mode === 'part') { const keys = Object.keys(DATA.participants).sort((a,b) => { let keyA = DATA.participants[a].sort_key || a; let keyB = DATA.participants[b].sort_key || b; return keyA.toLowerCase().localeCompare(keyB.toLowerCase()); }); html = `<div class="search-box"><select id="querySelect" onchange="runSearch()" style="border-color:#4a148c;"><option value="">-- Select a Study Participant --</option>`; keys.forEach(k => { let code = DATA.participants[k].kit_code || ''; let dStr = code ? `${DATA.participants[k].name} [${code}]` : DATA.participants[k].name; html += `<option value="${k}">${dStr}</option>`; }); html += `</select></div>`; } else if(mode === 'anc') { const keys = Object.keys(DATA.ancestors).sort((a,b) => DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name)); html = `<div class="search-box"><select id="querySelect" onchange="runSearch()" style="border-color:#006064;"><option value="">-- Select an Ancestral Line --</option>`; keys.forEach(k => { html += `<option value="${k}">${DATA.ancestors[k].name} (${DATA.ancestors[k].matches} proper matches)</option>`; }); html += `</select></div>`; } else if(mode === 'id') { html = `<div class="search-box"><input type="text" id="queryInput" placeholder="Enter GEDCOM ID (e.g. I1234) or Name..." style="border-color:#b71c1c;" onkeypress="if(event.key==='Enter') runSearch()"> <button onclick="runSearch()">Deep Search</button></div>`; } document.getElementById('search-ui').innerHTML = html; }
window.runSearch = function() { let resDiv = document.getElementById('proof-result'); let html = ''; if(mode === 'part') { let k = document.getElementById('querySelect').value; if(!k) { resDiv.style.display='none'; return; } let p = DATA.participants[k]; let cStat = p.css_status ? p.css_status : 'UNSCORED'; let cssColor = cStat === 'PASS' ? '#2e7d32' : (cStat === 'PARTIAL' ? '#ef6c00' : '#c62828'); html = `<div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1);"><div style="background:#f3e5f5; padding:20px; border-radius:8px; border-left:5px solid #ab47bc; margin-bottom:20px;"><h2 style="margin-top:0; color:#4a148c;">Participant Profile: ${p.name}</h2><p><strong>Status:</strong> <span class="badge" style="background:#4a148c; color:white;">${p.badge}</span> &nbsp; <strong>CSSv2 Validation:</strong> <span class="badge" style="background:${cssColor}; color:white;">${cStat}</span></p><p><strong>Total Evidence Mass:</strong> ${p.cm} cM corroborating ${p.matches} node connections.</p></div><h3 style="color:#4a148c; border-bottom:2px solid #ccc; padding-bottom:5px;">Confirmed Ancestral Intersections</h3><table class="brief-table"><thead><tr><th>Target Node</th><th>Shared cM</th><th>Participant's Triangulation Path</th></tr></thead><tbody>`; let matches = DB.filter(m => m.participant === p.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0)); matches.forEach(m => { html += `<tr><td style="width:25%;"><strong>${m.ancestor}</strong></td><td style="width:10%; color:#4a148c; font-weight:bold;">${m.cm} cM</td><td>${makeCascade(m.lineage)}</td></tr>`; }); html += `</tbody></table></div>`; } else if(mode === 'anc') { let k = document.getElementById('querySelect').value; if(!k) { resDiv.style.display='none'; return; } let a = DATA.ancestors[k]; html = `<div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1);"><div style="background:#e0f7fa; padding:20px; border-radius:8px; border-left:5px solid #00acc1; margin-bottom:20px;"><h2 style="margin-top:0; color:#006064;">Biological Proof: ${a.name}</h2><p><strong>Forensic Validation:</strong> <span class="badge badge-${a.badge.toLowerCase()}">${a.badge} Standard</span></p><p><strong>Integrity Score:</strong> ${a.integrity}% (Verified by ${a.testers} independent kits)</p><p><strong>Total Evidence:</strong> ${a.cm} cM shared across ${a.matches} matching paths.</p></div><table class="brief-table"><thead><tr><th>Matching Kit</th><th>Shared cM</th><th>Documented Lineage Path</th></tr></thead><tbody>`; let matches = DB.filter(m => m.ancestor === a.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0)); matches.forEach(m => { html += `<tr><td style="width:25%;"><strong>${m.participant}</strong></td><td style="width:10%; color:#006064; font-weight:bold;">${m.cm} cM</td><td>${makeCascade(m.lineage)}</td></tr>`; }); html += `</tbody></table></div>`; } else if(mode === 'id') { let q = document.getElementById('queryInput').value.trim().toLowerCase(); if(!q) { resDiv.style.display='none'; return; } let qNum = q.replace(/[^0-9]/g, ''); let matches = DB.filter(m => { if(qNum && m.search_ids && m.search_ids.split(',').map(x=>x.replace(/[^0-9]/g,'')).includes(qNum)) return true; if(m.lineage && m.lineage.toLowerCase().includes(q)) return true; if(m.id && m.id.toLowerCase().includes(q)) return true; return false; }).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0)); html = `<div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1);"><div style="background:#ffebee; padding:20px; border-radius:8px; border-left:5px solid #b71c1c; margin-bottom:20px;"><h2 style="margin-top:0; color:#b71c1c;">Deep Path Search: "${q}"</h2><p style="font-size:16px;">Found <strong>${matches.length}</strong> lineage connections passing through this node.</p></div>`; if(matches.length > 0) { html += `<table class="brief-table"><thead><tr><th style="color:#b71c1c;">Participant</th><th style="color:#b71c1c;">Primary Target</th><th style="color:#b71c1c;">cM</th><th style="color:#b71c1c;">Intersecting Lineage Path</th></tr></thead><tbody>`; matches.forEach(m => { let hlPath = m.lineage; if(qNum) { let regex = new RegExp(`\\(I?${qNum}\\)`, 'gi'); hlPath = hlPath.replace(regex, match => `<mark style="background:#ffcdd2; color:#b71c1c; font-weight:bold; padding:2px 4px; border-radius:3px;">${match}</mark>`); } else if (q.length > 3) { let regex = new RegExp(`(${q})`, 'gi'); hlPath = hlPath.replace(regex, match => `<mark style="background:#ffcdd2; color:#b71c1c; font-weight:bold; padding:2px 4px; border-radius:3px;">${match}</mark>`); } html += `<tr><td style="width:20%;"><strong>${m.participant}</strong></td><td style="width:20%;"><strong>${m.ancestor}</strong></td><td style="width:5%; color:#b71c1c; font-weight:bold;">${m.cm}</td><td>${makeCascade(hlPath)}</td></tr>`; }); html += `</tbody></table></div>`; } else { html += `<p style="text-align:center; color:#777; font-style:italic;">No matching kits found passing through this ID or name. Note: Uncorroborated singletons may not appear in this index.</p></div>`; } } resDiv.innerHTML = html; resDiv.style.display = 'block'; } document.addEventListener('DOMContentLoaded', () => setMode(mode));
</script>"""

# 🌟 4. DOSSIER UI (WITH LIVE AUDIT HOOKS) 🌟
DOSS_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Forensic Dossier</title><style>__CSS_BASE__</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#004d40;">📁 Forensic Dossier</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div id="dos-ui" class="no-print"></div><div id="report-stack"></div></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
window.makeCascade = function(lin) { let parts = String(lin).split('->'); let h = '<div style="text-align:left; font-size:13px; line-height:1.6; font-family:\'Georgia\',serif; margin:8px 0;">'; parts.forEach((p, i) => { let pad = i * 15; let prfx = i === 0 ? '' : '&uarr; '; let fw = (i === 0) ? 'font-weight:bold; color:#000;' : 'color:#444;'; h += `<div style="margin-left:${pad}px; ${fw}">${prfx}${p.trim()}</div>`; }); h += '</div>'; return h; };
document.addEventListener('DOMContentLoaded', function() {
    const dosKeys = Object.keys(DATA.participants).sort((a,b) => { let keyA = DATA.participants[a].sort_key || a; let keyB = DATA.participants[b].sort_key || b; return keyA.toLowerCase().localeCompare(keyB.toLowerCase()); });
    let sel = '<select id="dosSelect" onchange="renderDossier()" style="padding:12px; width:100%; border:2px solid #004d40; border-radius:4px; font-size:16px; margin-bottom:20px;"><option value="">-- Select Kit to Generate Formal Dossier --</option>'; dosKeys.forEach(k => { let code = DATA.participants[k].kit_code || ''; let displayStr = code ? `${DATA.participants[k].name} [${code}]` : DATA.participants[k].name; sel += `<option value="${k}">${displayStr}</option>`; }); sel += '</select>'; document.getElementById('dos-ui').innerHTML = sel;

    window.renderDossier = function() {
        let k = document.getElementById('dosSelect').value; if(!k) { document.getElementById('report-stack').innerHTML = ''; return; }
        let p = DATA.participants[k];
        let matches = DB.filter(m => m.participant === p.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0));
        let topAnc = matches.length > 0 ? matches[0].ancestor : "None Found";

        let cStat = p.css_status ? p.css_status : 'UNSCORED';
        let cssColor = cStat === 'PASS' ? '#2e7d32' : (cStat === 'PARTIAL' ? '#ef6c00' : '#c62828');

        let topAncDisplay = topAnc;
        if(p.target_id && p.target_id !== 'None' && p.target_id !== '') {
            let pureId = String(p.target_id).replace(/[^0-9]/g, '');
            topAncDisplay = `<a href="https://yates.one-name.net/tng/verticalchart.php?personID=I${pureId}&tree=tree1&parentset=0&display=vertical&generations=15" target="_blank" style="color:#004d40; text-decoration:underline;">${topAnc}</a>`;
        }

        let diagHtml = '';
        if(p.diag_log && p.diag_log.length > 0) {
            diagHtml = `<div style="margin-top:20px; background:#fff3e0; padding:15px; border-left:4px solid #ff9800;">
                <h4 style="margin-top:0; color:#e65100;">Pedigree Audit Flags (DOCS Score: ${parseFloat(p.docs_score).toFixed(1)})</h4>
                <ul style="margin-bottom:0; font-size:14px; color:#333;">
                    ${p.diag_log.map(msg => `<li style="margin-bottom:5px;">${msg}</li>`).join('')}
                </ul>
            </div>`;
        } else if (p.docs_score >= 85) {
             diagHtml = `<div style="margin-top:20px; background:#e8f5e9; padding:15px; border-left:4px solid #4caf50;">
                <h4 style="margin-top:0; color:#2e7d32;">Pedigree Audit (DOCS Score: ${parseFloat(p.docs_score).toFixed(1)})</h4>
                <p style="margin:0; font-size:14px; color:#333;">✅ <strong>Platinum Standard Pedigree.</strong> The documented spine is fully sourced with zero temporal anomalies.</p>
            </div>`;
        }

        let html = `<div style="border: 3px double #004d40; padding: 40px; background: white; margin-top:20px;">
            <div style="text-align:center; border-bottom:2px solid #004d40; padding-bottom:20px; margin-bottom:30px;">
                <h1 style="color:#004d40; text-transform:uppercase; margin:0; font-size:28px;">Forensic Evidence Dossier</h1>
                <p style="margin:5px 0 0 0; color:#555; font-style:italic;">Yates DNA Study Lineage Reconstruction</p>
            </div>
            <div style="font-size:16px; line-height:1.8; background:#f4f4f4; padding:20px; border:1px solid #ddd; margin-bottom:30px;">
                <p style="margin:0;"><strong>SUBJECT IDENTIFIER:</strong> ${p.name}</p>
                <p style="margin:0;"><strong>CSSv2 VALIDATION STATUS:</strong> <span style="color:${cssColor}; font-weight:bold;">${cStat}</span></p>
                <p style="margin:0;"><strong>EVIDENCE INTEGRITY SCORE:</strong> ${p.integrity}%</p>
                <p style="margin:0;"><strong>PRIMARY CORROBORATED NODE:</strong> ${topAncDisplay}</p>
                <hr style="border-top:1px solid #ccc; margin:15px 0;">
                <p style="margin:0;"><strong>EXECUTIVE SUMMARY:</strong> This subject shares ${p.cm} cM of autosomal DNA across ${p.matches} independently verified ancestral nodes within the Yates study. The empirical data confirms the biological validity of the subject's descent pathway. This lineage is anchored by a target node possessing a Node Saturation (NS) of <strong>${p.ns}</strong>, validating the connection across <strong>${p.br}</strong> independent descendant branches.</p>
                ${diagHtml}
            </div>
            <h3 style="color:#004d40; text-transform:uppercase; font-size:18px;">Cross-Referenced Match Index</h3>
            <table class="brief-table"><thead><tr><th style="background:#e0f2f1;">Intersected Study Node</th><th style="background:#e0f2f1;">cM</th><th style="background:#e0f2f1;">Documented Route to Node</th></tr></thead><tbody>`;
            matches.forEach(m => { html += `<tr><td style="width:30%;"><strong>${m.ancestor}</strong></td><td style="width:10%; color:#004d40; font-weight:bold;">${m.cm}</td><td>${makeCascade(m.lineage)}</td></tr>`; });
            html += '</tbody></table></div>';
            document.getElementById('report-stack').innerHTML = html;
    }
});</script></body></html>"""

# 🌟 5. EXPLICIT TEXT PLACEHOLDERS 🌟
CONTENTS_CONTENT = r"""<style>.guide-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:25px;max-width:1200px;margin:30px auto;font-family:sans-serif;}.guide-card{background:white;padding:25px;border-radius:8px;border-left:5px solid #006064;box-shadow:0 4px 10px rgba(0,0,0,0.05);transition:transform 0.2s;text-align:left;}.guide-card:hover{transform:translateY(-5px)}.card-title{font-size:1.4em;font-weight:bold;color:#004d40;margin-top:0}.card-why{color:#b71c1c;font-weight:bold;margin:10px 0 5px 0;font-size:0.9em;text-transform:uppercase}.card-what{color:#555;font-size:1em;line-height:1.5;margin-bottom:20px}.card-btn{display:inline-block;padding:10px 20px;background:#00838f;color:white;text-decoration:none;border-radius:4px;font-weight:bold}.card-btn:hover{background:#006064}</style><div style="text-align:center;max-width:800px;margin:0 auto 20px auto;color:#444;font-size:1.1em;font-family:sans-serif;">This site transforms raw DNA data into forensic genealogical evidence.</div><div class="guide-grid"><div class="guide-card"><h2 class="card-title">1. The DNA Register</h2><div class="card-why">Why View This?</div><div class="card-what">To see the raw evidence. This is the master list of all matches in the study, sorted by ancestral line.</div><a href="ons_yates_dna_register.shtml" class="card-btn">Open Register</a></div><div class="guide-card"><h2 class="card-title">2. DNA Network</h2><div class="card-why">Why View This?</div><div class="card-what">To see the big picture. Visual clusters showing which ancestral lines are genetically proven by multiple testers.</div><a href="dna_network.html" class="card-btn">View Network</a></div><div class="guide-card"><h2 class="card-title">3. Lineage Proof Engine</h2><div class="card-why">Why View This?</div><div class="card-what">To verify a connection. An interactive tool that tests if a specific ancestor is biologically confirmed.</div><a href="proof_engine.html" class="card-btn">Run Proof</a></div><div class="guide-card" style="border-left-color:#f57f17;"><h2 class="card-title" style="color:#e65100;">4. Brick Wall Buster</h2><div class="card-why" style="color:#bf360c;">Why View This?</div><div class="card-what">Break through a dead end. Suggests which family line you likely belong to based on match dominance.</div><a href="proof_engine.html" class="card-btn" style="background:#ef6c00;">Bust This Wall</a></div><div class="guide-card"><h2 class="card-title">5. Forensic Dossier</h2><div class="card-why">Why View This?</div><div class="card-what">Get your "Scorecard." Generate forensic reports on yourself or an ancestor, grading evidence strength.</div><a href="dna_dossier.html" class="card-btn">Create Dossier</a></div><div class="guide-card"><h2 class="card-title">6. Research Admin Hub</h2><div class="card-why">Why View This?</div><div class="card-what">For study managers. High-level audit showing participant statistics, masked IDs, and study metrics.</div><a href="research_admin.html" class="card-btn" style="background:#455a64;">Admin Access</a></div><div class="guide-card"><h2 class="card-title">7. Data Glossary</h2><div class="card-why">Why View This?</div><div class="card-what">Understand the terms. Definitions for forensic terms like "Keystone Tester" and "Spanish naming."</div><a href="data_glossary.shtml" class="card-btn" style="background:#78909c;">Read Glossary</a></div><div class="guide-card" style="border-left-color:#fbc02d;"><h2 class="card-title">8. Corrections</h2><div class="card-why">See Something?</div><div class="card-what">Genealogy is collaboration. If you can solve a mystery, tell us. Include the <strong>Person ID (e.g. I1234)</strong>.</div><a href="mailto:ron@yates.one-name.net" class="card-btn" style="background:#f9a825;color:#333;">Email Correction</a></div></div>"""
SHARE_CONTENT = r"""<div style="max-width:900px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="background:white; padding:40px; border-radius:8px; box-shadow:0 2px 10px rgba(0,0,0,0.1);"><h2 style="color:#006064; margin-top:0; border-bottom:2px solid #00838f; padding-bottom:10px;">Share Your Ancestry DNA Matches</h2><p style="font-size:1.1em; line-height:1.6;">Ancestry provides a built-in sharing feature that allows you to grant limited access to your DNA matches <strong>without sharing your personal account details</strong>. You remain in full control of your account at all times.</p><h3 style="color:#00838f; margin-top:30px;">How sharing works on Ancestry</h3><p>From your AncestryDNA Settings page, you may invite another individual by email and assign them the <strong>Collaborator</strong> role. This allows the study administrator to analyze your match list to build Forensic Handshakes.</p><div style="background:#f1f8e9; padding:25px; border-radius:8px; border:1px solid #c5e1a5; margin-top:30px;"><h3 style="color:#33691e; margin-top:0;">How to share for the Yates One-Name Study</h3><ol style="line-height:1.8; font-size:1.1em; color:#333;"><li>Open your <strong>AncestryDNA Settings</strong>.</li><li>Use the <strong>Invite</strong> function.</li><li>Send the invitation to <strong>Ron Yates</strong> at: <br><span style="font-family:monospace; font-weight:bold; background:white; padding:5px 10px; border:1px solid #aaa; border-radius:3px; display:inline-block; margin-top:5px; color:#c62828;">yatesvilleron@gmail.com</span></li><li>Assign the role <strong>Collaborator</strong>.</li></ol><p style="font-size:0.95em; color:#558b2f; margin-top:15px; background:white; padding:10px; border-radius:4px; border:1px solid #dcedc8;"><strong>Why Collaborator?</strong> The <em>Collaborator</em> role allows Ron to review shared matches to perform triangulation logic.</p></div></div></div>"""
SUBSCRIBE_CONTENT = r"""<div style="background:white; padding:40px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1); max-width:800px; margin:20px auto; text-align:center; font-family:sans-serif;"><h2 style="color:#006064; margin-top:0;">Join the Yates Research Community</h2><div style="background:#e0f2f1; padding:25px; border-radius:8px; border:1px solid #b2dfdb; display:inline-block;"><h3 style="margin-top:0; color:#004d40;">📧 One-Click Subscribe</h3><p>Participation in the Yates DNA Study is completely free.</p><a href="mailto:yates-one-name-study+subscribe@groups.io?subject=Subscribe" style="display:inline-block; padding:15px 30px; background:#00838f; color:white; text-decoration:none; border-radius:5px; font-weight:bold; margin-top:10px;">Subscribe Now</a></div></div>"""
GLOSSARY_INLINE = r"""<div style="max-width:1100px; margin:20px auto; font-family:sans-serif; color:#333;"><h2 style="color:#006064;border-bottom:2px solid #004d40;padding-bottom:10px; margin-bottom:20px;">ONS Yates Study: Data Glossary</h2><details open style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">1. Core Concepts (Plain English)</span></summary><div style="padding:15px; line-height:1.6;"><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Ancestral Node:</strong><br>A "Node" is simply a specific ancestor on a family tree where different descendant lines intersect. If you think of a family tree like a roadmap, a node is the exact intersection where the paths of multiple DNA cousins meet. Proving a node confirms that the historical ancestor actually existed and passed down identifiable DNA.</li><li style="margin-bottom:15px;"><strong>Forensic Handshake:</strong><br>A Forensic Handshake happens when two or more DNA testers, who do not know each other and descend from different branches, all share DNA that points back to the exact same Ancestral Node. It acts as a genetic cross-reference that proves the paper trail is real. One match is a hint; a 'handshake' between multiple independent matches is proof.</li><li style="margin-bottom:15px;"><strong>Platinum Standard:</strong><br>Lineages that have achieved undeniable biological and documentary proof (30+ matches and 10+ unique sources).</li><li style="margin-bottom:15px;"><strong>Keystone Tester:</strong><br>A high-value study participant (with 15+ matches) whose DNA heavily anchors a specific branch of the family tree.</li></ul></div></details><details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">2. Identity Columns</span></summary><div style="padding:15px; line-height:1.6;"><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Tester-Participant-MASKED (The Trigger):</strong><br>The unique privacy code extracted from the user's NPFX tag.</li><li style="margin-bottom:15px;"><strong>Tester-Participant-Unmasked:</strong><br>The real name of the tester.</li></ul></div></details><details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">3. Genealogy Terms</span></summary><div style="padding:15px; line-height:1.6;"><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Spanish Naming System:</strong><br>A traditional Hispanic naming convention in which an individual bears one or more given names followed by two surnames: the first inherited from the father (paternal surname) and the second from the mother (maternal surname). This system is historically rooted in Spain and is especially useful in genealogy because it preserves both parental lineages and improves identification in historical records.</li><li style="margin-bottom:15px;"><strong>Née:</strong><br>A term meaning “born as,” used to indicate a woman’s maiden or birth surname before marriage. In genealogical and historical records, née identifies the surname a woman carried in her natal family line, preserving her connection to her parents and ancestry. For example, “Maria García, née López” shows that María’s birth surname was López, even though she later used García after marriage.</li></ul></div></details></div>"""
GEDMATCH_INLINE = r"""<style>.ged-table td { padding: 12px; border-bottom: 1px solid #ddd; font-size: 15px; } .ged-table a { color: #00838f; font-weight: bold; text-decoration: none; } .ged-table a:hover { text-decoration: underline; color: #006064; } .ged-table tbody tr:hover { background-color: #f9f9f9; }</style><div style="max-width:900px; margin:20px auto; font-family:sans-serif; color:#333;"><div style="background:white;padding:40px;border-radius:8px;border:1px solid #ddd;box-shadow:0 4px 10px rgba(0,0,0,0.05);"><h2 style="color:#01579b;border-bottom:2px solid #03a9f4;padding-bottom:10px;margin-top:0;">GEDmatch Hub: Known Yates Kits</h2><p style="font-size:1.1em; line-height:1.6; color:#555; margin-bottom:30px;">This registry contains the known GEDmatch kit numbers for study participants. Use these IDs to perform one-to-one segment comparisons.</p><table class="brief-table sortable ged-table" style="width:100%; border-collapse:collapse; border:1px solid #ddd;"><thead><tr style="background:#eceff1;"><th style="padding:12px; text-align:left; border-bottom:2px solid #000; color:#263238; width: 40%; cursor:pointer;">GEDmatch Kit #</th><th style="padding:12px; text-align:left; border-bottom:2px solid #000; color:#263238; cursor:pointer;">Participant Name</th></tr></thead><tbody><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">F201688</a></td><td>(Y-35 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M024169</a></td><td>(Y-44 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M040727</a></td><td>(Y-44 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M673507</a></td><td>(Y-44 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">FR6532899</a></td><td>Adams, Sarah Sally</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M183543</a></td><td>Baig, Natalie</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A696433</a></td><td>Barnes-2</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A630666</a></td><td>Broms, Mary Beth</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">QW1447502</a></td><td>Crownover, Kathy Van Pelt</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T220912</a></td><td>Dallys E (Natalie Baig mother)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A493108</a></td><td>Girtain-Yates, Alma</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A849238</a></td><td>Girtain-Yates, Andy</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A004522</a></td><td>Girtain-Yates, Kathryn</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A214154</a></td><td>Godwin, Alta Barnes</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A291753</a></td><td>Laswell, Jack</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T422899</a></td><td>Leicher, John</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A323108</a></td><td>Lindsey</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">BW4660858</a></td><td>Little, Ilene</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M722226</a></td><td>McCollum, Michael</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">GS8478007</a></td><td>Moore, Wright</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T648223</a></td><td>Reddoch, James A. (FTDNA)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A145010</a></td><td>Russett, Andrea Yates</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A507391</a></td><td>Sopp, Margaret</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A303537</a></td><td>Tabor, Sudie</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A909608</a></td><td>Varapodio, Joyce</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">KQ5739791</a></td><td>Wishard, Glenn</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">F316112</a></td><td>Yates, Abraham</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T810459</a></td><td>Yates, Arthur Lewis</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T749670</a></td><td>Yates, Benjamin</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T153410</a></td><td>Yates, Charlie Martin</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T977010</a></td><td>Yates, Howard Garrison</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A061248</a></td><td>Yates, James Robert</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">XE6552308</a></td><td>Yates, James Taos</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A131894</a></td><td>Yates, John</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A546217</a></td><td>Yates, John</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T608107</a></td><td>Yates, John F., Jr.</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A822241</a></td><td>Yates, John H.</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">F227484</a></td><td>Yates, John Henry</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T159617</a></td><td>Yates, Patricia Lynn</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">DV4371781</a></td><td>Yates, Robert David</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A861466</a></td><td>Yates, Ron</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">DN2421283</a></td><td>Yates, Ron</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">MA8197080</a></td><td>Yates, Ronald Eugene</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M735337</a></td><td>Yates, Steph Solyon</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M407025</a></td><td>Yates, Timothy Brian</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M441343</a></td><td>Yates, Timothy Joe</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A139695</a></td><td>YatesJr, James Carey</td></tr></tbody></table></div></div>"""
THEORY_PAGE_CONTENT = r"""<div class="proof-card" style="max-width: 1000px; margin: 20px auto; text-align: left; font-family: 'Georgia', serif; line-height: 1.8; font-size: 16px; color: #333;">
    <h1 style="color:#006064; border-bottom:2px solid #004d40; padding-bottom:10px; margin-top:0; font-family: 'Segoe UI', sans-serif;">Yates DNA Study</h1>
    <h2 style="color:#00838f; font-size:1.3em; font-family: 'Segoe UI', sans-serif;">Theory of the Case — Origin and Methodological Development</h2>
    <p>The Yates DNA Study began as an effort to reconstruct ancestral lineages using autosomal DNA matches and traditional genealogical research. Over time, this work developed into a structured methodology for lineage reconstruction based on the convergence of independent genetic and documentary evidence.</p>
    <p>The central premise of the study is that ancestral correctness becomes increasingly probable as independent DNA-derived lines converge on the same upstream ancestral couples. This convergence can be observed both visually, through reversed pedigree analysis, and quantitatively through Collateral Saturation metrics.</p>
    <p>The methodology evolved in stages. Early work focused on assembling pedigree lines from DNA matches and identifying repeated ancestors. As the number of participants and matches increased, these observations developed into a formal framework capable of measuring lineage convergence and evaluating lineage reliability.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Methodological Scope</h2>
    <p>The study uses the collateral lines of individuals who share autosomal DNA and who report Yates ancestry or associated family connections. Each line begins with a confirmed DNA match and extends through that match’s documented pedigree.</p>
    <p>Collateral descendants represent siblings and cousins of direct ancestors. Because autosomal DNA is inherited across all ancestral lines, these collateral descendants collectively preserve detectable genetic signals from shared ancestors.</p>
    <p>Each pedigree line is treated as an independent reconstruction derived from a separate DNA match. Lines are extended only when the genealogical evidence remains plausible and consistent with known historical information.</p>
    <p>The method is designed to scale. Individual matches or small clusters may be inconclusive, but network-level evidence becomes increasingly reliable as replication across independent descendant lines increases.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Independent Derivation of Pedigree Lines</h2>
    <p>Each ancestral line used in the study originates from a confirmed DNA match. The pedigree associated with each match is developed independently using documentary sources, shared family trees, and established genealogical research.</p>
    <p>This independence is essential. The repeated appearance of the same ancestral couples across independently derived lines provides evidence of common descent that does not depend on any single pedigree.</p>
    <p>The study therefore avoids reliance on a single master tree. Instead, it builds a network of independently derived pedigrees whose points of agreement provide the strongest evidence of shared ancestry.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Reversed Pedigree Analysis</h2>
    <p>Pedigree lines are analyzed in reversed orientation, beginning with the earliest known ancestors and proceeding forward toward the present.</p>
    <p>When pedigrees are displayed in this form, ancestral couples repeated across multiple independent lines become immediately visible. Repeated blocks of identical names form recognizable patterns across the dataset.</p>
    <p>These repeated patterns represent the convergence of independent descendant lines on common ancestral nodes. The visual appearance of these repeated ancestral blocks was the earliest indication that large-scale autosomal lineage reconstruction might be possible.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Frequency as Evidence</h2>
    <p>Ancestors that appear frequently across independent pedigree lines are interpreted as strong candidates for shared ancestry.</p>
    <p>When fifteen or more independently derived DNA-match lines converge on the same ancestral couple, the probability that this convergence is accidental becomes extremely small. High-frequency ancestral couples therefore represent the most reliable genealogical anchors within the study.</p>
    <p>Lower-frequency ancestors are retained as hypotheses. As additional participants are added to the study, previously rare lines may later become important components of the larger network.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Development of Collateral Saturation</h2>
    <p>As the study expanded, repeated ancestral convergence began to appear as stable network structures rather than isolated observations. This led to the development of the Collateral Saturation method.</p>
    <p>Collateral Saturation evaluates evidence at the level of descendant networks rather than individual matches. Proof-grade conclusions emerge when independent descendant branches replicate the same ancestral relationships across multiple lines.</p>
    <p>The Collateral Saturation Score (CSS v2a) was developed to provide normalized comparison across participants using measurable properties including Proper Matches, Handshake Evidence, Dominance Ratio, Branch Replication, Unique Testers, and Node Saturation.</p>
    <p>In saturated lineages, properties such as Unique Testers and Node Saturation become shared characteristics of the ancestral node itself, while Proper Matches and Dominance Ratio distinguish individual descendants.</p>
    <p>Metric calculations are anchored to the genetically determined target node rather than pedigree depth. This ensures consistent measurement across participants regardless of documentary completeness.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Network-Level Proof</h2>
    <p>Testing demonstrated that Collateral Saturation behaves as a network-level threshold phenomenon. Individual participants may fail proof-grade criteria when evaluated in isolation, yet multiple independently related participants sharing the same ancestral node can combine into a stable proof-grade cluster.</p>
    <p>This "Virtual Group" behavior demonstrates that lineage proof emerges from replicated network evidence rather than from any single participant. The strength of the lineage derives from the network as a whole.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Extension to the ANCHOR Framework</h2>
    <p>Collateral Saturation demonstrated that autosomal DNA networks can identify ancestral lineages through replicated genetic evidence. However, genetic evidence alone does not establish the historical identity of those ancestors.</p>
    <p>The ANCHOR Framework was developed to integrate genetic network evidence with documentary pedigree evidence into a unified lineage reconstruction methodology.</p>
    <p>Within the ANCHOR Framework:</p>
    <ul style="margin-bottom: 20px;">
        <li><b>Collateral Saturation</b> provides the genetic evidence engine.</li>
        <li><b>CSS v2a</b> measures genetic network strength.</li>
        <li><b>DOCS</b> measures documentary pedigree robustness using GEDCOM-derived evidence.</li>
        <li><b>ANCHOR</b> measures the combined strength of genetic and documentary evidence.</li>
    </ul>
    <p>DOCS measures pedigree robustness rather than lineage correctness. Lineage conclusions depend primarily on replicated genetic evidence, while documentary evidence provides historical structure and confirmation.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Practical Conclusion</h2>
    <p>When reversed pedigrees and quantitative scoring highlight ancestors repeated across many independent DNA-match lines, those ancestors become leading candidates for direct lineage placement.</p>
    <p>Lineage reconstruction becomes most reliable when independent genetic networks and documentary pedigrees converge on the same ancestral couples.</p>
    <p>The Yates DNA Study therefore demonstrates a scalable method for lineage reconstruction based on replication, convergence, and network stability.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Alignment with Genealogical Standards</h2>
    <p>This project integrates autosomal DNA evidence with documentary genealogy in accordance with accepted genealogical standards. DNA evidence is evaluated alongside traditional records, and conclusions are tested for sufficiency, replication, and stability.</p>
    <ul>
        <li>Planned comparison of DNA results across independent descendant lines</li>
        <li>Integration of autosomal DNA with documentary evidence</li>
        <li>Cumulative evaluation based on collateral match density and replication</li>
    </ul>
</div>"""
NETWORK_PAGE_CONTENT = r"""<style>
#network-container { width: 100%; height: 750px; border: 2px solid #ccc; border-radius: 8px; background: #fff; box-shadow: inset 0 2px 10px rgba(0,0,0,0.05); }
.network-ui { text-align: center; margin-bottom: 25px; background: #e0f2f1; padding: 20px; border-radius: 8px; border: 1px solid #b2dfdb; }
.network-ui select { padding: 12px; font-size: 16px; border: 2px solid #006064; border-radius: 6px; min-width: 400px; cursor: pointer; }
.vis-tooltip { position: absolute; padding: 10px; background: rgba(0, 0, 0, 0.8) !important; color: white !important; font-family: sans-serif; font-size: 14px; border-radius: 4px; pointer-events: none; z-index: 1000; box-shadow: 0 4px 6px rgba(0,0,0,0.3); }
</style>
<script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>

<div class="proof-card" style="max-width: 1400px;">
    <h2 style="color:#006064; text-align:center; margin-top:0; border-bottom:2px solid #004d40; padding-bottom:10px;">DNA Network Visualizer</h2>
    <p style="text-align:center; font-size:16px; color:#555; margin-bottom:30px;">This interactive force-directed graph visualizes the genetic clusters in the Yates Study. Select an ancestral line below to instantly map its descendant network.</p>

    <div class="network-ui no-print">
        <select id="networkSelect" onchange="drawNetwork()">
            <option value="">-- Select an Ancestral Cluster --</option>
        </select>
    </div>

    <div id="network-container"></div>
</div>

<script>
__JS_GLOBALS__;

let network = null;

document.addEventListener('DOMContentLoaded', function() {
    let sel = document.getElementById('networkSelect');
    const ancKeys = Object.keys(DATA.ancestors).sort((a,b) => DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name));

    ancKeys.forEach(k => {
        if (DATA.ancestors[k].matches >= 2) {
            let opt = document.createElement('option');
            opt.value = k;
            opt.innerHTML = `${DATA.ancestors[k].name} (${DATA.ancestors[k].matches} proper matches)`;
            sel.appendChild(opt);
        }
    });
});

function drawNetwork() {
    let selectedAncKey = document.getElementById('networkSelect').value;
    if(!selectedAncKey) {
        if(network) { network.destroy(); network = null; }
        return;
    }

    let targetAncName = DATA.ancestors[selectedAncKey].name;
    let matches = DB.filter(m => m.ancestor === targetAncName);

    let nodes = new vis.DataSet();
    let edges = new vis.DataSet();

    nodes.add({
        id: targetAncName,
        label: targetAncName.split('&').join('\n&'),
        shape: 'box',
        color: { background: '#006064', border: '#004d40' },
        font: { color: 'white', size: 20, face: 'Georgia', bold: true },
        margin: 15,
        shadow: true,
        title: `<b>${targetAncName}</b><br>Total Matches: ${matches.length}<br>Integrity: ${DATA.ancestors[selectedAncKey].integrity}%`
    });

    let addedTesters = new Set();
    matches.forEach(m => {
        if(!addedTesters.has(m.participant)) {
            let tData = DATA.participants[m.participant];
            let badgeColor = '#ab47bc';
            let statText = "UNSCORED";

            if (tData) {
                if (tData.css_status === 'PASS') { badgeColor = '#2e7d32'; statText = "PASS"; }
                else if (tData.css_status === 'PARTIAL') { badgeColor = '#ef6c00'; statText = "PARTIAL"; }
                else if (tData.css_status === 'FAIL') { badgeColor = '#c62828'; statText = "FAIL"; }
            }

            nodes.add({
                id: m.participant,
                label: m.participant,
                shape: 'dot',
                size: 15,
                color: { background: badgeColor, border: 'white', borderWidth: 2 },
                font: { color: '#333', size: 14, face: 'Arial' },
                shadow: true,
                title: `<b>${m.participant}</b><br>Shared cM: ${m.cm}<br>CSS Status: ${statText}`
            });

            edges.add({
                from: m.participant,
                to: targetAncName,
                label: m.cm + ' cM',
                font: { align: 'middle', size: 12, color: '#777', background: 'white' },
                color: { color: '#ccc', highlight: badgeColor },
                width: 2,
                arrows: 'to'
            });
            addedTesters.add(m.participant);
        }
    });

    let container = document.getElementById('network-container');
    let data = { nodes: nodes, edges: edges };
    let options = {
        physics: {
            forceAtlas2Based: { gravitationalConstant: -100, centralGravity: 0.01, springLength: 200, springConstant: 0.08 },
            maxVelocity: 50, solver: 'forceAtlas2Based', timestep: 0.35, stabilization: { iterations: 150 }
        },
        interaction: { hover: true, tooltipDelay: 200 }
    };
    if(network) { network.destroy(); }
    network = new vis.Network(container, data, options);
}
</script>"""

print("✅ Cell 3 (Template Vault) Loaded Successfully.")

      [CELL 3] TEMPLATE VAULT LOADING...
✅ Cell 3 (Template Vault) Loaded Successfully.


In [39]:
# @title [CELL 4] The Publisher & Uploader
def run_publisher():
    print("="*60)
    print("      [CELL 4] PUBLISHER STARTING...")
    print("="*60)

    import os, json, pytz
    import pandas as pd
    from datetime import datetime
    from ftplib import FTP_TLS
    try:
        from google.colab import userdata
    except ImportError:
        pass

    try:
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
    except Exception as e:
        return print(f"❌ Credential Error: {e}")

    JSON_DB = "compiled_database.json"
    CSV_DB = "engine_database.csv"

    if not os.path.exists(JSON_DB):
        return print("❌ ERROR: compiled_database.json not found. Run Cell 2 first.")

    print("    [+] Loading Pre-Calculated Data...")
    with open(JSON_DB, "r") as f:
        master_data = json.load(f)

    # Convert the raw JSON objects back into the strings Javascript needs
    JS_GLOBALS = f"const PRECOMPUTED_JSON={json.dumps(master_data['PRECOMPUTED'])}; const DATA={json.dumps(master_data['DATA'])}; const DB={json.dumps(master_data['DB'])};"

    # We need the DF just to build the massive HTML tables for the old pages
    df = pd.read_csv(CSV_DB, encoding="iso-8859-15")
    df.fillna('', inplace=True)
    df.replace('nan', '', inplace=True)

    # ✨ FIX: Translate the raw CSV headers into the variable names the Publisher uses
    df.rename(columns={
        "Authority_Directory_Label": "Dir_Label",
        "Tester_Code": "Kit_Code",
        "Match_Lineage": "Lineage",
        "Match_Path_IDs": "s_ids",
        "Tester_Display": "Kit_Name"
    }, inplace=True)

    print("    [+] Building Data Tables...")
    df_valid = df[df['Dir_Label'] != 'No Matches'].copy()
    df_valid['sort_key'] = df_valid['Tester_Sort_Key']
    mc = df_valid['Dir_Label'].value_counts()

    def normalize_id(val): return f"I{str(val).replace('@', '').strip()}" if str(val).replace('@', '').strip().isdigit() else str(val).replace('@', '').strip()

    def format_reg(r):
        m_id, m_name, d_label, kit_name, cm_val = str(r.get("Match_ID", "")), str(r.get("Match_Name", "")), str(r.get("Dir_Label", "")).split('(')[0].strip(), str(r["Kit_Name"]), str(r["cM"])
        lin_len = len(str(r.get("Lineage", "")).split("->"))
        text = f"<b>{kit_name}</b> is a {cm_val} cM match to <a href='https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(m_id)}&tree=tree1&parentset=0&display=vertical&generations=15' target='_blank'><b>{m_name}</b></a> via {d_label} back {lin_len} generations."
        if mc.get(r['Dir_Label'], 0) == 1: return f"<div style='background-color: #fffde7; padding: 12px; margin: -12px; border-left: 5px solid #fbc02d;'>{text} <span style='float:right; font-size:0.85em; color:#e65100; font-weight:bold; background:#fff8e1; padding:3px 8px; border-radius:4px; border:1px solid #ffe082;'>🌟 Singleton Line</span></div>"
        return f"<div style='padding: 12px; margin: -12px;'>{text}</div>"

    def format_tree(r):
        m_id, m_name, lin_str, kit_name = str(r.get("Match_ID", "")), str(r.get("Match_Name", "")), str(r.get("Lineage", "")), str(r["Kit_Name"])
        linked_lin = lin_str.replace(m_name, f'<a href="https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(m_id)}&tree=tree1&parentset=0&display=vertical&generations=15" target="_blank" style="color:#006064;text-decoration:none;font-weight:bold;">{m_name}</a>') if m_name in lin_str else lin_str
        text = f"<b style='color:#4a148c;'>{kit_name}</b>: {linked_lin}"
        if mc.get(r['Dir_Label'], 0) == 1: return f"<div style='background-color: #fffde7; padding: 12px; margin: -12px; border-left: 5px solid #fbc02d;'>{text} <span style='float:right; font-size:0.85em; color:#e65100; font-weight:bold; background:#fff8e1; padding:3px 8px; border-radius:4px; border:1px solid #ffe082;'>🌟 Singleton Line</span></div>"
        return f"<div style='padding: 12px; margin: -12px;'>{text}</div>"

    df_valid['Reg_Narrative'] = df_valid.apply(format_reg, axis=1)
    df_valid['Tree_Narrative'] = df_valid.apply(format_tree, axis=1)

    df_reg_za = df_valid.sort_values(by=['Dir_Label', 'sort_key'], ascending=[False, True]).copy()
    df_reg_za.rename(columns={'Reg_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    df_reg_az = df_valid.sort_values(by=['sort_key', 'Dir_Label'], ascending=[True, False]).copy()
    df_reg_az.rename(columns={'Reg_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)

    df_tree_za = df_valid.sort_values(by=['Dir_Label', 'sort_key'], ascending=[False, True]).copy()
    df_tree_za.rename(columns={'Tree_Narrative': 'Visual Lineage Path'}, inplace=True)
    df_tree_az = df_valid.sort_values(by=['sort_key', 'Dir_Label'], ascending=[True, False]).copy()
    df_tree_az.rename(columns={'Tree_Narrative': 'Visual Lineage Path'}, inplace=True)

    toggle_reg_za = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Register:</strong> &nbsp;<span style="color:#006064; font-weight:bold;">By Ancestral Line (Z-A)</span> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1; text-decoration:none;">By Participant (A-Z)</a></div>'
    toggle_reg_az = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1; text-decoration:none;">By Ancestral Line (Z-A)</a> &nbsp;|&nbsp; <span style="color:#006064; font-weight:bold;">By Participant (A-Z)</span></div>'
    toggle_tree_za = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Trees:</strong> &nbsp;<span style="color:#006064; font-weight:bold;">By Ancestral Line (Z-A)</span> &nbsp;|&nbsp; <a href="just-trees-az.shtml" style="color:#00acc1; text-decoration:none;">By Participant (A-Z)</a></div>'
    toggle_tree_az = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Trees:</strong> &nbsp;<a href="just-trees.shtml" style="color:#00acc1; text-decoration:none;">By Ancestral Line (Z-A)</a> &nbsp;|&nbsp; <span style="color:#006064; font-weight:bold;">By Participant (A-Z)</span></div>'

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%B %d, %Y %-I:%M %p EST")
    stats_bar_full = f'<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Study Data Current As Of:</strong> {timestamp} | <strong>Total Autosomal matches:</strong> {len(df):,}</div>'
    current_year = str(datetime.now(est).year)
    LEGAL_FOOTER = r"""<div class="legal-footer no-print" style="margin-top:50px;padding:20px;background:#f4f4f4;border-top:1px solid #ddd;text-align:center;color:#666;font-family:sans-serif;font-size:0.85em;clear:both;"><p style="margin-bottom:5px;font-size:1.1em;color:#333;"><strong>&copy; __YEAR__ Ronald Eugene Yates. All Rights Reserved.</strong></p><p style="margin-bottom:5px;">Generated by <em>The Forensic Genealogy Publisher&trade;</em></p><p style="font-style:italic;color:#888;margin-bottom:0;max-width:800px;margin-left:auto;margin-right:auto;">The terms "Forensic Handshake", "Brick Wall Buster", and "Collateral Saturation" are trademarks of Ronald Eugene Yates.</p></div>""".replace('__YEAR__', current_year)

    def make_page(title, content, nav_b, bar, extra_css=""):
        s_info = SITE_INFO if nav_b else ""
        return f"<!DOCTYPE html>\n<html lang=\"en\">\n<head>\n<meta charset=\"UTF-8\">\n<title>{title}</title>\n<link rel=\"stylesheet\" href=\"partials_unified.css\">\n<link rel=\"stylesheet\" href=\"dna_tree_styles.css\">\n{extra_css}\n</head>\n<body id=\"top\">\n<div class=\"wrap\">\n<h1 class=\"centerline no-print\">{title}</h1>\n<div id=\"nav-slot\">{bar}{NAV_HTML}</div>\n{s_info}{content}\n</div>\n{LEGAL_FOOTER}{JS_CORE}{BTT_BTN}\n</body>\n</html>"

    print("    [+] Building Admin Hub...")
    df_testers = df[['Kit_Code', 'Tester_Name', 'Tester_ID', 'Tester_Sort_Key']].drop_duplicates('Kit_Code')
    match_counts = df_valid.groupby('Kit_Code').size().reset_index(name='Match_Count')
    part_stats = pd.merge(df_testers, match_counts, on='Kit_Code', how='left')
    part_stats['Match_Count'] = part_stats['Match_Count'].fillna(0).astype(int)
    tester_sort_map = df.drop_duplicates('Kit_Code').set_index('Kit_Code')['Tester_Sort_Key'].to_dict()
    part_stats['Sort_Key'] = part_stats['Kit_Code'].map(tester_sort_map).fillna('zzz')

    def make_admin_row(r):
        tid = str(r['Tester_ID']).strip()
        tname, kcode, mc_val = str(r["Tester_Name"]), str(r["Kit_Code"]), r['Match_Count']
        t_link = f'<a href="https://yates.one-name.net/tng/getperson.php?personID=I{tid}&tree=tree1" target="_blank" style="color:#00838f;text-decoration:underline;font-weight:bold;">{tname}</a>' if tid and tid != 'nan' else f'<b style="color:#333;">{tname}</b>'
        tid_display = f" <span style='color:#777;font-size:0.85em;'>[I{tid}]</span>" if tid and tid != 'nan' else ""
        mc_str = f"<span style='color:#d32f2f;font-weight:bold;'>0</span>" if mc_val == 0 else str(mc_val)
        return f"<tr><td data-sort='{r['Sort_Key']}' style='text-align:center; padding:10px;'>{t_link}{tid_display}<br><span style='color:#666;font-size:0.85em;'>Kit: {kcode}</span></td><td style='text-align:center;font-size:1.1em;vertical-align:middle; padding:10px;'>{mc_str}</td></tr>"

    admin_rows_az = [make_admin_row(r) for _, r in part_stats.sort_values(['Sort_Key', 'Tester_Name'], ascending=[True, True]).iterrows()]
    admin_rows_count = [make_admin_row(r) for _, r in part_stats.sort_values(['Match_Count', 'Sort_Key'], ascending=[True, True]).iterrows()]
    admin_content = f"""<div class="dashboard-grid"><a href="ons_yates_dna_register.shtml" class="dash-card"><span class="dash-icon">📋</span><span class="dash-title">DNA Register</span></a><a href="dna_network.html" class="dash-card"><span class="dash-icon">🕸️</span><span class="dash-title">DNA Network</span></a><a href="proof_consolidator.html" class="dash-card" style="border-color:#4a148c; background:#f3e5f5;"><span class="dash-icon">🎓</span><span class="dash-title" style="color:#4a148c;">Report</span></a><a href="proof_engine.html" class="dash-card"><span class="dash-icon">🔬</span><span class="dash-title">Proof Engine</span></a><a href="dna_dossier.html" class="dash-card"><span class="dash-icon">📁</span><span class="dash-title">Forensic Dossier</span></a><a href="gedmatch_integration.shtml" class="dash-card" style="border-color:#0277bd; background:#e1f5fe;"><span class="dash-icon">🧬</span><span class="dash-title" style="color:#01579b;">GEDmatch Hub</span></a><a href="dna_theory_of_the_case.htm" class="dash-card" style="border-color:#d84315; background:#fbe9e7;"><span class="dash-icon">⚓</span><span class="dash-title" style="color:#bf360c;">Anchor Theory</span></a><a href="engine_database.csv" download class="dash-card" style="border-color:#b71c1c; background:#ffebee;"><span class="dash-icon">💾</span><span class="dash-title" style="color:#c62828;">Download CSV Database</span></a></div><div class="audit-table-wrapper" style="text-align:center;"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;text-align:left;">Report - {len(part_stats)} Testers</h2><div style="text-align:center; margin-bottom:20px; font-family:sans-serif;"><button onclick="document.getElementById('table-az').style.display='block'; document.getElementById('table-count').style.display='none'; this.style.background='#006064'; this.style.color='white'; document.getElementById('btn-count').style.background='#e0f7fa'; document.getElementById('btn-count').style.color='#006064';" id="btn-az" style="padding:10px 20px; cursor:pointer; font-weight:bold; background:#006064; color:white; border:1px solid #00acc1; border-radius:4px;">Sort A-Z (Default)</button><button onclick="document.getElementById('table-count').style.display='block'; document.getElementById('table-az').style.display='none'; this.style.background='#006064'; this.style.color='white'; document.getElementById('btn-az').style.background='#e0f7fa'; document.getElementById('btn-az').style.color='#006064';" id="btn-count" style="padding:10px 20px; cursor:pointer; font-weight:bold; background:#e0f7fa; color:#006064; border:1px solid #00acc1; border-radius:4px; margin-left:10px;">Sort by Match Count (Low to High)</button></div><div id="table-az" style="max-height:600px;overflow-y:auto;border:1px solid #ddd;background:#fafafa; display:block; max-width:650px; margin:0 auto;"><table style="width:100%;border-collapse:collapse;"><thead><tr><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:60%;">Participant Kit</th><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:40%;">Matches</th></tr></thead><tbody>{''.join(admin_rows_az)}</tbody></table></div><div id="table-count" style="max-height:600px;overflow-y:auto;border:1px solid #ddd;background:#fafafa; display:none; max-width:650px; margin:0 auto;"><table style="width:100%;border-collapse:collapse;"><thead><tr><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:60%;">Participant Kit</th><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:40%;">Matches</th></tr></thead><tbody>{''.join(admin_rows_count)}</tbody></table></div></div>"""

    print("    [+] Assembling HTML Pages...")
    pages_to_upload = {}

    # Generate the Pages
    pages_to_upload["proof_consolidator.html"] = make_page("Master Proof Report", CONSOLIDATOR_HTML, False, stats_bar_full, extra_css=CONSOLIDATOR_CSS).replace('</body>', CONSOLIDATOR_JS.replace('__JS_GLOBALS__', JS_GLOBALS) + '</body>')
    pages_to_upload["proof_engine.html"] = PROOF_ENGINE_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["dna_dossier.html"] = DOSS_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["research_admin.html"] = make_page("Yates Research Admin Hub", admin_content, False, stats_bar_full, extra_css=ADMIN_CSS)
    pages_to_upload["ons_yates_dna_register.shtml"] = make_page("ONS Yates Study DNA Register", toggle_reg_za + f'<div class="table-scroll-wrapper">{df_reg_za.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)
    pages_to_upload["ons_yates_dna_register_participants.shtml"] = make_page("ONS Yates Study DNA Register", toggle_reg_az + f'<div class="table-scroll-wrapper">{df_reg_az.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)
    pages_to_upload["just-trees.shtml"] = make_page("Ancestor Register (Trees View)", toggle_tree_za + f'<div class="table-scroll-wrapper">{df_tree_za[["Visual Lineage Path"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)
    pages_to_upload["just-trees-az.shtml"] = make_page("Ancestor Register (Trees View)", toggle_tree_az + f'<div class="table-scroll-wrapper">{df_tree_az[["Visual Lineage Path"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)

    # Static Info Pages
    pages_to_upload["data_glossary.shtml"] = make_page("Data Glossary", GLOSSARY_INLINE, False, stats_bar_full, extra_css="")
    pages_to_upload["gedmatch_integration.shtml"] = make_page("GEDmatch Hub", GEDMATCH_INLINE, False, stats_bar_full, extra_css="")
    pages_to_upload["contents.shtml"] = make_page("Yates Study User Guide", CONTENTS_CONTENT, False, stats_bar_full, extra_css="")
    pages_to_upload["share_dna.shtml"] = make_page("Share Your Ancestry DNA Matches", SHARE_CONTENT, False, stats_bar_full, extra_css="")
    pages_to_upload["subscribe.shtml"] = make_page("Join the Yates Research Community", SUBSCRIBE_CONTENT, False, stats_bar_full, extra_css="")

    # Anchor overrides
    pages_to_upload["dna_theory_of_the_case.htm"] = make_page("DNA Theory of the Case", THEORY_PAGE_CONTENT, False, stats_bar_full, extra_css="")
    pages_to_upload["anchor_frame.htm"] = '<meta http-equiv="refresh" content="0; url=dna_theory_of_the_case.htm" />'
    pages_to_upload["anchor.html"] = '<meta http-equiv="refresh" content="0; url=dna_theory_of_the_case.htm" />'

    # Network Overrides
    pages_to_upload["dna_network.html"] = make_page("DNA Network Visualizer", NETWORK_PAGE_CONTENT.replace('__JS_GLOBALS__', JS_GLOBALS), False, stats_bar_full, extra_css="")
    pages_to_upload["dna_network.shtml"] = '<meta http-equiv="refresh" content="0; url=dna_network.html" />'

    # Legacy Links
    pages_to_upload["admin_singletons.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]
    pages_to_upload["admin_singletons_participants.shtml"] = pages_to_upload["ons_yates_dna_register_participants.shtml"]
    pages_to_upload["yates_ancestor_register.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]

    # --- STEP 6: UPLOAD ---
    print("\n[LOCAL] Overwriting Files...")
    for fn, content in pages_to_upload.items():
        if os.path.exists(fn): os.remove(fn)
        with open(fn, "w", encoding="utf-8") as f: f.write(content)

    print("\n[STEP 3] Uploading via FTP to Live Server...")
    try:
        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p(); ftps.cwd("ons-study")

        files_to_upload = list(pages_to_upload.keys())
        if os.path.exists(CSV_DB): files_to_upload.append(CSV_DB)

        upload_count = 0
        for fn in files_to_upload:
            with open(fn, "rb") as fh: ftps.storbinary(f"STOR {fn}", fh)
            upload_count += 1
            print(f"    [{upload_count}/{len(files_to_upload)}] 📤 Uploaded: {fn}")

        ftps.quit()
        print(f"\n🎉 SUCCESS. Uploaded {upload_count} files directly to the active server.")
    except Exception as e:
        print(f"\n⚠️ FTP SKIP: {e}. ZIP and manual upload required.")

print("✅ Cell 4 (Light Publisher) Loaded.")

✅ Cell 4 (Light Publisher) Loaded.


In [40]:
# @title [CELL 5] The Master Orchestrator (One-Click Deploy)
print("="*60)
print("      [CELL 5] MASTER ORCHESTRATOR STARTING...")
print("="*60)

try:
    print("\n[1/2] FIRING UP THE DATA ENGINE (Cell 2)...")
    # Note: Cell 2 runs on its own when you click its play button.
    # The Orchestrator now just fires the Publisher!

    print("\n[2/2] FIRING UP THE PUBLISHER (Cell 4)...")
    run_publisher()

    print("\n" + "="*60)
    print(" 🎉 FULL SYSTEM DEPLOYMENT COMPLETE! 🎉")
    print("="*60)
    print("Your data has been successfully processed, merged with the")
    print("Template Vault, and deployed directly to the live server.")

except NameError as e:
    print(f"\n❌ ORCHESTRATOR ERROR: {e}")
    print("    It looks like Colab forgot the functions. Please make sure you")
    print("    have clicked 'Run' on Cell 4 (Publisher) to load it.")
except Exception as e:
    print(f"\n❌ UNEXPECTED ERROR: {e}")

      [CELL 5] MASTER ORCHESTRATOR STARTING...

[1/2] FIRING UP THE DATA ENGINE (Cell 2)...

[2/2] FIRING UP THE PUBLISHER (Cell 4)...
      [CELL 4] PUBLISHER STARTING...
    [+] Loading Pre-Calculated Data...
    [+] Building Data Tables...
    [+] Building Admin Hub...
    [+] Assembling HTML Pages...

[LOCAL] Overwriting Files...

[STEP 3] Uploading via FTP to Live Server...
    [1/22] 📤 Uploaded: proof_consolidator.html
    [2/22] 📤 Uploaded: proof_engine.html
    [3/22] 📤 Uploaded: dna_dossier.html
    [4/22] 📤 Uploaded: research_admin.html
    [5/22] 📤 Uploaded: ons_yates_dna_register.shtml
    [6/22] 📤 Uploaded: ons_yates_dna_register_participants.shtml
    [7/22] 📤 Uploaded: just-trees.shtml
    [8/22] 📤 Uploaded: just-trees-az.shtml
    [9/22] 📤 Uploaded: data_glossary.shtml
    [10/22] 📤 Uploaded: gedmatch_integration.shtml
    [11/22] 📤 Uploaded: contents.shtml
    [12/22] 📤 Uploaded: share_dna.shtml
    [13/22] 📤 Uploaded: subscribe.shtml
    [14/22] 📤 Uploaded: dna_theory

In [ ]:
# @title [CELL ] The Time Machine (Data Vault & Archiver)
def run_archiver():
    print("="*60)
    print("      [CELL 6] THE TIME MACHINE (Archive & Sync)")
    print("="*60)

    import zipfile
    import os
    import pytz
    import json
    from datetime import datetime
    from google.colab import files
    from google.colab import userdata

    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%Y-%m-%d_%H%M_%S")

    # --- 1. GENERATE SITE SNAPSHOT JSON ---
    print("[STEP 1] Generating Site Snapshot JSON...")
    snapshot_data = {}
    html_files = [f for f in os.listdir('.') if f.lower().endswith(('.shtml', '.html'))]

    if not html_files:
        print("    ❌ No generated HTML files found! Run Cell 5 first.")
        return

    for f_name in html_files:
        try:
            with open(f_name, 'r', encoding='utf-8') as fh:
                snapshot_data[f_name] = fh.read()
        except Exception as e:
            print(f"    ⚠️ Could not read {f_name}: {e}")

    snapshot_name = f"site_snapshot_{timestamp}.json"
    with open(snapshot_name, 'w', encoding='utf-8') as f:
        json.dump(snapshot_data, f)
    print(f"    ✅ Created snapshot JSON: {snapshot_name}")

    # --- 2. CREATE MASTER ZIP VAULT ---
    extensions = ('.csv', '.shtml', '.html', '.json', '.js', '.css', '.ged')
    files_to_pack = [f for f in os.listdir('.') if f.lower().endswith(extensions) and "sample_data" not in f]

    zip_name = f"Yates_Study_Backup_{timestamp}.zip"

    print(f"\n[STEP 2] Compressing {len(files_to_pack)} files into {zip_name}...")
    try:
        with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zf:
            for file in files_to_pack:
                zf.write(file)
        print(f"    ✅ Archive Created: {zip_name} ({os.path.getsize(zip_name)/1024/1024:.2f} MB)")
    except Exception as e:
        print(f"    ❌ Compression Failed: {e}")
        return

    # --- 3. FTP UPLOAD (BACKUPS FOLDER) ---
    print("\n[STEP 3] Uploading to Web Server Vault (FTP)...")
    try:
        from ftplib import FTP_TLS
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")

        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()

        try:
            ftps.cwd("/ons-study/backups")
        except:
            try:
                ftps.mkd("/ons-study/backups")
                ftps.cwd("/ons-study/backups")
            except:
                print("    ⚠️ Could not navigate to /ons-study/backups/. Uploading to root.")

        with open(zip_name, "rb") as fh:
            ftps.storbinary(f"STOR {zip_name}", fh)
        print(f"    ✅ FTP Success: /backups/{zip_name}")

        with open(snapshot_name, "rb") as fh:
            ftps.storbinary(f"STOR {snapshot_name}", fh)
        print(f"    ✅ FTP Success: /backups/{snapshot_name}")

        ftps.quit()
    except Exception as e:
        print(f"    ⚠️ FTP Upload skipped/failed: {e}")

    # --- 4. DROPBOX SYNC (REFRESH TOKEN METHOD) ---
    print("\n[STEP 4] Syncing to Dropbox...")
    try:
        # Pull the specific Refresh Token keys from Colab Secrets
        dbx_app_key = os.environ.get("DBX_APP_KEY") or userdata.get("DBX_APP_KEY")
        dbx_app_secret = os.environ.get("DBX_APP_SECRET") or userdata.get("DBX_APP_SECRET")
        dbx_refresh = os.environ.get("DBX_REFRESH_TOKEN") or userdata.get("DBX_REFRESH_TOKEN")

        if not dbx_refresh:
            print("    ❌ ERROR: 'DBX_REFRESH_TOKEN' not found in Colab Secrets.")
            print("       Make sure your keys are named exactly DBX_APP_KEY, DBX_APP_SECRET, and DBX_REFRESH_TOKEN.")
        else:
            try:
                import dropbox
            except ImportError:
                os.system('pip install dropbox')
                import dropbox

            # Authenticate using the robust refresh method
            dbx = dropbox.Dropbox(
                app_key=dbx_app_key,
                app_secret=dbx_app_secret,
                oauth2_refresh_token=dbx_refresh
            )

            target_path = f"/Yates_Study_Sync/archives/{snapshot_name}"

            with open(snapshot_name, "rb") as f:
                dbx.files_upload(f.read(), target_path)
            print(f"    ✅ Dropbox Sync Success: {target_path}")

    except Exception as e:
        print(f"    ❌ Dropbox Upload Failed: {e}")

    # --- 5. TRIGGER LOCAL DOWNLOAD ---
    print("\n[STEP 5] Triggering Local Download...")
    try:
        files.download(zip_name)
        print("    ✅ Please check your browser downloads for the Archive.")
    except:
        print("    ⚠️ Could not auto-download. You can download the zip manually from the Colab files pane.")

run_archiver()

      [CELL 6] THE TIME MACHINE (Archive & Sync)
[STEP 1] Generating Site Snapshot JSON...
    ✅ Created snapshot JSON: site_snapshot_2026-02-25_2033_22.json

[STEP 2] Compressing 24 files into Yates_Study_Backup_2026-02-25_2033_22.zip...
    ✅ Archive Created: Yates_Study_Backup_2026-02-25_2033_22.zip (32.84 MB)

[STEP 3] Uploading to Web Server Vault (FTP)...
    ✅ FTP Success: /backups/Yates_Study_Backup_2026-02-25_2033_22.zip
    ✅ FTP Success: /backups/site_snapshot_2026-02-25_2033_22.json

[STEP 4] Syncing to Dropbox...
    ✅ Dropbox Sync Success: /Yates_Study_Sync/archives/site_snapshot_2026-02-25_2033_22.json

[STEP 5] Triggering Local Download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

    ✅ Please check your browser downloads for the Archive.


In [ ]:
# @title [OLD BELOW]

In [20]:
# @title [CELL 3] The Data Engine (V124 - Sort Authority Inject)
def run_engine():
    print("="*60)
    print("      [CELL 3] ENGINE STARTING (V124 - SORT AUTHORITY)...")
    print("="*60)

    import os, sys, re, csv
    from ftplib import FTP_TLS
    from google.colab import userdata
    from datetime import datetime

    CSV_DB = "engine_database.csv"
    if os.path.exists(CSV_DB): os.remove(CSV_DB)

    try:
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
    except: pass
    REMOTE_SUBDIR = "ons-study"
    KEY_FILE = "match_to_unmasked.csv"
    PROCESSED_GED = "_processed_unmasked.ged"

    def clean_and_standardize(raw_name):
        if not raw_name: return "findme"
        s = raw_name.replace("/", "").strip()
        triggers = ["unknown", "missing", "searching", "still searching", "living", "private", "nee", "nee ?", "wife", "mrs"]
        if s.lower() in triggers or s == "": return "findme"
        if "?" in s: return "findme"
        if "unknown" in s.lower(): return "findme"
        return s

    def get_surname(full_name):
        if not full_name or "findme" in full_name.lower(): return ""
        clean = re.sub(r'\b(jr\.?|sr\.?|iii|iv|esq\.?|m\.d\.?|ph\.d\.?)\b', '', full_name, flags=re.IGNORECASE)
        parts = clean.replace(',', '').split()
        return parts[-1] if parts else ""

    def make_directory_label(name, dates):
        if "findme" in name.lower(): return name
        sur = get_surname(name)
        if not sur: return name
        firsts = re.sub(f"{re.escape(sur)}$", "", name).strip()
        return f"{sur}, {firsts} {dates}"

    print("\n[STEP 1] Resolving Files (Local Priority)...")
    if os.path.exists(KEY_FILE):
        print(f"    ✅ Found {KEY_FILE} locally. Skipping FTP download.")
    else:
        print(f"    🌐 {KEY_FILE} not found locally. Attempting FTP fetch...")
        try:
            ftps = FTP_TLS()
            ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p()
            try:
                with open(KEY_FILE, "wb") as f: ftps.retrbinary(f"RETR /{REMOTE_SUBDIR}/{KEY_FILE}", f.write)
                print(f"    ✅ Successfully downloaded {KEY_FILE}.")
            except Exception as e:
                print(f"    ⚠️ FTP download failed: {e}")
            ftps.quit()
        except Exception as e:
            print(f"    ⚠️ FTP connection failed: {e}")

    all_files = os.listdir('.')
    ged_files = [f for f in all_files if f.lower().endswith('.ged') and "_processed" not in f.lower()]
    if not ged_files: return print("❌ No GEDCOM found. Please upload one.")
    ged_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    DEFAULT_GEDCOM = ged_files[0]
    print(f"    👉 Source GEDCOM: {DEFAULT_GEDCOM}")

    def resolve_code(payload):
        m = re.search(r'(\d+)\s*&?\s*([^ \t\n\r\f\v]+)', payload)
        return m.group(2).lower() if m else None

    print("\n[STEP 2] Loading Tester Authority CSV...")
    csv_auth = {}
    if os.path.exists(KEY_FILE):
        with open(KEY_FILE, 'r', errors='replace') as f:
            reader = csv.reader(f)
            for i, row in enumerate(reader):
                if len(row) >= 2:
                    if i == 0 and ("tester" in row[0].lower() or "masked" in row[0].lower() or "code" in row[0].lower()):
                        continue
                    code = row[0].strip().lower()
                    name = row[1].strip()
                    tid = row[2].strip() if len(row) > 2 else ""
                    if tid: tid = "I" + re.sub(r'[^0-9]', '', tid)
                    # ✨ NEW: Explicitly pull Column D (Index 3) as the Sort Key
                    sort_key = row[3].strip().lower() if len(row) > 3 else ""
                    csv_auth[code] = {"name": name, "id": tid, "sort_key": sort_key}

    print("\n[STEP 3] Parsing GEDCOM for Study| Tags & Lineages...")
    import shutil
    shutil.copyfile(DEFAULT_GEDCOM, PROCESSED_GED)

    individuals = {}; families = {}; study_testers = {}

    def is_yates(name_str):
        n = (name_str or "").lower()
        return "yates" in n or "yeates" in n or "yate" in n

    current_id = None; current_fam = None; current_tag = None
    with open(PROCESSED_GED, "r", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip(); parts = line.split(" ", 2)
            if len(parts) < 2: continue
            lvl, tag, val = parts[0], parts[1], parts[2] if len(parts)>2 else ""

            if lvl == "0" and "INDI" in val:
                current_id = tag.replace("@", "")
                individuals[current_id] = {"name": "findme", "famc": None, "fams": [], "match_code": "", "cm": 0, "birt": "", "deat": ""}
                current_fam = None; current_tag = "INDI"
            elif current_id and lvl != "0":
                if tag == "NAME" and lvl == "1":
                    individuals[current_id]["name"] = clean_and_standardize(val)
                elif tag == "FAMC" and lvl == "1":
                    individuals[current_id]["famc"] = val.replace("@", "")
                elif tag == "FAMS" and lvl == "1":
                    individuals[current_id]["fams"].append(val.replace("@", ""))

                elif tag == "NICK" and lvl == "2" and "Study|" in val:
                    tester_code = val.split("Study|")[-1].strip().lower()
                    study_testers[tester_code] = {"id": current_id, "name": individuals[current_id]["name"]}

                elif tag == "NPFX" and lvl == "2":
                    code = resolve_code(val)
                    if code: individuals[current_id]["match_code"] = code.lower()
                    m = re.search(r'^(\d+)|(\d+)\s*cM', val, re.IGNORECASE)
                    if m: individuals[current_id]["cm"] = int(m.group(1) or m.group(2))

                elif tag == "BIRT": current_tag = "BIRT"
                elif tag == "DEAT": current_tag = "DEAT"
                elif tag == "DATE" and current_tag:
                    m = re.search(r'\d{4}', val)
                    if m: individuals[current_id][current_tag.lower()] = m.group(0)
                    current_tag = None

            if lvl == "0" and "FAM" in val:
                current_fam = tag.replace("@", "")
                families[current_fam] = {"husb": None, "wife": None}
                current_id = None
            elif current_fam and lvl != "0":
                if tag == "HUSB": families[current_fam]["husb"] = val.replace("@", "")
                elif tag == "WIFE": families[current_fam]["wife"] = val.replace("@", "")

    def get_parents(pid):
        if not pid or pid not in individuals: return None, None
        famc = individuals[pid]["famc"]
        if not famc or famc not in families: return None, None
        return families[famc]["husb"], families[famc]["wife"]

    def get_mother_surname(pid):
        if not pid: return ""
        _, mom_id = get_parents(pid)
        if mom_id and mom_id in individuals: return get_surname(individuals[mom_id]["name"])
        return ""

    def to_spanish_name(pid, current_name):
        if "findme" in current_name.lower(): return current_name
        mom_surname = get_mother_surname(pid)
        if not mom_surname or "findme" in mom_surname.lower(): return current_name
        if mom_surname.lower() not in current_name.lower(): return f"{current_name}-{mom_surname}"
        return current_name

    # 🌟 THE DEEP ANCESTRY RADAR
    yates_memo = {}
    def has_yates_ancestry(pid):
        if not pid or pid not in individuals: return False
        if pid in yates_memo: return yates_memo[pid]

        if is_yates(individuals[pid].get("name", "")):
            yates_memo[pid] = True
            return True

        dad_id, mom_id = get_parents(pid)
        res = has_yates_ancestry(dad_id) or has_yates_ancestry(mom_id)
        yates_memo[pid] = res
        return res

    def climb_full_line(start_id):
        curr = start_id; lineage_data = []
        while curr:
            p = individuals.get(curr)
            if not p: break
            spanish_name = to_spanish_name(curr, p["name"])
            spouse_name = "findme"; spouse_id = None
            if p["fams"]:
                fid = p["fams"][0]
                if fid in families:
                    f = families[fid]
                    sid = f["wife"] if f["husb"] == curr else f["husb"]
                    if sid and sid in individuals:
                        spouse_name = individuals[sid]["name"]; spouse_id = sid
            spouse_spanish = to_spanish_name(spouse_id, spouse_name) if spouse_id else spouse_name
            lineage_data.append({"name": spanish_name, "raw_name": p["name"], "id": curr, "spouse": spouse_spanish, "spouse_raw": spouse_name, "spouse_id": spouse_id})

            dad_id, mom_id = get_parents(curr)
            if not dad_id and not mom_id: break

            dad_has_yates = has_yates_ancestry(dad_id)
            mom_has_yates = has_yates_ancestry(mom_id)

            if dad_has_yates and not mom_has_yates: curr = dad_id
            elif mom_has_yates and not dad_has_yates: curr = mom_id
            else: curr = dad_id if dad_id else mom_id

        return lineage_data

    def format_dates(uid):
        if not uid or uid not in individuals: return "findme"
        b = individuals[uid]["birt"] or "findme"
        d = individuals[uid]["deat"] or "findme"
        b = re.sub(r'\?', 'findme', b); d = re.sub(r'\?', 'findme', d)
        if b == "findme" and d == "findme": return "findme"
        return f"({b} - {d})"

    testers = {}
    # Load from CSV Auth, including the new Sort Key
    for code, data in csv_auth.items():
        testers[code] = {"name": data["name"], "id": data["id"], "sort_key": data.get("sort_key", "")}

    # Load from GEDCOM Nick tags if missing
    for code, data in study_testers.items():
        if code not in testers:
            testers[code] = {"name": data["name"], "id": data["id"], "sort_key": ""}
        elif not testers[code]["id"]:
            testers[code]["id"] = data["id"]

    for kcode, tdata in testers.items():
        t_lin = ""; t_pids = ""
        if tdata["id"] and tdata["id"] in individuals:
            lin_data = climb_full_line(tdata["id"])
            if lin_data:
                full = list(reversed(lin_data))
                t_lin = " -> ".join([x["name"] for x in full])
                t_pids = ",".join([x["id"] for x in full])
        tdata["lineage_str"] = t_lin
        tdata["path_ids"] = t_pids

    print("\n[STEP 4] Constructing Database...")
    rows = []
    for uid, p in individuals.items():
        if p["match_code"]: # It's a found match!
            kit_code = p["match_code"]

            if kit_code in testers:
                t_name = testers[kit_code]["name"]
                t_id = testers[kit_code]["id"]
                t_lin = testers[kit_code]["lineage_str"]
                t_pids = testers[kit_code]["path_ids"]
                t_sort = testers[kit_code]["sort_key"] # ✨ Pull the Sort Key!
                tester_display = f"{t_name} [{t_id}]" if t_id else f"{t_name} [{kit_code}]"
            else:
                t_name = kit_code
                t_id = ""
                t_lin = ""
                t_pids = ""
                t_sort = ""
                tester_display = f"{kit_code} [{kit_code}]"

            lineage_data = climb_full_line(uid)
            if not lineage_data: continue

            full_line = list(reversed(lineage_data))
            gen1 = full_line[0]

            top_name = gen1["raw_name"]
            top_dates = format_dates(gen1["id"])
            spouse_name = gen1["spouse_raw"]
            spouse_id = gen1["spouse_id"]
            spouse_dates = format_dates(spouse_id)

            if spouse_name != "findme":
                husb_sur = get_surname(top_name); wife_sur = get_surname(spouse_name)
                if husb_sur.lower() == wife_sur.lower(): spouse_name += f" (née {wife_sur})"

            pair_dated = f"{top_name} {top_dates}"
            if spouse_name != "findme": dir_label = make_directory_label(top_name, top_dates) + f" & {spouse_name}"
            else: dir_label = make_directory_label(top_name, top_dates)

            if spouse_name != "findme": pair_dated += f" & {spouse_name} {spouse_dates}"
            pair_simple = f"{top_name} & {spouse_name}" if spouse_name != "findme" else top_name

            clean_top = re.sub(r'[^a-zA-Z0-9]', '', top_name)
            clean_sp = re.sub(r'[^a-zA-Z0-9]', '', spouse_name.split('(')[0]) if spouse_name != "findme" else "ZZZ"
            sort_key = f"{clean_top}_{clean_sp}"

            path_names = []
            for i, x in enumerate(full_line):
                if i == 0: path_names.append(pair_dated)
                else: path_names.append(x["name"])

            lineage_str = " -> ".join(path_names)
            path_ids = ",".join([x["id"] for x in full_line])

            _, fa1_mom_id = get_parents(gen1["id"])
            fa1_mother = to_spanish_name(fa1_mom_id, individuals[fa1_mom_id]["name"]) if fa1_mom_id else "findme"

            fa2_mother = "findme"
            if spouse_id:
                _, fa2_mom_id = get_parents(spouse_id)
                if fa2_mom_id: fa2_mother = to_spanish_name(fa2_mom_id, individuals[fa2_mom_id]["name"])

            # ✨ NEW: Injecting Tester_Sort_Key directly into the CSV row
            rows.append({
                "Tester_Code": kit_code,
                "Tester_Name": t_name,
                "Tester_ID": t_id,
                "Tester_Display": tester_display,
                "Tester_Sort_Key": t_sort,
                "Tester_Lineage": t_lin,
                "Tester_Path_IDs": t_pids,
                "Match_Name": p["name"],
                "Match_ID": uid,
                "cM": p["cm"],
                "Match_Lineage": lineage_str,
                "Match_Path_IDs": path_ids,
                "Authority_Directory_Label": dir_label,
                "Authority_FirstAncestor": pair_simple,
                "Authority_FirstAncestor_alpha": sort_key,
                "Authority_FirstAncestor_dated": pair_dated,
                "fa_1 extracted": top_name, "fa_1_Dates": top_dates, "fa_1_Mother": fa1_mother,
                "fa_2 extracted": spouse_name, "fa_2 Dates": spouse_dates, "fa_2_Mother": fa2_mother,
                "Gen_Count": len(full_line)
            })

    rows.sort(key=lambda r: r["Authority_Directory_Label"])

    fieldnames = [
        "Tester_Code", "Tester_Name", "Tester_ID", "Tester_Display", "Tester_Sort_Key",
        "Tester_Lineage", "Tester_Path_IDs",
        "Match_Name", "Match_ID", "cM", "Match_Lineage", "Match_Path_IDs",
        "Authority_Directory_Label", "Authority_FirstAncestor", "Authority_FirstAncestor_alpha", "Authority_FirstAncestor_dated",
        "fa_1 extracted", "fa_1_Dates", "fa_1_Mother",
        "fa_2 extracted", "fa_2 Dates", "fa_2_Mother",
        "Gen_Count"
    ]

    with open(CSV_DB, "w", encoding="iso-8859-15", newline="", errors="replace") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader(); writer.writerows(rows)

    print(f"\n[SUCCESS] Engine V124 Complete. Saved {len(rows)} verified matches to {CSV_DB}.")

# Execute the engine
run_engine()

print("✅ Cell 3 (Engine V124 - Sort Authority Inject) Loaded.")

      [CELL 3] ENGINE STARTING (V124 - SORT AUTHORITY)...

[STEP 1] Resolving Files (Local Priority)...
    ✅ Found match_to_unmasked.csv locally. Skipping FTP download.
    👉 Source GEDCOM: yates_study_2025.ged

[STEP 2] Loading Tester Authority CSV...

[STEP 3] Parsing GEDCOM for Study| Tags & Lineages...

[STEP 4] Constructing Database...

[SUCCESS] Engine V124 Complete. Saved 1713 verified matches to engine_database.csv.
✅ Cell 3 (Engine V124 - Sort Authority Inject) Loaded.


In [27]:
# @title [CELL 4] The Template Library (Patch 70: CSS Variables & Scaled Matrices)
print("="*60)
print("      [CELL 4] TEMPLATE LIBRARY LOADING (CSS Fix + Scaled Matrices)...")
print("="*60)

# 🌟 0. CATCH-ALL CSS (SAFELY RESTORED) 🌟
CONTENTS_CSS = ""
SHARE_CSS = ""
GLOSS_CSS = ""
GLOSSARY_CSS = ""
SUBSCRIBE_CSS = ""
THEORY_CSS = ""
TREE_CSS = ""
ADMIN_CSS = ""
DOSS_CSS = ""
REGISTER_CSS = "<style>.table-scroll-wrapper{overflow-x:auto; background:white; padding:20px; border-radius:8px; box-shadow:0 4px 6px rgba(0,0,0,0.1); max-width:1400px; margin:20px auto; display:flex; justify-content:center; flex-direction:column;} .table-scroll-wrapper table.dataframe { margin: 0 auto; width: 100%; }</style>"

# 🌟 1. GLOBAL CSS & NAV 🌟
NAV_HTML = r"""<style>nav.oldnav ul{display:flex;flex-wrap:wrap;justify-content:center;background-color:#006064!important;border-bottom:2px solid #00acc1!important;margin:0;padding:0;list-style:none} nav.oldnav li{display:inline-block} nav.oldnav a{display:block;padding:10px 15px;text-decoration:none;color:#e0f7fa!important;font-size:14px} nav.oldnav a:hover{background-color:#00838f!important} @media print { nav.oldnav, #nav-slot, .no-print { display: none !important; } }</style><nav class="oldnav"><ul><li><a href="/ons-study/research_admin.html" style="color:#ffcc80 !important; font-weight:bold;">Admin Hub</a></li><li><a href="/ons-study/contents.shtml" style="color:#ffcc80 !important; font-weight:bold;">Guide</a></li><li><a href="/ons-study/ons_yates_dna_register.shtml">DNA Register</a></li><li><a href="/ons-study/just-trees.shtml">Trees</a></li><li><a href="/ons-study/dna_network.html">DNA Network</a></li><li><a href="/ons-study/proof_engine.html">Proof Engine</a></li><li><a href="/ons-study/dna_dossier.html">Forensic Dossier</a></li><li><a href="/ons-study/proof_consolidator.html" style="background-color:#4a148c; color:#fff !important; font-weight:bold; border-left:1px solid #7c43bd; border-right:1px solid #7c43bd;">Report</a></li><li><a href="/ons-study/anchor_frame.htm" style="background-color:#d84315; color:#fff !important; font-weight:bold; border-left:1px solid #ffab91; border-right:1px solid #ffab91;">ANCHOR</a></li><li><a href="/ons-study/dna_theory_of_the_case.htm" style="color:#b2dfdb;">Theory</a></li><li><a href="/ons-study/data_glossary.shtml">Data Glossary</a></li><li><a href="/ons-study/gedmatch_integration.shtml" style="color:#81d4fa;">GEDmatch</a></li><li><a href="/ons-study/share_dna.shtml" style="background-color:#0277bd; font-weight:bold;">Share DNA</a></li><li><a href="/ons-study/subscribe.shtml" style="background-color:#004d40; font-weight:bold;">Subscribe</a></li></ul></nav>"""
SITE_INFO = r"""<div class="no-print" style="background:#e0f2f1;border:1px solid #b2dfdb;padding:20px;margin:20px auto;width:90%;border-radius:8px;font-family:sans-serif;"><h3 style="color:#006064;margin-top:0;border-bottom:2px solid #004d40;padding-bottom:10px;">Establishing Kinship Through Collateral DNA Saturation</h3><p style="color:#333;line-height:1.6;margin-bottom:0;">This register employs <em>Collateral DNA Saturation</em>—a method blending genealogical reasoning with data-driven logic to prove connections using multiple independent DNA cousins.</p></div>"""
CSS_BASE = r"""body{font-family:'Segoe UI',sans-serif;background:#f0f2f5;padding:20px;display:flex;flex-direction:column;min-height:100vh;margin:0;} .wrap{flex:1;} .proof-card{background:white;max-width:1100px;margin:20px auto;border-radius:8px;box-shadow:0 4px 15px rgba(0,0,0,0.1);padding:40px} .badge{padding:5px 10px;border-radius:4px;font-weight:bold;font-size:0.85em;text-transform:uppercase;border:1px solid #ccc;} .badge-platinum{background:#eceff1;color:#263238} .badge-gold{background:#fff8e1;color:#f57f17} .badge-silver{background:#f5f5f5;color:#616161} .badge-bronze{background:#efebe9;color:#5d4037} table{width:100%;border-collapse:collapse;margin-top:15px;margin-bottom:40px;font-family:'Georgia',serif;font-size:15px;} th{background:#eceff1;color:#263238;padding:12px;text-align:left;border-bottom:2px solid #000;} td{padding:12px;border-bottom:1px solid #ddd;vertical-align:top;} @media print{ .no-print{display:none !important;} .only-print{display:block !important;} .proof-card{box-shadow:none;border:none;padding:0;margin:0;} body{background:white;padding:0;display:block;} th{background:#f0f0f0 !important;color:#000 !important;} .badge{border:1px solid #000;color:#000;background:transparent !important;} .legal-footer{background:transparent !important; border-top:2px solid #000 !important; color:#000 !important; page-break-inside:avoid !important; padding:10px 0 !important; margin-top:30px !important;} } .only-print{display:none;}"""
JS_CORE = r"""<script type="text/javascript">(function(){ function textOf(c){var val = c.getAttribute('data-sort') || c.textContent || c.innerText;return (val || '').replace(/ +/g,' ').trim().toLowerCase();} function sortTable(t,i,d){if(!(t&&t.tBodies&&t.tBodies[0]))return;var tb=t.tBodies[0],r=Array.prototype.slice.call(tb.rows||[]),asc=(d==='asc');r.sort(function(a,b){var A=textOf(a.cells[i]),B=textOf(b.cells[i]),nA=parseFloat(A.replace(/[^0-9.-]/g,'')),nB=parseFloat(B.replace(/[^0-9.-]/g,''));if(!isNaN(nA)&&!isNaN(nB))return asc?(nA-nB):(nB-nA);return(A<B)?(asc?-1:1):(A>B)?(asc?1:-1):0;});var f=document.createDocumentFragment();for(var k=0;k<r.length;k++)f.appendChild(r[k]);tb.appendChild(f);} function makeSortable(t){if(!(t&&t.tHead&&t.tHead.rows.length))return;var th=t.tHead.rows[0].cells;for(var i=0;i<th.length;i++){(function(idx){var h=th[idx],d='asc';h.style.cursor='pointer';h.onclick=function(){d=(d==='asc')?'desc':'asc';for(var j=0;j<th.length;j++)th[j].innerHTML=th[j].innerHTML.replace(' (asc)','').replace(' (desc)','');h.innerHTML+=(d==='asc'?' (asc)':' (desc)');sortTable(t,idx,d);};})(i);}} window.filterTable = function() { var input = document.getElementById("tableSearch"); var filter = input.value.toUpperCase(); var table = document.getElementById("reg-table") || document.querySelector("table.dataframe"); var tr = table.getElementsByTagName("tr"); for (var i = 1; i < tr.length; i++) { var tdArr = tr[i].getElementsByTagName("td"); var found = false; for (var j = 0; j < tdArr.length; j++) { if (tdArr[j]) { var txtValue = tdArr[j].textContent || tdArr[j].innerText; if (txtValue.toUpperCase().indexOf(filter) > -1) { found = true; break; } } } tr[i].style.display = found ? "" : "none"; } } function init(){ var t=document.getElementsByTagName('table'); for(var i=0;i<t.length;i++) if(t[i].className.indexOf('sortable') !== -1) makeSortable(t[i]); } if(document.readyState==='loading')document.addEventListener('DOMContentLoaded',init,false);else init(); })();</script>"""
BTT_BTN = r"""<style>.btt{position:fixed;bottom:20px;right:20px;background:#00838f;color:white;padding:10px 15px;text-decoration:none;border-radius:4px;font-weight:bold;box-shadow:0 2px 5px rgba(0,0,0,0.3);z-index:1000;opacity:0.9;} .btt:hover{opacity:1;background:#006064;} @media print { .btt { display: none !important; } }</style><a href="#top" class="btt no-print">⬆️ Top</a>"""

# 🌟 2. NEW ANCHOR UI 🌟
ANCHOR_CSS = r"""<style>.pill-nav{display:flex;justify-content:center;flex-wrap:wrap;margin:30px auto;gap:15px;max-width:1000px}.pill{padding:12px 25px;border-radius:25px;border:2px solid #ddd;background:#fff;cursor:pointer;font-weight:bold;color:#555;font-size:15px;transition:all 0.2s;box-shadow:0 2px 4px rgba(0,0,0,0.05)}.pill:hover{background:#f4f4f4;transform:translateY(-2px)}.pill.active-framework{background:#0277bd;color:white;border-color:#01579b;box-shadow:0 4px 10px rgba(2,119,189,0.3)}.pill.active-css{background:#4a148c;color:white;border-color:#38006b;box-shadow:0 4px 10px rgba(74,20,140,0.3)}.pill.active-docs{background:#5d4037;color:white;border-color:#3e2723;box-shadow:0 4px 10px rgba(93,64,55,0.3)}.pill.active-master{background:#d84315;color:white;border-color:#bf360c;box-shadow:0 4px 10px rgba(216,67,21,0.3)}.anchor-table-wrapper{overflow-x:auto;display:flex;justify-content:center;width:100%;margin-top:20px}.anchor-table{width:100%;max-width:1100px;margin:0 auto;border-collapse:collapse;font-size:14px;text-align:center;background:white}.anchor-table th{padding:12px;border-bottom:2px solid #000;cursor:pointer;text-align:center}.anchor-table td{padding:12px;border-bottom:1px solid #ddd;text-align:center;vertical-align:middle}.anchor-table tr:hover{background-color:#f9f9f9}.diag-btn{font-size:12px;padding:6px 12px;background:#fff3e0;border:1px solid #ffb74d;color:#d84315;border-radius:4px;cursor:pointer;font-weight:bold;transition:all 0.2s;margin:0 auto}.diag-btn:hover{background:#ffe0b2;transform:scale(1.05)}.diag-btn-css{font-size:12px;padding:6px 12px;background:#f3e5f5;border:1px solid #ce93d8;color:#4a148c;border-radius:4px;cursor:pointer;font-weight:bold;transition:all 0.2s;margin:0 auto}.diag-btn-css:hover{background:#e1bee7;transform:scale(1.05)}</style>"""
ANCHOR_CONTENT = r"""<div class="proof-card" style="border-top: 5px solid #0277bd; max-width: 1200px; margin: 20px auto; text-align: center;"><h2 style="color:#01579b; border-bottom:2px solid #81d4fa; padding-bottom:10px; margin-top:0; display:inline-block;">⚓ ANCHOR: Autosomal Network Corroboration for Historical Origin Reconstruction</h2><div class="pill-nav no-print"><button id="btn-framework" class="pill active-framework" onclick="switchAnchorTab('framework')">📑 Framework & Theory</button><button id="btn-css" class="pill" onclick="switchAnchorTab('css')">🧬 Genetic Evidence (CSS)</button><button id="btn-docs" class="pill" onclick="switchAnchorTab('docs')">📄 Documentary Evidence (DOCS)</button><button id="btn-master" class="pill" onclick="switchAnchorTab('master')">⚓ Master ANCHOR Score</button></div><div id="tab-framework" class="anchor-tab-content" style="display:block;">__TAB_FRAMEWORK__</div><div id="tab-css" class="anchor-tab-content anchor-table-wrapper" style="display:none;">__TAB_CSS__</div><div id="tab-docs" class="anchor-tab-content anchor-table-wrapper" style="display:none;">__TAB_DOCS__</div><div id="tab-master" class="anchor-tab-content anchor-table-wrapper" style="display:none;">__TAB_MASTER__</div></div><script>window.runDiag=function(pName,AX,CC,BC,DC,TP){let msg=`Documentary Diagnostic: ${pName}\n------------------------------------------------------\n`;let issues=0;if(AX===0){msg+=`❌ [CRITICAL] Apex Reach Failed: Documented lineage does not reach the genetic target node.\n`;issues++;}if(CC<0.5){msg+=`⚠️ [WEAK] Citation Coverage: Only ${(CC*100).toFixed(0)}% of the pedigree spine has sources attached. Add GEDCOM sources.\n`;issues++;}if(BC<0.6){msg+=`⚠️ [WEAK] Birth Completeness: Significant missing birth dates or places in the spine.\n`;issues++;}if(DC<0.6){msg+=`⚠️ [WEAK] Death Completeness: Significant missing death dates or places in the spine.\n`;issues++;}if(TP<0.8){msg+=`⚠️ [WARNING] Temporal Plausibility: Check generation gaps or lifespans for unrealistic dates.\n`;issues++;}if(issues===0){msg+=`✅ [PLATINUM] Spine is robust, well-sourced, and temporally plausible. Excellent DOCS score.\n`;}else{msg+=`\nAction: Update your GEDCOM to resolve these warnings and recalculate.`;}alert(msg);} window.runCssDiag=function(pName,pm,br,dr){let msg=`Genetic Diagnostic: ${pName}\n------------------------------------------------------\n`;let passes=0;if(pm>=15){msg+=`✅ [PASS] Proper Matches: ${pm} (Requirement: 15+)\n`;passes++;}else{msg+=`❌ [FAIL] Proper Matches: ${pm} (Needs ${15-pm} more proper matches to study criteria)\n`;}if(br>=3){msg+=`✅ [PASS] Independent Branches: ${br} (Requirement: 3+)\n`;passes++;}else{msg+=`❌ [FAIL] Independent Branches: ${br} (Needs ${3-br} more independent branch line to prove triangulation)\n`;}if(dr>=1.5){msg+=`✅ [PASS] Dominance Ratio: ${dr.toFixed(1)} (Requirement: 1.5+)\n`;passes++;}else{msg+=`❌ [FAIL] Dominance Ratio: ${dr.toFixed(1)} (Needs a mathematically stronger primary signal vs secondary clusters)\n`;}if(passes===3){msg+=`\n🎯 CONCLUSION: Network is fully saturated and reliable. Proof threshold met.`;}else{msg+=`\n⚠️ ACTION REQUIRED: Target recruitment of DNA cousins from unrepresented branches to build volume and triangulate.`;}alert(msg);} window.switchAnchorTab=function(tab){document.querySelectorAll('.anchor-tab-content').forEach(el=>el.style.display='none');document.querySelectorAll('.pill').forEach(el=>{el.classList.remove('active-master','active-css','active-docs','active-framework');});let target=document.getElementById('tab-'+tab);if(tab==='framework'){target.style.display='block';}else{target.style.display='flex';}let btn=document.getElementById('btn-'+tab);if(tab==='master')btn.classList.add('active-master');if(tab==='css')btn.classList.add('active-css');if(tab==='docs')btn.classList.add('active-docs');if(tab==='framework')btn.classList.add('active-framework');}</script>"""

# 🌟 3. ACADEMIC PROOF CONSOLIDATOR (WITH SCALED MATRICES!) 🌟
CONSOLIDATOR_CSS = r"""<style>.consol-panel { background: #f3e5f5; border: 1px solid #ab47bc; padding: 25px; border-radius: 8px; margin-bottom: 25px; font-family: 'Segoe UI', sans-serif; text-align: center; } .consol-panel select { padding: 8px; font-size: 14px; width: 100%; border: 1px solid #7b1fa2; border-radius: 4px; } .consol-btn { background: #4a148c; color: white; border: none; padding: 12px 25px; font-size: 16px; font-weight: bold; border-radius: 4px; cursor: pointer; box-shadow: 0 4px 6px rgba(0,0,0,0.1); margin-top: 10px; } .consol-btn:hover { background: #38006b; } .vg-checkbox-container { height:150px; overflow-y:auto; border:1px solid #7b1fa2; background:white; border-radius:4px; padding:10px; font-size:13px; text-align:left; } .vg-checkbox-container label { display:block; margin-bottom:5px; cursor:pointer; } .vg-checkbox-container label:hover { background-color:#f3e5f5; } .academic-brief { background: white; max-width: 1100px; margin: 0 auto 30px auto; padding: 60px 80px; box-shadow: 0 4px 20px rgba(0,0,0,0.1); font-family: 'Georgia', serif; color: #000; line-height: 1.6; } .title-page { page-break-after: always; display: flex; flex-direction: column; justify-content: center; min-height: 70vh; padding: 20px; } .brief-section-title { font-size: 18px; text-transform: uppercase; border-bottom: 1px solid #ccc; padding-bottom: 5px; margin-top: 40px; margin-bottom: 20px; font-weight: bold; } .brief-table { width: 100%; border-collapse: collapse; margin-top: 15px; font-size: 13px; } .brief-table th { background: #f0f0f0; color: #000; border-bottom: 2px solid #000; border-top: 1px solid #000; padding: 10px; text-align: left; } .brief-table td { padding: 10px; border-bottom: 1px solid #ddd; vertical-align: middle; } @media print { .no-print { display: none !important; } body { background: white; padding: 0; } .academic-brief { box-shadow: none; padding: 0; max-width: 100%; border: none; margin-bottom: 0; } .matrix-page { page-break-before: always; page-break-inside: avoid; width: 100%; } .brief-table { font-size: 11px; width: 100%; table-layout: auto; } .brief-table th, .brief-table td { padding: 4px 6px; word-wrap: break-word; } }</style>"""
CONSOLIDATOR_HTML = f"""<div class="no-print consol-panel"><h2 style="color:#4a148c; margin-top:0;">Virtual Group & White Paper Builder</h2><div style="display:flex; justify-content:center; gap:20px; flex-wrap:wrap; margin-bottom:15px;"><div style="flex:1; max-width:400px; min-width:250px; text-align:left;"><label style="font-size:12px; font-weight:bold; color:#4a148c;">Select Kits for Virtual Group Analysis</label><div id="groupCheckboxes" class="vg-checkbox-container"></div></div><div style="flex:1; max-width:350px; min-width:250px; text-align:left;"><label style="font-size:12px; font-weight:bold; color:#4a148c;">Custom Group Name (e.g. "VA Yates Protocol")</label><input type="text" id="customGroupName" style="width:100%; box-sizing:border-box; padding:8px; border:1px solid #7b1fa2; border-radius:4px; margin-bottom:10px;" placeholder="Optional"><button class="consol-btn" style="width:100%; box-sizing:border-box;" onclick="runConsolidator('matrix')">📄 Generate Academic White Paper</button></div></div></div><div id="report-container"></div>"""
CONSOLIDATOR_JS = r"""<script>__JS_GLOBALS__
const PRECOMPUTED = typeof __PRECOMPUTED_JSON__ !== 'undefined' ? __PRECOMPUTED_JSON__ : [];
function cleanNum(str){let res="";for(let i=0;i<str.length;i++)if(str[i]>='0'&&str[i]<='9')res+=str[i];return res;}
document.addEventListener('DOMContentLoaded', function() { const groupDiv = document.getElementById('groupCheckboxes'); if(!groupDiv) return; const validTesters = DB.filter(r => r.t_names && r.t_names.trim() !== ""); const uniqueTesters = [...new Set(validTesters.map(r => r.participant))].sort((a, b) => { let keyA = DATA.participants[a] ? (DATA.participants[a].sort_key || a) : a; let keyB = DATA.participants[b] ? (DATA.participants[b].sort_key || b) : b; return keyA.toLowerCase().localeCompare(keyB.toLowerCase()); }); uniqueTesters.forEach(t => { let kcode = DATA.participants[t] ? DATA.participants[t].kit_code : ''; let displayStr = kcode ? `${t} [${kcode}]` : t; const lbl = document.createElement('label'); lbl.innerHTML = `<input type="checkbox" value="${t}" class="vg-checkbox"> ${displayStr}`; groupDiv.appendChild(lbl); }); });
const getStudyStats = () => { const d = new Date(); return `Study Data Current As Of: ${d.toLocaleDateString('en-US', { month: 'long', day: 'numeric', year: 'numeric' })} ${d.toLocaleTimeString('en-US', { hour: 'numeric', minute: '2-digit', timeZoneName: 'short' })} | Total Autosomal matches: ${DB.length.toLocaleString()}`; };
const getTitlePage = (testerArray, customName) => { const year = new Date().getFullYear(); const titleName = customName || (testerArray.length > 1 ? `Virtual Group (${testerArray.length} Kits)` : testerArray[0]); return `<div class="academic-brief title-page"><div style="font-family: Arial, sans-serif; text-align:center; line-height:1.6;"><h1 style="font-size:36px; border-bottom:none; margin-bottom:5px;">Collateral Saturation</h1><h2 style="font-size:20px; font-weight:normal; color:#444; margin-top:0;">A Quantitative Method for Autosomal Lineage Reconstruction</h2><br><br><br><p style="font-size:18px;"><b>Ronald Eugene Yates, MPH</b><br>University of California, Los Angeles<br>1975</p><br><br><br><p style="font-size:16px;">Yates DNA Study<br>Autosomal Lineage Reconstruction Project</p><br><br><br><h3 style="color:#4a148c;">Analysis Target: ${titleName}</h3><br><br><p style="font-size:16px;">${year}</p><p style="font-size:14px; color:#004d40; margin-top:20px; font-weight:bold;">${getStudyStats()}</p><br><br><br><p style="font-size:14px; color:#555;">&copy; ${year} Ronald Eugene Yates<br>All Rights Reserved.</p></div></div>`; };
const getMethodologyPage = () => { return `<div class="academic-brief" style="page-break-before: always;"><h2 style="color: #4a148c; border-bottom: 2px solid #ccc; padding-bottom: 5px; margin-top:0; font-size:22px; text-transform:uppercase;">Methodological Principles of Collateral Saturation</h2><p style="font-size:15px; line-height:1.6; color:#333; margin-top:20px;"><b>Collateral Saturation</b> is a lineage-validation method in which autosomal DNA evidence is evaluated at the level of descendant networks rather than isolated matches. A lineage hypothesis is considered reliable when it is supported by sufficient descendant density, replicated across independent branches, and remains stable under perturbation tests.</p><ol style="line-height:1.8;"><li><b>Minimum Descendant Count (PM &ge; 15):</b> The network must have sufficient representation to filter noise.</li><li><b>Branch Independence (BR &ge; 3):</b> Triangulation must occur across distinct, non-overlapping descendent lines.</li><li><b>Dominance Ratio (DR &ge; 1.5):</b> The primary genetic signal must clearly overpower secondary pedigree collapse signals.</li></ol></div>`; };
const getVirtualGroupPage = (testerArray) => { if (testerArray.length <= 1) return ""; let kitsHtml = testerArray.map(t => { let kcode = DATA.participants[t] ? DATA.participants[t].kit_code : ''; return `<li>${t} ${kcode ? `[${kcode}]` : ''}</li>`; }).join(""); return `<div class="academic-brief" style="page-break-before: always;"><h2 style="color: #4a148c; border-bottom: 2px solid #ccc; padding-bottom: 5px; margin-top:0; font-size:22px; text-transform:uppercase;">Virtual Group Protocol</h2><p style="font-size:15px; line-height:1.6;">In cases where isolated DNA kits lack sufficient power to achieve Collateral Saturation independently, multiple verifiable descendants of a specific ancestor can be logically joined into a <b>Virtual Group</b>. This protocol aggregates their match networks, treating them as a single proof-grade evaluation unit to reconstruct the older lineage.</p><h3 style="margin-top:20px; font-size:16px;">Kits Formally Merged for this Analysis (${testerArray.length}):</h3><ul style="line-height:1.6; color:#111;">${kitsHtml}</ul></div>`; };

function getMatrixHTML(vgCSS, selectedKits) {
    let matrixRows = PRECOMPUTED.filter(r => selectedKits.includes(r.pName));
    if (vgCSS && vgCSS.isGroup) matrixRows.push(vgCSS);
    let html = "";

    matrixRows.sort((a,b) => b.ANCHOR - a.ANCHOR);
    let t1 = `<table class="brief-table"><thead><tr style="background:#fff3e0;"><th style="text-align:left; color:#d84315;">Participant Kit</th><th>Primary Ancestor</th><th>DNA Score</th><th>Paper Score</th><th style="background:#d84315; color:white;">ANCHOR Score</th></tr></thead><tbody>`;
    matrixRows.forEach(r => { let nameFmt = r.isGroup ? `<span style="color:#d84315; font-weight:bold;">★ ${r.pName}</span>` : `<strong>${r.pName}</strong>`; t1 += `<tr><td style="text-align:left;">${nameFmt}</td><td>${r.targetAnc}</td><td style="color:#4a148c; font-weight:bold;">${r.cssFinal.toFixed(1)}</td><td style="color:#5d4037; font-weight:bold;">${r.DOCS.toFixed(1)}</td><td style="background:#fbe9e7; font-weight:bold; color:#d84315; font-size:1.1em;">${r.ANCHOR.toFixed(2)}</td></tr>`; });
    t1 += `</tbody></table>`;
    html += `<div class="academic-brief matrix-page"><div class="brief-section-title">Master ANCHOR Evaluation Matrix</div>${t1}</div>`;

    matrixRows.sort((a,b) => b.cssFinal - a.cssFinal);
    let t2 = `<table class="brief-table"><thead><tr style="background:#f3e5f5;"><th style="text-align:left; color:#4a148c;">Participant Kit</th><th>PM</th><th>HC-T</th><th>DR</th><th>BR</th><th>NS</th><th>Status</th><th style="background:#4a148c; color:white;">CSS Score</th></tr></thead><tbody>`;
    matrixRows.forEach(r => { let nameFmt = r.isGroup ? `<span style="color:#d84315; font-weight:bold;">★ ${r.pName}</span>` : `<strong>${r.pName}</strong>`; let status_color = r.ST_str === "PASS" ? "green" : (r.ST_str === "PARTIAL" ? "orange" : "red"); let status_badge = `<span style="color:${status_color}; font-weight:bold;">${r.ST_str}</span>`; t2 += `<tr><td style="text-align:left;">${nameFmt}</td><td>${r.PM}</td><td>${r.HC_T}</td><td>${r.DR.toFixed(1)}</td><td>${r.BR}</td><td>${r.NS}</td><td>${status_badge}</td><td style="background:#f3e5f5; font-weight:bold; color:#4a148c; font-size:1.1em;">${r.cssFinal.toFixed(2)}</td></tr>`; });
    t2 += `</tbody></table>`;
    html += `<div class="academic-brief matrix-page"><div class="brief-section-title">Genetic Evidence (CSS) Matrix</div>${t2}</div>`;

    matrixRows.sort((a,b) => b.DOCS - a.DOCS);
    let t3 = `<table class="brief-table"><thead><tr style="background:#efebe9;"><th style="text-align:left; color:#3e2723;">Participant Kit</th><th>Generations</th><th>Vital Completeness</th><th>Citation Coverage</th><th>Temporal Plausibility</th><th style="background:#5d4037; color:white;">DOCS Score</th></tr></thead><tbody>`;
    matrixRows.forEach(r => { let nameFmt = r.isGroup ? `<span style="color:#d84315; font-weight:bold;">★ ${r.pName}</span>` : `<strong>${r.pName}</strong>`; let vts = `${((r.BC + r.DC)/2 * 100).toFixed(0)}%`; let cts = `${(r.CC * 100).toFixed(0)}%`; let tps = `${(r.TP * 100).toFixed(0)}%`; t3 += `<tr><td style="text-align:left;">${nameFmt}</td><td>${r.GD}</td><td>${vts}</td><td>${cts}</td><td>${tps}</td><td style="background:#efebe9; font-weight:bold; color:#5d4037; font-size:1.1em;">${r.DOCS.toFixed(1)}</td></tr>`; });
    t3 += `</tbody></table>`;
    html += `<div class="academic-brief matrix-page"><div class="brief-section-title">Documentary Evidence (DOCS) Matrix</div>${t3}</div>`;

    return html;
}

const getAppendixA = () => { return `<div class="academic-brief matrix-page" style="max-width: 1200px; text-align: left;"><h2 style="color: #4a148c; border-bottom: 2px solid #ccc; padding-bottom: 5px; margin-top:0; font-size:22px; text-transform:uppercase;">Appendix A (Addendum): ANCHOR Documentary Evidence Matrix (GEDCOM-Derived)</h2><p style="font-size:14px;"><b>Purpose:</b> These fields quantify documentary pedigree robustness using information commonly available in a standard GEDCOM (vital events, places, sources/citations, and plausibility checks). These measures are combined into <b>DOCS</b> (Documentary Score), which is then blended with <b>CSS v2a</b> to produce the combined <b>ANCHOR</b> score.</p><h3 style="color: #4a148c; margin-top: 30px;">A5. ANCHOR Documentary Matrix Fields</h3><table class="brief-table" style="font-size: 13px;"><thead><tr><th>Field</th><th>Abbrev</th><th>Definition</th><th>Computation</th><th>Desired Range</th></tr></thead><tbody><tr><td><b>Apex Reach</b></td><td>AX</td><td>Whether the participant’s documented spine reaches the target ancestral node (prevents pedigree-depth bias).</td><td>AX = 1 if targetID is present in participant t_ids; else 0 (gate/penalty).</td><td><b>Must be 1</b> for proof-grade documentary scoring.</td></tr><tr><td><b>GEDCOM Depth</b></td><td>GD</td><td>Generations/person-nodes in the scored spine segment (participant &rarr; target node).</td><td>GD = count(spineIDs) (log-capped normalization).</td><td>8–14 typical; higher is better if sourced.</td></tr><tr><td><b>Node Coverage</b></td><td>NC</td><td>Fraction of spine persons meeting minimum documentation presence.</td><td>NC = (# persons with any vital/place/source) / (# persons scored).</td><td>&ge;0.70 good; &ge;0.85 strong.</td></tr><tr><td><b>Birth Completeness</b></td><td>BC</td><td>Completeness of birth facts across the spine.</td><td>Per person: 1.0 if birth date+place; 0.6 if one present; 0 if none; BC=mean.</td><td>&ge;0.60 good; &ge;0.80 strong.</td></tr><tr><td><b>Death Completeness</b></td><td>DC</td><td>Completeness of death facts across the spine.</td><td>Per person: 1.0 if death date+place; 0.6 if one present; 0 if none; DC=mean.</td><td>&ge;0.50 good; &ge;0.75 strong.</td></tr><tr><td><b>Date Quality</b></td><td>DQ</td><td>Precision/quality of dates (prefers exactness; penalizes ambiguity).</td><td>Full date (YYYY-MM-DD) &gt; month/year &gt; year-only &gt; qualified (ABT/BEF/AFT) &gt; missing; DQ=mean.</td><td>&ge;0.60 good; &ge;0.80 strong.</td></tr><tr><td><b>Place Quality</b></td><td>PQ</td><td>Granularity/quality of place strings (more locality levels = stronger).</td><td>&ge;3 comma-levels=1.0; 2=0.7; 1=0.4; none=0; PQ=mean.</td><td>&ge;0.60 good; &ge;0.80 strong.</td></tr><tr><td><b>Geo Consistency</b></td><td>GC</td><td>Stability/coherence of locations across adjacent generations (flags implausible “jumps”).</td><td>GC = fraction of adjacent gen-pairs sharing at least one place token (county/state/country heuristic).</td><td>&ge;0.60 good; &ge;0.75 strong.</td></tr><tr><td><b>Citation Coverage</b></td><td>CC</td><td>Fraction of spine persons with &ge;1 citation/source (core proof requirement).</td><td>CC = (# persons with sources/citations) / (# persons scored).</td><td>&ge;0.50 good; &ge;0.70 strong.</td></tr><tr><td><b>Source Density</b></td><td>SD</td><td>Overall density of sources/citations across the scored spine.</td><td>SD = log_norm(total_sources + total_citations, cap&approx;50).</td><td>&ge;0.40 good; &ge;0.65 strong.</td></tr><tr><td><b>Temporal Plausibility</b></td><td>TP</td><td>Sanity checks for time realism (birth/death order, lifespan bounds, generation spacing).</td><td>TP = 1 &minus; (violations / checks), clipped 0–1.</td><td>&ge;0.85 good; &ge;0.95 strong.</td></tr><tr><td><b>Identity Collision Multiplier</b></td><td>IDm</td><td>Penalty when spine contains duplicate IDs or internal contradictions (signals GEDCOM hygiene issues).</td><td>IDm = 1.0 (none); 0.85 (minor); 0.70 (severe).</td><td>Desired 1.0.</td></tr></tbody></table><h3 style="color: #4a148c; margin-top: 30px;">A6. Composite Documentary Score</h3><p style="font-size:14px;"><b>DOCS</b> is a 0–100 composite that summarizes documentary strength:</p><table class="brief-table" style="font-size: 13px;"><thead><tr><th>Composite</th><th>Abbrev</th><th>Definition</th><th>Computation</th><th>Interpretation</th></tr></thead><tbody><tr><td><b>Documentary Score</b></td><td>DOCS</td><td>Normalized, weighted documentary robustness of the scored spine (GEDCOM-derived).</td><td>DOCS = 100 &times; weighted_mean(AX, GD, NC, BC, DC, DQ, PQ, GC, CC, SD, TP) &times; IDm<br><span style="color:#555;">Recommended weights: CC=2.0; SD=1.5; TP=1.5; AX=2.0; others=1.0 (v1).</span><br><span style="color:#b71c1c;"><b>AX Gate:</b> if AX=0, DOCS is capped (e.g., &le;35) to prevent strong scores from incomplete pedigrees.</span></td><td>85–100 Platinum; 70–85 Strong; 50–70 Moderate; 35–50 Weak; &lt;35 Insufficient.</td></tr></tbody></table><h3 style="color: #4a148c; margin-top: 30px;">A7. ANCHOR Combined Score (DNA + Documentary)</h3><table class="brief-table" style="font-size: 13px;"><thead><tr><th>Composite</th><th>Abbrev</th><th>Definition</th><th>Computation</th><th>Interpretation</th></tr></thead><tbody><tr><td><b>ANCHOR Score</b></td><td>ANCH</td><td>Combined proof-strength score blending CSS v2a (DNA network evidence) with DOCS (GEDCOM documentary evidence).</td><td><b>v1 (simple):</b> ANCH = 0.65 &times; CSSv2a + 0.35 &times; DOCS<br><b>v1b (with synergy):</b> ANCH = min(100, blend + 10 &times; (min(CSSv2a, DOCS)/100))<br><span style="color:#555;">Synergy rewards agreement (high DNA + high documentary support).</span></td><td>Higher scores indicate both a saturated genetic network and a robust documented spine. ANCH is intended for cross-participant ranking and publication-grade reporting.</td></tr></tbody></table><p style="font-size:12px; color:#666; margin-top:18px;"><b>Implementation note:</b> DOCS uses only fields commonly found in GEDCOM (birth/death facts, places, sources/citations). Where GEDCOM detail is sparse, DOCS will appropriately remain low, preventing documentary overconfidence.</p></div>`; };
window.runConsolidator = function(mode) { if(mode === 'matrix') { const boxes = document.querySelectorAll('.vg-checkbox:checked'); const selectedKits = Array.from(boxes).map(b => b.value); let customName = document.getElementById('customGroupName').value.trim(); if (customName === "") customName = null; let vgCSS = null; if (selectedKits.length > 0) { vgCSS = getCSS(selectedKits, customName); } document.title = "Academic_White_Paper"; let reportHTML = getTitlePage(selectedKits, customName) + getMethodologyPage() + getVirtualGroupPage(selectedKits) + getMatrixHTML(vgCSS, selectedKits) + getAppendixA(); document.getElementById('report-container').innerHTML = reportHTML; setTimeout(() => { if(window.init) window.init(); }, 100); } }
</script>"""

# 🌟 4. THE UNIFIED PROOF ENGINE 🌟
PROOF_ENGINE_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Forensic Proof Engine</title><style>__CSS_BASE__ .search-tabs{display:flex;justify-content:center;flex-wrap:wrap;gap:10px;margin-bottom:25px} .search-tab{padding:12px 20px;border:2px solid #ddd;background:#fff;cursor:pointer;font-weight:bold;border-radius:25px;color:#555;font-size:15px;transition:all 0.2s;box-shadow:0 2px 4px rgba(0,0,0,0.05)} .search-tab:hover{background:#f4f4f4;transform:translateY(-2px)} .search-tab.active-part{border-color:#4a148c;background:#4a148c;color:white;box-shadow:0 4px 10px rgba(74,20,140,0.3)} .search-tab.active-anc{border-color:#006064;background:#006064;color:white;box-shadow:0 4px 10px rgba(0,96,100,0.3)} .search-tab.active-id{border-color:#b71c1c;background:#b71c1c;color:white;box-shadow:0 4px 10px rgba(183,28,28,0.3)} .search-box{display:flex;gap:10px;justify-content:center;margin-bottom:20px} .search-box select,.search-box input{padding:15px;width:100%;max-width:400px;border:2px solid #ccc;border-radius:8px;font-size:16px;box-shadow:inset 0 1px 3px rgba(0,0,0,0.05)} .search-box button{padding:15px 25px;background:#b71c1c;color:white;border:none;border-radius:8px;cursor:pointer;font-weight:bold;font-size:16px;transition:all 0.2s} .search-box button:hover{background:#d32f2f;transform:scale(1.05)}</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#4a148c;">🔬 Forensic Proof Engine</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card no-print" style="border-top: 5px solid #4a148c;"><div class="search-tabs"><button id="tab-part" class="search-tab" onclick="setMode('part')">👤 1. Participant View</button><button id="tab-anc" class="search-tab" onclick="setMode('anc')">🌳 2. Ancestor View</button><button id="tab-id" class="search-tab" onclick="setMode('id')">🔍 3. Deep Path Dive (ID#)</button></div><div id="search-ui"></div></div><div class="proof-card" id="proof-result" style="display:none; margin-top:20px; padding:0; border:none; box-shadow:none; background:transparent;"></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
window.makeCascade = function(lin) { let parts = String(lin).split('->'); let h = '<div style="text-align:left; font-size:13px; line-height:1.6; font-family:\'Georgia\',serif; margin:8px 0;">'; parts.forEach((p, i) => { let pad = i * 15; let prfx = i === 0 ? '' : '&uarr; '; let fw = (i === 0) ? 'font-weight:bold; color:#000;' : 'color:#444;'; h += `<div style="margin-left:${pad}px; ${fw}">${prfx}${p.trim()}</div>`; }); h += '</div>'; return h; };
let mode = 'part'; function setMode(newMode) { mode = newMode; document.getElementById('tab-part').className = 'search-tab' + (mode==='part'?' active-part':''); document.getElementById('tab-anc').className = 'search-tab' + (mode==='anc'?' active-anc':''); document.getElementById('tab-id').className = 'search-tab' + (mode==='id'?' active-id':''); document.getElementById('proof-result').style.display = 'none'; let html = ''; if(mode === 'part') { const keys = Object.keys(DATA.participants).sort((a,b) => { let keyA = DATA.participants[a].sort_key || a; let keyB = DATA.participants[b].sort_key || b; return keyA.toLowerCase().localeCompare(keyB.toLowerCase()); }); html = `<div class="search-box"><select id="querySelect" onchange="runSearch()" style="border-color:#4a148c;"><option value="">-- Select a Study Participant --</option>`; keys.forEach(k => { let code = DATA.participants[k].kit_code || ''; let dStr = code ? `${DATA.participants[k].name} [${code}]` : DATA.participants[k].name; html += `<option value="${k}">${dStr}</option>`; }); html += `</select></div>`; } else if(mode === 'anc') { const keys = Object.keys(DATA.ancestors).sort((a,b) => DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name)); html = `<div class="search-box"><select id="querySelect" onchange="runSearch()" style="border-color:#006064;"><option value="">-- Select an Ancestral Line --</option>`; keys.forEach(k => { html += `<option value="${k}">${DATA.ancestors[k].name} (${DATA.ancestors[k].matches} proper matches)</option>`; }); html += `</select></div>`; } else if(mode === 'id') { html = `<div class="search-box"><input type="text" id="queryInput" placeholder="Enter GEDCOM ID (e.g. I1234) or Name..." style="border-color:#b71c1c;" onkeypress="if(event.key==='Enter') runSearch()"> <button onclick="runSearch()">Deep Search</button></div>`; } document.getElementById('search-ui').innerHTML = html; }
window.runSearch = function() { let resDiv = document.getElementById('proof-result'); let html = ''; if(mode === 'part') { let k = document.getElementById('querySelect').value; if(!k) { resDiv.style.display='none'; return; } let p = DATA.participants[k]; let cStat = p.css_status ? p.css_status : 'UNSCORED'; let cssColor = cStat === 'PASS' ? '#2e7d32' : (cStat === 'PARTIAL' ? '#ef6c00' : '#c62828'); html = `<div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1);"><div style="background:#f3e5f5; padding:20px; border-radius:8px; border-left:5px solid #ab47bc; margin-bottom:20px;"><h2 style="margin-top:0; color:#4a148c;">Participant Profile: ${p.name}</h2><p><strong>Status:</strong> <span class="badge" style="background:#4a148c; color:white;">${p.badge}</span> &nbsp; <strong>CSSv2 Validation:</strong> <span class="badge" style="background:${cssColor}; color:white;">${cStat}</span></p><p><strong>Total Evidence Mass:</strong> ${p.cm} cM corroborating ${p.matches} node connections.</p></div><h3 style="color:#4a148c; border-bottom:2px solid #ccc; padding-bottom:5px;">Confirmed Ancestral Intersections</h3><table class="brief-table"><thead><tr><th>Target Node</th><th>Shared cM</th><th>Participant's Triangulation Path</th></tr></thead><tbody>`; let matches = DB.filter(m => m.participant === p.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0)); matches.forEach(m => { html += `<tr><td style="width:25%;"><strong>${m.ancestor}</strong></td><td style="width:10%; color:#4a148c; font-weight:bold;">${m.cm} cM</td><td>${makeCascade(m.lineage)}</td></tr>`; }); html += `</tbody></table></div>`; } else if(mode === 'anc') { let k = document.getElementById('querySelect').value; if(!k) { resDiv.style.display='none'; return; } let a = DATA.ancestors[k]; html = `<div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1);"><div style="background:#e0f7fa; padding:20px; border-radius:8px; border-left:5px solid #00acc1; margin-bottom:20px;"><h2 style="margin-top:0; color:#006064;">Biological Proof: ${a.name}</h2><p><strong>Forensic Validation:</strong> <span class="badge badge-${a.badge.toLowerCase()}">${a.badge} Standard</span></p><p><strong>Integrity Score:</strong> ${a.integrity}% (Verified by ${a.testers} independent kits)</p><p><strong>Total Evidence:</strong> ${a.cm} cM shared across ${a.matches} matching paths.</p></div><table class="brief-table"><thead><tr><th>Matching Kit</th><th>Shared cM</th><th>Documented Lineage Path</th></tr></thead><tbody>`; let matches = DB.filter(m => m.ancestor === a.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0)); matches.forEach(m => { html += `<tr><td style="width:25%;"><strong>${m.participant}</strong></td><td style="width:10%; color:#006064; font-weight:bold;">${m.cm} cM</td><td>${makeCascade(m.lineage)}</td></tr>`; }); html += `</tbody></table></div>`; } else if(mode === 'id') { let q = document.getElementById('queryInput').value.trim().toLowerCase(); if(!q) { resDiv.style.display='none'; return; } let qNum = q.replace(/[^0-9]/g, ''); let matches = DB.filter(m => { if(qNum && m.search_ids && m.search_ids.split(',').map(x=>x.replace(/[^0-9]/g,'')).includes(qNum)) return true; if(m.lineage && m.lineage.toLowerCase().includes(q)) return true; if(m.id && m.id.toLowerCase().includes(q)) return true; return false; }).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0)); html = `<div style="background:white; padding:30px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1);"><div style="background:#ffebee; padding:20px; border-radius:8px; border-left:5px solid #b71c1c; margin-bottom:20px;"><h2 style="margin-top:0; color:#b71c1c;">Deep Path Search: "${q}"</h2><p style="font-size:16px;">Found <strong>${matches.length}</strong> lineage connections passing through this node.</p></div>`; if(matches.length > 0) { html += `<table class="brief-table"><thead><tr><th style="color:#b71c1c;">Participant</th><th style="color:#b71c1c;">Primary Target</th><th style="color:#b71c1c;">cM</th><th style="color:#b71c1c;">Intersecting Lineage Path</th></tr></thead><tbody>`; matches.forEach(m => { let hlPath = m.lineage; if(qNum) { let regex = new RegExp(`\\(I?${qNum}\\)`, 'gi'); hlPath = hlPath.replace(regex, match => `<mark style="background:#ffcdd2; color:#b71c1c; font-weight:bold; padding:2px 4px; border-radius:3px;">${match}</mark>`); } else if (q.length > 3) { let regex = new RegExp(`(${q})`, 'gi'); hlPath = hlPath.replace(regex, match => `<mark style="background:#ffcdd2; color:#b71c1c; font-weight:bold; padding:2px 4px; border-radius:3px;">${match}</mark>`); } html += `<tr><td style="width:20%;"><strong>${m.participant}</strong></td><td style="width:20%;"><strong>${m.ancestor}</strong></td><td style="width:5%; color:#b71c1c; font-weight:bold;">${m.cm}</td><td>${makeCascade(hlPath)}</td></tr>`; }); html += `</tbody></table></div>`; } else { html += `<p style="text-align:center; color:#777; font-style:italic;">No matching kits found passing through this ID or name. Note: Uncorroborated singletons may not appear in this index.</p></div>`; } } resDiv.innerHTML = html; resDiv.style.display = 'block'; } document.addEventListener('DOMContentLoaded', () => setMode(mode));
</script>"""

# 🌟 5. DOSSIER UI (WITH LIVE AUDIT HOOKS) 🌟
DOSS_TMPL = r"""<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"><title>Forensic Dossier</title><style>__CSS_BASE__</style></head><body><div class="wrap"><h1 class="centerline no-print" style="margin-top:30px; color:#004d40;">📁 Forensic Dossier</h1><div id="nav-slot" class="no-print">__STATS_BAR____NAV_HTML__</div><div class="proof-card"><div id="dos-ui" class="no-print"></div><div id="report-stack"></div></div>__LEGAL_FOOTER__</div>
<script>__JS_GLOBALS__;
window.makeCascade = function(lin) { let parts = String(lin).split('->'); let h = '<div style="text-align:left; font-size:13px; line-height:1.6; font-family:\'Georgia\',serif; margin:8px 0;">'; parts.forEach((p, i) => { let pad = i * 15; let prfx = i === 0 ? '' : '&uarr; '; let fw = (i === 0) ? 'font-weight:bold; color:#000;' : 'color:#444;'; h += `<div style="margin-left:${pad}px; ${fw}">${prfx}${p.trim()}</div>`; }); h += '</div>'; return h; };
document.addEventListener('DOMContentLoaded', function() {
    const dosKeys = Object.keys(DATA.participants).sort((a,b) => { let keyA = DATA.participants[a].sort_key || a; let keyB = DATA.participants[b].sort_key || b; return keyA.toLowerCase().localeCompare(keyB.toLowerCase()); });
    let sel = '<select id="dosSelect" onchange="renderDossier()" style="padding:12px; width:100%; border:2px solid #004d40; border-radius:4px; font-size:16px; margin-bottom:20px;"><option value="">-- Select Kit to Generate Formal Dossier --</option>'; dosKeys.forEach(k => { let code = DATA.participants[k].kit_code || ''; let displayStr = code ? `${DATA.participants[k].name} [${code}]` : DATA.participants[k].name; sel += `<option value="${k}">${displayStr}</option>`; }); sel += '</select>'; document.getElementById('dos-ui').innerHTML = sel;

    window.renderDossier = function() {
        let k = document.getElementById('dosSelect').value; if(!k) { document.getElementById('report-stack').innerHTML = ''; return; }
        let p = DATA.participants[k];
        let matches = DB.filter(m => m.participant === p.name).sort((x,y) => parseInt(y.cm||0) - parseInt(x.cm||0));
        let topAnc = matches.length > 0 ? matches[0].ancestor : "None Found";

        let cStat = p.css_status ? p.css_status : 'UNSCORED';
        let cssColor = cStat === 'PASS' ? '#2e7d32' : (cStat === 'PARTIAL' ? '#ef6c00' : '#c62828');

        // 1. Live TNG Link Assembly
        let topAncDisplay = topAnc;
        if(p.target_id && p.target_id !== 'None' && p.target_id !== '') {
            let pureId = String(p.target_id).replace(/[^0-9]/g, '');
            topAncDisplay = `<a href="https://yates.one-name.net/tng/verticalchart.php?personID=I${pureId}&tree=tree1&parentset=0&display=vertical&generations=15" target="_blank" style="color:#004d40; text-decoration:underline;">${topAnc}</a>`;
        }

        // 2. Pedigree Diagnostic Audit Box Assembly
        let diagHtml = '';
        if(p.diag_log && p.diag_log.length > 0) {
            diagHtml = `<div style="margin-top:20px; background:#fff3e0; padding:15px; border-left:4px solid #ff9800;">
                <h4 style="margin-top:0; color:#e65100;">Pedigree Audit Flags (DOCS Score: ${parseFloat(p.docs_score).toFixed(1)})</h4>
                <ul style="margin-bottom:0; font-size:14px; color:#333;">
                    ${p.diag_log.map(msg => `<li style="margin-bottom:5px;">${msg}</li>`).join('')}
                </ul>
            </div>`;
        } else if (p.docs_score >= 85) {
             diagHtml = `<div style="margin-top:20px; background:#e8f5e9; padding:15px; border-left:4px solid #4caf50;">
                <h4 style="margin-top:0; color:#2e7d32;">Pedigree Audit (DOCS Score: ${parseFloat(p.docs_score).toFixed(1)})</h4>
                <p style="margin:0; font-size:14px; color:#333;">✅ <strong>Platinum Standard Pedigree.</strong> The documented spine is fully sourced with zero temporal anomalies.</p>
            </div>`;
        }

        let html = `<div style="border: 3px double #004d40; padding: 40px; background: white; margin-top:20px;">
            <div style="text-align:center; border-bottom:2px solid #004d40; padding-bottom:20px; margin-bottom:30px;">
                <h1 style="color:#004d40; text-transform:uppercase; margin:0; font-size:28px;">Forensic Evidence Dossier</h1>
                <p style="margin:5px 0 0 0; color:#555; font-style:italic;">Yates DNA Study Lineage Reconstruction</p>
            </div>
            <div style="font-size:16px; line-height:1.8; background:#f4f4f4; padding:20px; border:1px solid #ddd; margin-bottom:30px;">
                <p style="margin:0;"><strong>SUBJECT IDENTIFIER:</strong> ${p.name}</p>
                <p style="margin:0;"><strong>CSSv2 VALIDATION STATUS:</strong> <span style="color:${cssColor}; font-weight:bold;">${cStat}</span></p>
                <p style="margin:0;"><strong>EVIDENCE INTEGRITY SCORE:</strong> ${p.integrity}%</p>
                <p style="margin:0;"><strong>PRIMARY CORROBORATED NODE:</strong> ${topAncDisplay}</p>
                <hr style="border-top:1px solid #ccc; margin:15px 0;">
                <p style="margin:0;"><strong>EXECUTIVE SUMMARY:</strong> This subject shares ${p.cm} cM of autosomal DNA across ${p.matches} independently verified ancestral nodes within the Yates study. The empirical data confirms the biological validity of the subject's descent pathway. This lineage is anchored by a target node possessing a Node Saturation (NS) of <strong>${p.ns}</strong>, validating the connection across <strong>${p.br}</strong> independent descendant branches.</p>
                ${diagHtml}
            </div>
            <h3 style="color:#004d40; text-transform:uppercase; font-size:18px;">Cross-Referenced Match Index</h3>
            <table class="brief-table"><thead><tr><th style="background:#e0f2f1;">Intersected Study Node</th><th style="background:#e0f2f1;">cM</th><th style="background:#e0f2f1;">Documented Route to Node</th></tr></thead><tbody>`;
            matches.forEach(m => { html += `<tr><td style="width:30%;"><strong>${m.ancestor}</strong></td><td style="width:10%; color:#004d40; font-weight:bold;">${m.cm}</td><td>${makeCascade(m.lineage)}</td></tr>`; });
            html += '</tbody></table></div>';
            document.getElementById('report-stack').innerHTML = html;
    }
});</script></body></html>"""

# 🌟 6. EXPLICIT PLACEHOLDERS 🌟
CONTENTS_CONTENT = r"""<style>.guide-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(300px,1fr));gap:25px;max-width:1200px;margin:30px auto;font-family:sans-serif;}.guide-card{background:white;padding:25px;border-radius:8px;border-left:5px solid #006064;box-shadow:0 4px 10px rgba(0,0,0,0.05);transition:transform 0.2s;text-align:left;}.guide-card:hover{transform:translateY(-5px)}.card-title{font-size:1.4em;font-weight:bold;color:#004d40;margin-top:0}.card-why{color:#b71c1c;font-weight:bold;margin:10px 0 5px 0;font-size:0.9em;text-transform:uppercase}.card-what{color:#555;font-size:1em;line-height:1.5;margin-bottom:20px}.card-btn{display:inline-block;padding:10px 20px;background:#00838f;color:white;text-decoration:none;border-radius:4px;font-weight:bold}.card-btn:hover{background:#006064}</style><div style="text-align:center;max-width:800px;margin:0 auto 20px auto;color:#444;font-size:1.1em;font-family:sans-serif;">This site transforms raw DNA data into forensic genealogical evidence.</div><div class="guide-grid"><div class="guide-card"><h2 class="card-title">1. The DNA Register</h2><div class="card-why">Why View This?</div><div class="card-what">To see the raw evidence. This is the master list of all matches in the study, sorted by ancestral line.</div><a href="ons_yates_dna_register.shtml" class="card-btn">Open Register</a></div><div class="guide-card"><h2 class="card-title">2. DNA Network</h2><div class="card-why">Why View This?</div><div class="card-what">To see the big picture. Visual clusters showing which ancestral lines are genetically proven by multiple testers.</div><a href="dna_network.html" class="card-btn">View Network</a></div><div class="guide-card"><h2 class="card-title">3. Lineage Proof Engine</h2><div class="card-why">Why View This?</div><div class="card-what">To verify a connection. An interactive tool that tests if a specific ancestor is biologically confirmed.</div><a href="proof_engine.html" class="card-btn">Run Proof</a></div><div class="guide-card" style="border-left-color:#f57f17;"><h2 class="card-title" style="color:#e65100;">4. Brick Wall Buster</h2><div class="card-why" style="color:#bf360c;">Why View This?</div><div class="card-what">Break through a dead end. Suggests which family line you likely belong to based on match dominance.</div><a href="proof_engine.html" class="card-btn" style="background:#ef6c00;">Bust This Wall</a></div><div class="guide-card"><h2 class="card-title">5. Forensic Dossier</h2><div class="card-why">Why View This?</div><div class="card-what">Get your "Scorecard." Generate forensic reports on yourself or an ancestor, grading evidence strength.</div><a href="dna_dossier.html" class="card-btn">Create Dossier</a></div><div class="guide-card"><h2 class="card-title">6. Research Admin Hub</h2><div class="card-why">Why View This?</div><div class="card-what">For study managers. High-level audit showing participant statistics, masked IDs, and study metrics.</div><a href="research_admin.html" class="card-btn" style="background:#455a64;">Admin Access</a></div><div class="guide-card"><h2 class="card-title">7. Data Glossary</h2><div class="card-why">Why View This?</div><div class="card-what">Understand the terms. Definitions for forensic terms like "Keystone Tester" and "Spanish naming."</div><a href="data_glossary.shtml" class="card-btn" style="background:#78909c;">Read Glossary</a></div><div class="guide-card" style="border-left-color:#fbc02d;"><h2 class="card-title">8. Corrections</h2><div class="card-why">See Something?</div><div class="card-what">Genealogy is collaboration. If you can solve a mystery, tell us. Include the <strong>Person ID (e.g. I1234)</strong>.</div><a href="mailto:ron@yates.one-name.net" class="card-btn" style="background:#f9a825;color:#333;">Email Correction</a></div></div>"""
SHARE_CONTENT = r"""<div style="max-width:900px; margin:0 auto; font-family:sans-serif; color:#333;"><div style="background:white; padding:40px; border-radius:8px; box-shadow:0 2px 10px rgba(0,0,0,0.1);"><h2 style="color:#006064; margin-top:0; border-bottom:2px solid #00838f; padding-bottom:10px;">Share Your Ancestry DNA Matches</h2><p style="font-size:1.1em; line-height:1.6;">Ancestry provides a built-in sharing feature that allows you to grant limited access to your DNA matches <strong>without sharing your personal account details</strong>. You remain in full control of your account at all times.</p><h3 style="color:#00838f; margin-top:30px;">How sharing works on Ancestry</h3><p>From your AncestryDNA Settings page, you may invite another individual by email and assign them the <strong>Collaborator</strong> role. This allows the study administrator to analyze your match list to build Forensic Handshakes.</p><div style="background:#f1f8e9; padding:25px; border-radius:8px; border:1px solid #c5e1a5; margin-top:30px;"><h3 style="color:#33691e; margin-top:0;">How to share for the Yates One-Name Study</h3><ol style="line-height:1.8; font-size:1.1em; color:#333;"><li>Open your <strong>AncestryDNA Settings</strong>.</li><li>Use the <strong>Invite</strong> function.</li><li>Send the invitation to <strong>Ron Yates</strong> at: <br><span style="font-family:monospace; font-weight:bold; background:white; padding:5px 10px; border:1px solid #aaa; border-radius:3px; display:inline-block; margin-top:5px; color:#c62828;">yatesvilleron@gmail.com</span></li><li>Assign the role <strong>Collaborator</strong>.</li></ol><p style="font-size:0.95em; color:#558b2f; margin-top:15px; background:white; padding:10px; border-radius:4px; border:1px solid #dcedc8;"><strong>Why Collaborator?</strong> The <em>Collaborator</em> role allows Ron to review shared matches to perform triangulation logic.</p></div></div></div>"""
SUBSCRIBE_CONTENT = r"""<div style="background:white; padding:40px; border-radius:8px; box-shadow:0 4px 15px rgba(0,0,0,0.1); max-width:800px; margin:20px auto; text-align:center; font-family:sans-serif;"><h2 style="color:#006064; margin-top:0;">Join the Yates Research Community</h2><div style="background:#e0f2f1; padding:25px; border-radius:8px; border:1px solid #b2dfdb; display:inline-block;"><h3 style="margin-top:0; color:#004d40;">📧 One-Click Subscribe</h3><p>Participation in the Yates DNA Study is completely free.</p><a href="mailto:yates-one-name-study+subscribe@groups.io?subject=Subscribe" style="display:inline-block; padding:15px 30px; background:#00838f; color:white; text-decoration:none; border-radius:5px; font-weight:bold; margin-top:10px;">Subscribe Now</a></div></div>"""
GLOSSARY_INLINE = r"""<div style="max-width:1100px; margin:20px auto; font-family:sans-serif; color:#333;"><h2 style="color:#006064;border-bottom:2px solid #004d40;padding-bottom:10px; margin-bottom:20px;">ONS Yates Study: Data Glossary</h2><details open style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">1. Core Concepts (Plain English)</span></summary><div style="padding:15px; line-height:1.6;"><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Ancestral Node:</strong><br>A "Node" is simply a specific ancestor on a family tree where different descendant lines intersect. If you think of a family tree like a roadmap, a node is the exact intersection where the paths of multiple DNA cousins meet. Proving a node confirms that the historical ancestor actually existed and passed down identifiable DNA.</li><li style="margin-bottom:15px;"><strong>Forensic Handshake:</strong><br>A Forensic Handshake happens when two or more DNA testers, who do not know each other and descend from different branches, all share DNA that points back to the exact same Ancestral Node. It acts as a genetic cross-reference that proves the paper trail is real. One match is a hint; a 'handshake' between multiple independent matches is proof.</li><li style="margin-bottom:15px;"><strong>Platinum Standard:</strong><br>Lineages that have achieved undeniable biological and documentary proof (30+ matches and 10+ unique sources).</li><li style="margin-bottom:15px;"><strong>Keystone Tester:</strong><br>A high-value study participant (with 15+ matches) whose DNA heavily anchors a specific branch of the family tree.</li></ul></div></details><details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">2. Identity Columns</span></summary><div style="padding:15px; line-height:1.6;"><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Tester-Participant-MASKED (The Trigger):</strong><br>The unique privacy code extracted from the user's NPFX tag.</li><li style="margin-bottom:15px;"><strong>Tester-Participant-Unmasked:</strong><br>The real name of the tester.</li></ul></div></details><details style="background:white;margin-bottom:15px;border:1px solid #ddd;border-radius:5px;overflow:hidden;"><summary style="background:#e0f2f1;padding:15px;cursor:pointer;font-weight:bold;color:#006064;list-style:none;"><span style="font-size:1.1em;">3. Genealogy Terms</span></summary><div style="padding:15px; line-height:1.6;"><ul style="list-style-type:none;padding-left:0;"><li style="margin-bottom:15px;"><strong>Spanish Naming System:</strong><br>A traditional Hispanic naming convention in which an individual bears one or more given names followed by two surnames: the first inherited from the father (paternal surname) and the second from the mother (maternal surname). This system is historically rooted in Spain and is especially useful in genealogy because it preserves both parental lineages and improves identification in historical records.</li><li style="margin-bottom:15px;"><strong>Née:</strong><br>A term meaning “born as,” used to indicate a woman’s maiden or birth surname before marriage. In genealogical and historical records, née identifies the surname a woman carried in her natal family line, preserving her connection to her parents and ancestry. For example, “Maria García, née López” shows that María’s birth surname was López, even though she later used García after marriage.</li></ul></div></details></div>"""
GEDMATCH_INLINE = r"""<style>.ged-table td { padding: 12px; border-bottom: 1px solid #ddd; font-size: 15px; } .ged-table a { color: #00838f; font-weight: bold; text-decoration: none; } .ged-table a:hover { text-decoration: underline; color: #006064; } .ged-table tbody tr:hover { background-color: #f9f9f9; }</style><div style="max-width:900px; margin:20px auto; font-family:sans-serif; color:#333;"><div style="background:white;padding:40px;border-radius:8px;border:1px solid #ddd;box-shadow:0 4px 10px rgba(0,0,0,0.05);"><h2 style="color:#01579b;border-bottom:2px solid #03a9f4;padding-bottom:10px;margin-top:0;">GEDmatch Hub: Known Yates Kits</h2><p style="font-size:1.1em; line-height:1.6; color:#555; margin-bottom:30px;">This registry contains the known GEDmatch kit numbers for study participants. Use these IDs to perform one-to-one segment comparisons.</p><table class="brief-table sortable ged-table" style="width:100%; border-collapse:collapse; border:1px solid #ddd;"><thead><tr style="background:#eceff1;"><th style="padding:12px; text-align:left; border-bottom:2px solid #000; color:#263238; width: 40%; cursor:pointer;">GEDmatch Kit #</th><th style="padding:12px; text-align:left; border-bottom:2px solid #000; color:#263238; cursor:pointer;">Participant Name</th></tr></thead><tbody><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">F201688</a></td><td>(Y-35 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M024169</a></td><td>(Y-44 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M040727</a></td><td>(Y-44 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M673507</a></td><td>(Y-44 kit)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">FR6532899</a></td><td>Adams, Sarah Sally</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M183543</a></td><td>Baig, Natalie</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A696433</a></td><td>Barnes-2</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A630666</a></td><td>Broms, Mary Beth</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">QW1447502</a></td><td>Crownover, Kathy Van Pelt</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T220912</a></td><td>Dallys E (Natalie Baig mother)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A493108</a></td><td>Girtain-Yates, Alma</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A849238</a></td><td>Girtain-Yates, Andy</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A004522</a></td><td>Girtain-Yates, Kathryn</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A214154</a></td><td>Godwin, Alta Barnes</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A291753</a></td><td>Laswell, Jack</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T422899</a></td><td>Leicher, John</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A323108</a></td><td>Lindsey</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">BW4660858</a></td><td>Little, Ilene</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M722226</a></td><td>McCollum, Michael</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">GS8478007</a></td><td>Moore, Wright</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T648223</a></td><td>Reddoch, James A. (FTDNA)</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A145010</a></td><td>Russett, Andrea Yates</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A507391</a></td><td>Sopp, Margaret</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A303537</a></td><td>Tabor, Sudie</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A909608</a></td><td>Varapodio, Joyce</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">KQ5739791</a></td><td>Wishard, Glenn</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">F316112</a></td><td>Yates, Abraham</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T810459</a></td><td>Yates, Arthur Lewis</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T749670</a></td><td>Yates, Benjamin</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T153410</a></td><td>Yates, Charlie Martin</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T977010</a></td><td>Yates, Howard Garrison</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A061248</a></td><td>Yates, James Robert</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">XE6552308</a></td><td>Yates, James Taos</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A131894</a></td><td>Yates, John</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A546217</a></td><td>Yates, John</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T608107</a></td><td>Yates, John F., Jr.</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A822241</a></td><td>Yates, John H.</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">F227484</a></td><td>Yates, John Henry</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">T159617</a></td><td>Yates, Patricia Lynn</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">DV4371781</a></td><td>Yates, Robert David</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A861466</a></td><td>Yates, Ron</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">DN2421283</a></td><td>Yates, Ron</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">MA8197080</a></td><td>Yates, Ronald Eugene</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M735337</a></td><td>Yates, Steph Solyon</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M407025</a></td><td>Yates, Timothy Brian</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">M441343</a></td><td>Yates, Timothy Joe</td></tr><tr><td><a class="kit-link" href="https://www.gedmatch.com/" target="_blank" rel="noopener">A139695</a></td><td>YatesJr, James Carey</td></tr></tbody></table></div></div>"""
THEORY_PAGE_CONTENT = r"""<div class="proof-card" style="max-width: 1000px; margin: 20px auto; text-align: left; font-family: 'Georgia', serif; line-height: 1.8; font-size: 16px; color: #333;">
    <h1 style="color:#006064; border-bottom:2px solid #004d40; padding-bottom:10px; margin-top:0; font-family: 'Segoe UI', sans-serif;">Yates DNA Study</h1>
    <h2 style="color:#00838f; font-size:1.3em; font-family: 'Segoe UI', sans-serif;">Theory of the Case — Origin and Methodological Development</h2>
    <p>The Yates DNA Study began as an effort to reconstruct ancestral lineages using autosomal DNA matches and traditional genealogical research. Over time, this work developed into a structured methodology for lineage reconstruction based on the convergence of independent genetic and documentary evidence.</p>
    <p>The central premise of the study is that ancestral correctness becomes increasingly probable as independent DNA-derived lines converge on the same upstream ancestral couples. This convergence can be observed both visually, through reversed pedigree analysis, and quantitatively through Collateral Saturation metrics.</p>
    <p>The methodology evolved in stages. Early work focused on assembling pedigree lines from DNA matches and identifying repeated ancestors. As the number of participants and matches increased, these observations developed into a formal framework capable of measuring lineage convergence and evaluating lineage reliability.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Methodological Scope</h2>
    <p>The study uses the collateral lines of individuals who share autosomal DNA and who report Yates ancestry or associated family connections. Each line begins with a confirmed DNA match and extends through that match’s documented pedigree.</p>
    <p>Collateral descendants represent siblings and cousins of direct ancestors. Because autosomal DNA is inherited across all ancestral lines, these collateral descendants collectively preserve detectable genetic signals from shared ancestors.</p>
    <p>Each pedigree line is treated as an independent reconstruction derived from a separate DNA match. Lines are extended only when the genealogical evidence remains plausible and consistent with known historical information.</p>
    <p>The method is designed to scale. Individual matches or small clusters may be inconclusive, but network-level evidence becomes increasingly reliable as replication across independent descendant lines increases.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Independent Derivation of Pedigree Lines</h2>
    <p>Each ancestral line used in the study originates from a confirmed DNA match. The pedigree associated with each match is developed independently using documentary sources, shared family trees, and established genealogical research.</p>
    <p>This independence is essential. The repeated appearance of the same ancestral couples across independently derived lines provides evidence of common descent that does not depend on any single pedigree.</p>
    <p>The study therefore avoids reliance on a single master tree. Instead, it builds a network of independently derived pedigrees whose points of agreement provide the strongest evidence of shared ancestry.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Reversed Pedigree Analysis</h2>
    <p>Pedigree lines are analyzed in reversed orientation, beginning with the earliest known ancestors and proceeding forward toward the present.</p>
    <p>When pedigrees are displayed in this form, ancestral couples repeated across multiple independent lines become immediately visible. Repeated blocks of identical names form recognizable patterns across the dataset.</p>
    <p>These repeated patterns represent the convergence of independent descendant lines on common ancestral nodes. The visual appearance of these repeated ancestral blocks was the earliest indication that large-scale autosomal lineage reconstruction might be possible.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Frequency as Evidence</h2>
    <p>Ancestors that appear frequently across independent pedigree lines are interpreted as strong candidates for shared ancestry.</p>
    <p>When fifteen or more independently derived DNA-match lines converge on the same ancestral couple, the probability that this convergence is accidental becomes extremely small. High-frequency ancestral couples therefore represent the most reliable genealogical anchors within the study.</p>
    <p>Lower-frequency ancestors are retained as hypotheses. As additional participants are added to the study, previously rare lines may later become important components of the larger network.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Development of Collateral Saturation</h2>
    <p>As the study expanded, repeated ancestral convergence began to appear as stable network structures rather than isolated observations. This led to the development of the Collateral Saturation method.</p>
    <p>Collateral Saturation evaluates evidence at the level of descendant networks rather than individual matches. Proof-grade conclusions emerge when independent descendant branches replicate the same ancestral relationships across multiple lines.</p>
    <p>The Collateral Saturation Score (CSS v2a) was developed to provide normalized comparison across participants using measurable properties including Proper Matches, Handshake Evidence, Dominance Ratio, Branch Replication, Unique Testers, and Node Saturation.</p>
    <p>In saturated lineages, properties such as Unique Testers and Node Saturation become shared characteristics of the ancestral node itself, while Proper Matches and Dominance Ratio distinguish individual descendants.</p>
    <p>Metric calculations are anchored to the genetically determined target node rather than pedigree depth. This ensures consistent measurement across participants regardless of documentary completeness.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Network-Level Proof</h2>
    <p>Testing demonstrated that Collateral Saturation behaves as a network-level threshold phenomenon. Individual participants may fail proof-grade criteria when evaluated in isolation, yet multiple independently related participants sharing the same ancestral node can combine into a stable proof-grade cluster.</p>
    <p>This "Virtual Group" behavior demonstrates that lineage proof emerges from replicated network evidence rather than from any single participant. The strength of the lineage derives from the network as a whole.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Extension to the ANCHOR Framework</h2>
    <p>Collateral Saturation demonstrated that autosomal DNA networks can identify ancestral lineages through replicated genetic evidence. However, genetic evidence alone does not establish the historical identity of those ancestors.</p>
    <p>The ANCHOR Framework was developed to integrate genetic network evidence with documentary pedigree evidence into a unified lineage reconstruction methodology.</p>
    <p>Within the ANCHOR Framework:</p>
    <ul style="margin-bottom: 20px;">
        <li><b>Collateral Saturation</b> provides the genetic evidence engine.</li>
        <li><b>CSS v2a</b> measures genetic network strength.</li>
        <li><b>DOCS</b> measures documentary pedigree robustness using GEDCOM-derived evidence.</li>
        <li><b>ANCHOR</b> measures the combined strength of genetic and documentary evidence.</li>
    </ul>
    <p>DOCS measures pedigree robustness rather than lineage correctness. Lineage conclusions depend primarily on replicated genetic evidence, while documentary evidence provides historical structure and confirmation.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Practical Conclusion</h2>
    <p>When reversed pedigrees and quantitative scoring highlight ancestors repeated across many independent DNA-match lines, those ancestors become leading candidates for direct lineage placement.</p>
    <p>Lineage reconstruction becomes most reliable when independent genetic networks and documentary pedigrees converge on the same ancestral couples.</p>
    <p>The Yates DNA Study therefore demonstrates a scalable method for lineage reconstruction based on replication, convergence, and network stability.</p>

    <h2 style="color:#00838f; border-bottom:1px solid #eee; padding-bottom:5px; margin-top:40px; font-family: 'Segoe UI', sans-serif;">Alignment with Genealogical Standards</h2>
    <p>This project integrates autosomal DNA evidence with documentary genealogy in accordance with accepted genealogical standards. DNA evidence is evaluated alongside traditional records, and conclusions are tested for sufficiency, replication, and stability.</p>
    <ul>
        <li>Planned comparison of DNA results across independent descendant lines</li>
        <li>Integration of autosomal DNA with documentary evidence</li>
        <li>Cumulative evaluation based on collateral match density and replication</li>
    </ul>
</div>"""
NETWORK_PAGE_CONTENT = r"""<style>
#network-container { width: 100%; height: 750px; border: 2px solid #ccc; border-radius: 8px; background: #fff; box-shadow: inset 0 2px 10px rgba(0,0,0,0.05); }
.network-ui { text-align: center; margin-bottom: 25px; background: #e0f2f1; padding: 20px; border-radius: 8px; border: 1px solid #b2dfdb; }
.network-ui select { padding: 12px; font-size: 16px; border: 2px solid #006064; border-radius: 6px; min-width: 400px; cursor: pointer; }
.vis-tooltip { position: absolute; padding: 10px; background: rgba(0, 0, 0, 0.8) !important; color: white !important; font-family: sans-serif; font-size: 14px; border-radius: 4px; pointer-events: none; z-index: 1000; box-shadow: 0 4px 6px rgba(0,0,0,0.3); }
</style>
<script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>

<div class="proof-card" style="max-width: 1400px;">
    <h2 style="color:#006064; text-align:center; margin-top:0; border-bottom:2px solid #004d40; padding-bottom:10px;">DNA Network Visualizer</h2>
    <p style="text-align:center; font-size:16px; color:#555; margin-bottom:30px;">This interactive force-directed graph visualizes the genetic clusters in the Yates Study. Select an ancestral line below to instantly map its descendant network.</p>

    <div class="network-ui no-print">
        <select id="networkSelect" onchange="drawNetwork()">
            <option value="">-- Select an Ancestral Cluster --</option>
        </select>
    </div>

    <div id="network-container"></div>
</div>

<script>
__JS_GLOBALS__;

let network = null;

document.addEventListener('DOMContentLoaded', function() {
    let sel = document.getElementById('networkSelect');
    const ancKeys = Object.keys(DATA.ancestors).sort((a,b) => DATA.ancestors[a].name.localeCompare(DATA.ancestors[b].name));

    ancKeys.forEach(k => {
        if (DATA.ancestors[k].matches >= 2) {
            let opt = document.createElement('option');
            opt.value = k;
            opt.innerHTML = `${DATA.ancestors[k].name} (${DATA.ancestors[k].matches} proper matches)`;
            sel.appendChild(opt);
        }
    });
});

function drawNetwork() {
    let selectedAncKey = document.getElementById('networkSelect').value;
    if(!selectedAncKey) {
        if(network) { network.destroy(); network = null; }
        return;
    }

    let targetAncName = DATA.ancestors[selectedAncKey].name;
    let matches = DB.filter(m => m.ancestor === targetAncName);

    let nodes = new vis.DataSet();
    let edges = new vis.DataSet();

    nodes.add({
        id: targetAncName,
        label: targetAncName.split('&').join('\n&'),
        shape: 'box',
        color: { background: '#006064', border: '#004d40' },
        font: { color: 'white', size: 20, face: 'Georgia', bold: true },
        margin: 15,
        shadow: true,
        title: `<b>${targetAncName}</b><br>Total Matches: ${matches.length}<br>Integrity: ${DATA.ancestors[selectedAncKey].integrity}%`
    });

    let addedTesters = new Set();
    matches.forEach(m => {
        if(!addedTesters.has(m.participant)) {
            let tData = DATA.participants[m.participant];
            let badgeColor = '#ab47bc';
            let statText = "UNSCORED";

            if (tData) {
                if (tData.css_status === 'PASS') { badgeColor = '#2e7d32'; statText = "PASS"; }
                else if (tData.css_status === 'PARTIAL') { badgeColor = '#ef6c00'; statText = "PARTIAL"; }
                else if (tData.css_status === 'FAIL') { badgeColor = '#c62828'; statText = "FAIL"; }
            }

            nodes.add({
                id: m.participant,
                label: m.participant,
                shape: 'dot',
                size: 15,
                color: { background: badgeColor, border: 'white', borderWidth: 2 },
                font: { color: '#333', size: 14, face: 'Arial' },
                shadow: true,
                title: `<b>${m.participant}</b><br>Shared cM: ${m.cm}<br>CSS Status: ${statText}`
            });

            edges.add({
                from: m.participant,
                to: targetAncName,
                label: m.cm + ' cM',
                font: { align: 'middle', size: 12, color: '#777', background: 'white' },
                color: { color: '#ccc', highlight: badgeColor },
                width: 2,
                arrows: 'to'
            });
            addedTesters.add(m.participant);
        }
    });

    let container = document.getElementById('network-container');
    let data = { nodes: nodes, edges: edges };
    let options = {
        physics: {
            forceAtlas2Based: { gravitationalConstant: -100, centralGravity: 0.01, springLength: 200, springConstant: 0.08 },
            maxVelocity: 50, solver: 'forceAtlas2Based', timestep: 0.35, stabilization: { iterations: 150 }
        },
        interaction: { hover: true, tooltipDelay: 200 }
    };
    if(network) { network.destroy(); }
    network = new vis.Network(container, data, options);
}
</script>"""

print("✅ Cell 4 (CSS Variables & Scaled Matrices) Loaded Successfully.")

      [CELL 4] TEMPLATE LIBRARY LOADING (CSS Fix + Scaled Matrices)...
✅ Cell 4 (CSS Variables & Scaled Matrices) Loaded Successfully.


In [29]:
# @title [CELL 5] Core Publisher & FTP Uploader (Patch 64: HTML Ext Override)
def run_publisher():
    print("="*60)
    print("      [CELL 5] PUBLISHER STARTING (HTML Fix)...")
    print("="*60)

    import os, re, pytz, json, csv, math
    import pandas as pd
    from datetime import datetime
    from google.colab import userdata
    from ftplib import FTP_TLS

    try:
        HOST = os.environ.get("FTP_HOST") or userdata.get("FTP_HOST")
        USER = os.environ.get("FTP_USER") or userdata.get("FTP_USER")
        PASS = os.environ.get("FTP_PASS") or userdata.get("FTP_PASS")
    except Exception as e:
        return print(f"❌ Credential Error: {e}")

    CSV_DB = "engine_database.csv"

    if not os.path.exists(CSV_DB): return print("❌ ERROR: engine_database.csv not found.")

    df = pd.read_csv(CSV_DB, encoding="iso-8859-15")
    df.fillna('', inplace=True)
    df.replace('nan', '', inplace=True)

    # 🌟 1. FAST GEDCOM PARSER 🌟
    persons_data = {}
    ged_files = [f for f in os.listdir('.') if f.lower().endswith('.ged') and "_processed" not in f.lower()]
    if ged_files:
        ged_file = sorted(ged_files, key=lambda x: os.path.getmtime(x), reverse=True)[0]
        print(f"    [+] Parsing GEDCOM Vitals: {ged_file}")
        with open(ged_file, 'r', encoding='utf-8', errors='replace') as f:
            current_id = None
            mode = None
            for line in f:
                line = line.strip()
                if line.startswith("0 @I"):
                    current_id = line.split(" ")[1].replace("@", "")
                    persons_data[current_id] = {'bdate': '', 'bplace': '', 'ddate': '', 'dplace': '', 'sources_count': 0, 'citations_count': 0}
                    mode = None
                elif current_id:
                    if line.startswith("1 BIRT"): mode = "BIRT"
                    elif line.startswith("1 DEAT"): mode = "DEAT"
                    elif line.startswith("1 SOUR"):
                        persons_data[current_id]['sources_count'] += 1
                        mode = None
                    elif line.startswith("2 SOUR"): persons_data[current_id]['citations_count'] += 1
                    elif line.startswith("2 DATE"):
                        if mode == "BIRT": persons_data[current_id]['bdate'] = line[7:].strip()
                        elif mode == "DEAT": persons_data[current_id]['ddate'] = line[7:].strip()
                    elif line.startswith("2 PLAC"):
                        if mode == "BIRT": persons_data[current_id]['bplace'] = line[7:].strip()
                        elif mode == "DEAT": persons_data[current_id]['dplace'] = line[7:].strip()
                    elif line.startswith("1 "): mode = None

    df_testers = df[['Tester_Code', 'Tester_Name', 'Tester_ID', 'Tester_Sort_Key']].drop_duplicates('Tester_Code').rename(columns={"Tester_Code": "Kit_Code"})

    df['Kit_Name'] = df['Tester_Display']
    df.rename(columns={"Authority_Directory_Label": "Dir_Label", "Authority_FirstAncestor_alpha": "Alpha_Key", "Tester_Code": "Kit_Code", "Match_Lineage": "Lineage", "Match_Path_IDs": "s_ids"}, inplace=True)

    def normalize_id(val): return f"I{str(val).replace('@', '').strip()}" if str(val).replace('@', '').strip().isdigit() else str(val).replace('@', '').strip()
    def clean_num(s): return re.sub(r'[^0-9]', '', str(s))

    df['search_ids'] = df['s_ids']
    df['search_names'] = df['Lineage'].astype(str).str.replace(' -> ', '|')
    df['t_names'] = df['Tester_Lineage'].astype(str).str.replace(' -> ', '|')
    df['t_ids'] = df['Tester_Path_IDs'].astype(str).str.replace(',', '|')

    # 🌟 2. PYTHON MATH ENGINE (ANCHOR) 🌟
    print("    [+] Calculating ANCHOR Matrices Server-Side...")
    def year_from_date_str(s):
        if not s: return None
        m = re.search(r'(\d{4})', str(s))
        return int(m.group(1)) if m else None

    def date_quality_score(s):
        if not s: return 0
        t = str(s).upper().strip()
        if re.search(r'\d{4}-\d{2}-\d{2}', t): return 1.0
        if re.search(r'\b(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)\b', t) and re.search(r'\d{4}', t): return 0.8
        if re.search(r'\b\d{4}\b', t):
            if any(x in t for x in ["ABT", "AFT", "BEF", "BET", "CAL"]): return 0.45
            return 0.6
        return 0.3

    def place_quality_score(p):
        if not p: return 0
        parts = [x.strip() for x in str(p).split(",") if x.strip()]
        if len(parts) >= 3: return 1.0
        if len(parts) == 2: return 0.7
        if len(parts) == 1: return 0.4
        return 0

    def token_set(place_str):
        if not place_str: return set()
        return set([x.strip() for x in re.split(r'[, ]+', str(place_str).lower()) if x.strip()])

    def norm_log(val, cap):
        val = max(0, val or 0)
        return min(1.0, math.log(1 + val) / math.log(1 + cap))

    def weighted_mean(pairs):
        num = sum(v * w for v, w in pairs)
        den = sum(w for v, w in pairs)
        return (num / den) if den > 0 else 0

    def get_docs(spine_ids, target_id, pdata):
        total = len(spine_ids)
        if total == 0: return 0,0,0,0,0,0,0,0,0,0,0,0,1.0
        ax = 1 if target_id in spine_ids else 0
        gd = total
        gd_n = norm_log(gd, 14)
        bc_arr, dc_arr, dq_arr, pq_arr = [], [], [], []
        cited_count, tot_src, tot_cite, tp_checks, tp_viol, gc_checks, gc_good = 0, 0, 0, 0, 0, 0, 0
        id_collision = len(set(spine_ids)) != len(spine_ids)
        prev_tokens, prev_by = None, None
        covered = 0
        for pid in spine_ids:
            person = pdata.get(pid, {})
            bdate, bplace = person.get('bdate'), person.get('bplace')
            ddate, dplace = person.get('ddate'), person.get('dplace')
            sc, cc = person.get('sources_count', 0), person.get('citations_count', 0)
            b_has, d_has = bool(bdate) + bool(bplace), bool(ddate) + bool(dplace)
            bc_arr.append(1.0 if b_has == 2 else (0.6 if b_has == 1 else 0))
            dc_arr.append(1.0 if d_has == 2 else (0.6 if d_has == 1 else 0))
            dq_arr.append(max(date_quality_score(bdate), date_quality_score(ddate)))
            pq_arr.append(max(place_quality_score(bplace), place_quality_score(dplace)))
            tot_src += sc; tot_cite += cc
            if sc + cc > 0: cited_count += 1
            if bdate or bplace or ddate or dplace or sc > 0 or cc > 0: covered += 1
            by, dy = year_from_date_str(bdate), year_from_date_str(ddate)
            if by and dy:
                tp_checks += 1
                if dy <= by: tp_viol += 1
                if (dy - by) > 110: tp_viol += 1
            if prev_by and by:
                tp_checks += 1
                if not (12 <= abs(prev_by - by) <= 60): tp_viol += 1
            if by: prev_by = by
            toks = token_set(bplace or dplace or "")
            if prev_tokens is not None:
                gc_checks += 1
                if not prev_tokens.isdisjoint(toks): gc_good += 1
            prev_tokens = toks

        bc = sum(bc_arr) / total; dc = sum(dc_arr) / total; dq = sum(dq_arr) / total; pq = sum(pq_arr) / total
        cc_val = cited_count / total; sd = norm_log(tot_src + tot_cite, 50)
        tp = max(0, 1.0 - (tp_viol/tp_checks)) if tp_checks > 0 else 0.75
        gc = (gc_good/gc_checks) if gc_checks > 0 else 0.6
        nc = covered / total; idm = 0.85 if id_collision else 1.0

        pairs = [(ax, 2.0), (gd_n, 1.0), (nc, 1.0), (bc, 1.0), (dc, 1.0), (dq, 1.0), (pq, 1.0), (gc, 1.0), (cc_val, 2.0), (sd, 1.5), (tp, 1.5)]
        docs_base = 100 * weighted_mean(pairs) * idm
        docs = docs_base if ax == 1 else min(docs_base, 35)
        return docs, ax, gd, nc, bc, dc, dq, pq, gc, cc_val, sd, tp, idm

    id_to_kits = {}
    for _, r in df.iterrows():
        if r['s_ids']:
            ids = [clean_num(x) for x in str(r['s_ids']).split(',') if x]
            for i in ids:
                if i not in id_to_kits: id_to_kits[i] = set()
                id_to_kits[i].add(r['Kit_Name'])

    participant_scores = []
    for p_name, grp in df.groupby('Kit_Name'):
        my_matches = grp[grp['Dir_Label'] != 'No Matches']
        pm = int(len(my_matches))
        if pm == 0:
            participant_scores.append({
                'pName': str(p_name), 'targetAnc': "None", 'PM': 0, 'HC_T': 0, 'HC_2': 0,
                'DR': 0.0, 'TB': 0, 'BR': 0, 'NS': 0, 'ST_str': "UNSCORED", 'cssFinal': 0.0,
                'DOCS': 0.0, 'ANCHOR': 0.0, 'AX': 0, 'GD': 0, 'NC': 0.0, 'BC': 0.0, 'DC': 0.0,
                'DQ': 0.0, 'PQ': 0.0, 'GC': 0.0, 'CC': 0.0, 'SD': 0.0, 'TP': 0.0, 'IDm': 1.0, 'isGroup': False
            })
            continue

        dir_counts = my_matches['Dir_Label'].value_counts()
        target_anc = str(dir_counts.index[0]) if len(dir_counts) > 0 else "Unknown"
        hc_t = int(dir_counts.iloc[0]) if len(dir_counts) > 0 else pm
        hc_2 = int(dir_counts.iloc[1]) if len(dir_counts) > 1 else 0

        highest_heat = 0
        target_id = None
        for _, r in my_matches.iterrows():
            if r['s_ids']:
                for i in [clean_num(x) for x in str(r['s_ids']).split(',') if x]:
                    kit_count = len(id_to_kits.get(i, set()))
                    if kit_count > highest_heat:
                        highest_heat = kit_count
                        target_id = i

        ns = int(highest_heat)
        spine_ids = []
        tb = 0
        br = 0
        if target_id:
            tb = int(len(id_to_kits.get(target_id, set())))
            collaterals = df[df['s_ids'].apply(lambda x: target_id in [clean_num(y) for y in str(x).split(',') if y])]
            branches = set()
            for _, r in collaterals.iterrows():
                ids = [clean_num(x) for x in str(r['s_ids']).split(',') if x]
                names = str(r['search_names']).split('|')
                if target_id in ids:
                    idx = ids.index(target_id)
                    if idx + 1 < len(names): branches.add(names[idx+1].replace('findme', '?').replace('FINDME', '?').split(' (')[0].strip())
                    else: branches.add("Direct Descendant")
            br = int(len(branches))
            best_match = my_matches[my_matches['s_ids'].apply(lambda x: target_id in [clean_num(y) for y in str(x).split(',') if y])].iloc[0]
            spine_ids = [clean_num(x) for x in str(best_match['s_ids']).split(',') if x]

        dr = float(hc_t / (hc_2 if hc_2 > 0 else 1))
        pm_n, hc_n, dr_n, tb_n, ns_n = norm_log(pm, 150), norm_log(hc_t, 100), norm_log(dr, 10), norm_log(tb, 40), norm_log(ns, 150)
        br_n = 1.0 if br >= 6 else (0.85 if br == 5 else (0.70 if br == 4 else (0.50 if br == 3 else (0.25 if br == 2 else 0))))
        st_str, st_val = "FAIL", 0.60
        if pm >= 15:
            if br >= 3 and dr >= 1.5: st_str, st_val = "PASS", 1.0
            elif br >= 2: st_str, st_val = "PARTIAL", 0.85
        weighted_sum = (pm_n * 1.0) + (hc_n * 1.0) + (dr_n * 1.5) + (tb_n * 1.0) + (br_n * 2.0) + (ns_n * 1.0)
        css_final = float(100 * (weighted_sum / 7.5) * st_val)

        docs, ax, gd, nc, bc, dc, dq, pq, gc, cc, sd, tp, idm = get_docs(spine_ids, target_id, persons_data)
        anchor = float(min(100, (0.65 * css_final) + (0.35 * docs) + (10 * (min(css_final, docs) / 100))))

        participant_scores.append({
            'pName': str(p_name), 'targetAnc': str(target_anc), 'PM': int(pm), 'HC_T': int(hc_t), 'HC_2': int(hc_2),
            'DR': float(dr), 'TB': int(tb), 'BR': int(br), 'NS': int(ns), 'ST_str': str(st_str), 'cssFinal': float(css_final),
            'DOCS': float(docs), 'ANCHOR': float(anchor), 'AX': int(ax), 'GD': int(gd), 'NC': float(nc), 'BC': float(bc), 'DC': float(dc),
            'DQ': float(dq), 'PQ': float(pq), 'GC': float(gc), 'CC': float(cc), 'SD': float(sd), 'TP': float(tp), 'IDm': float(idm), 'isGroup': False
        })

    master_rows = sorted([p for p in participant_scores if p['PM'] > 0], key=lambda x: x['ANCHOR'], reverse=True)
    tab_master = '<table class="anchor-table sortable"><thead><tr style="background:#fff3e0;"><th style="text-align:left; color:#d84315;">Participant Kit</th><th style="color:#d84315;">Primary Ancestor</th><th style="color:#d84315;">DNA Score (CSS)</th><th style="color:#d84315;">Paper Score (DOCS)</th><th style="background:#d84315; color:white;">ANCHOR Score</th></tr></thead><tbody>'
    for r in master_rows:
        tab_master += f'<tr><td style="text-align:left;" data-sort="{r["pName"]}"><strong>{r["pName"]}</strong></td><td>{r["targetAnc"]}</td><td style="color:#4a148c; font-weight:bold;">{r["cssFinal"]:.1f}</td><td style="color:#5d4037; font-weight:bold;">{r["DOCS"]:.1f}</td><td style="background:#fbe9e7; font-weight:bold; color:#d84315; font-size:1.2em;" data-sort="{r["ANCHOR"]}">{r["ANCHOR"]:.2f}</td></tr>'
    tab_master += '</tbody></table>'

    css_rows = sorted([p for p in participant_scores if p['PM'] > 0], key=lambda x: x['cssFinal'], reverse=True)
    tab_css = '<table class="anchor-table sortable"><thead><tr style="background:#f3e5f5;"><th style="text-align:left; color:#4a148c;">Participant Kit</th><th style="color:#4a148c;">PM</th><th style="color:#4a148c;">HC-T</th><th style="color:#4a148c;">DR</th><th style="color:#4a148c;">BR</th><th style="color:#4a148c;">NS</th><th style="color:#4a148c;" title="PASS requires PM &ge; 15, BR &ge; 3, and DR &ge; 1.5">Status &#9432;</th><th style="background:#4a148c; color:white;">CSS Score</th><th style="color:#4a148c;">Diagnostics</th></tr></thead><tbody>'
    for r in css_rows:
        status_color = "green" if r["ST_str"] == "PASS" else ("orange" if r["ST_str"] == "PARTIAL" else "red")
        status_badge = f'<span style="color:{status_color}; font-weight:bold;">{r["ST_str"]}</span>'
        pname_safe = r["pName"].replace("'", "\\'")
        diag_btn = f'<button class="diag-btn-css" onclick="runCssDiag(\'{pname_safe}\', {r["PM"]}, {r["BR"]}, {r["DR"]})">🩺 Diagnose</button>'
        tab_css += f'<tr><td style="text-align:left;" data-sort="{r["pName"]}"><strong>{r["pName"]}</strong></td><td>{r["PM"]}</td><td>{r["HC_T"]}</td><td>{r["DR"]:.1f}</td><td>{r["BR"]}</td><td>{r["NS"]}</td><td data-sort="{r["ST_str"]}">{status_badge}</td><td style="background:#f3e5f5; font-weight:bold; color:#4a148c; font-size:1.1em;" data-sort="{r["cssFinal"]}">{r["cssFinal"]:.2f}</td><td>{diag_btn}</td></tr>'
    tab_css += '</tbody></table>'

    docs_rows = sorted([p for p in participant_scores if p['PM'] > 0], key=lambda x: x['DOCS'], reverse=True)
    tab_docs = '<table class="anchor-table sortable"><thead><tr style="background:#efebe9;"><th style="text-align:left; color:#3e2723;">Participant Kit</th><th style="color:#3e2723;" title="Generations">Gen</th><th style="color:#3e2723;" title="Vital Completeness">Vitals</th><th style="color:#3e2723;" title="Citation Coverage">Cites</th><th style="color:#3e2723;" title="Temporal Plausibility">Plsblty</th><th style="background:#5d4037; color:white;">DOCS Score</th><th style="color:#3e2723;">Diagnostics</th></tr></thead><tbody>'
    for r in docs_rows:
        vts = f"{((r['BC'] + r['DC'])/2 * 100):.0f}%"
        cts = f"{(r['CC'] * 100):.0f}%"
        tps = f"{(r['TP'] * 100):.0f}%"
        pname_safe = r["pName"].replace("'", "\\'")
        tab_docs += f'<tr><td style="text-align:left;" data-sort="{r["pName"]}"><strong>{r["pName"]}</strong></td><td>{r["GD"]}</td><td>{vts}</td><td>{cts}</td><td>{tps}</td><td style="background:#efebe9; font-weight:bold; color:#5d4037; font-size:1.1em;" data-sort="{r["DOCS"]}">{r["DOCS"]:.1f}</td><td><button class="diag-btn" onclick="runDiag(\'{pname_safe}\', {r["AX"]}, {r["CC"]}, {r["BC"]}, {r["DC"]}, {r["TP"]})">🩺 Diagnose</button></td></tr>'
    tab_docs += '</tbody></table>'

    precomputed_json = json.dumps([p for p in participant_scores if p['PM'] > 0])

    # 🌟 3. DATA DICTIONARIES (JSON BUILDERS) 🌟
    css_lookup = {ps['pName']: ps['ST_str'] for ps in participant_scores}

    anc_data = {}; part_data = {}

    for lbl, grp in df.groupby('Dir_Label'):
        if str(lbl).strip() == 'No Matches' or not str(lbl).strip():
            continue
        unique_t = len(grp['Kit_Name'].unique())
        integ = min(100, (len(grp)*2) + (unique_t*10))
        badge = "Platinum" if len(grp)>=30 else "Gold" if len(grp)>=15 else "Silver" if len(grp)>=5 else ("Bronze" if len(grp)>=2 else "Singleton")
        verdict = "Verified." if len(grp)>=2 else "Pending Corroboration."

        anc_data[str(lbl)] = {
            "name": str(lbl),
            "matches": int(len(grp)),
            "cm": int(grp['cM'].sum()),
            "badge": badge,
            "list_data": {str(k): int(v) for k, v in grp['Kit_Name'].value_counts().head(3).items()},
            "verdict": verdict,
            "integrity": int(integ),
            "testers": int(unique_t)
        }

    for kname, grp in df.groupby('Kit_Name'):
        valid_grp = grp[grp['Dir_Label'] != 'No Matches']
        dir_lbl = valid_grp.iloc[0]['Dir_Label'] if not valid_grp.empty else "No Matches"
        same_dir = df[df['Dir_Label'] == dir_lbl] if dir_lbl != "No Matches" else pd.DataFrame()

        sk = str(grp.iloc[0].get('Tester_Sort_Key', '')).strip()
        if not sk: sk = str(kname).lower()

        css_stat = css_lookup.get(str(kname), "UNSCORED")

        part_data[str(kname)] = {
            "name": str(kname),
            "sort_key": sk,
            "matches": int(len(valid_grp)),
            "cm": int(valid_grp['cM'].sum()) if not valid_grp.empty else 0,
            "badge": "Keystone Tester" if len(valid_grp)>=15 else "Study Participant",
            "list_data": {str(k): int(v) for k, v in valid_grp['Dir_Label'].value_counts().head(3).items()} if not valid_grp.empty else {},
            "verdict": f"Verified matches across {len(valid_grp['Dir_Label'].unique())} ancestral lines." if not valid_grp.empty else "No verified matches.",
            "integrity": int(min(100, len(same_dir) * 5)),
            "testers": 1,
            "kit_code": str(grp.iloc[0]['Kit_Code']),
            "css_status": css_stat
        }

    smart_json = json.dumps({"ancestors": anc_data, "participants": part_data, "persons": persons_data})
    db_json = df[['Dir_Label', 'Kit_Name', 'cM', 'Match_ID', 'Lineage', 'search_ids', 'search_names', 't_names', 't_ids', 'Tester_ID', 'Kit_Code']].rename(columns={'Dir_Label':'ancestor', 'Kit_Name':'participant', 'cM':'cm', 'Match_ID':'id', 'Lineage':'lineage', 'Tester_ID':'tester_id', 'Kit_Code':'kit_code'}).to_json(orient='records')
    JS_GLOBALS = f"const PRECOMPUTED_JSON={precomputed_json}; const DATA={smart_json}; const DB={db_json};"

    # 🌟 4. ADMIN HUB CONTENT GENERATION 🌟
    valid_df = df[df['Dir_Label'] != 'No Matches']
    match_counts = valid_df.groupby('Kit_Code').size().reset_index(name='Match_Count')

    part_stats = pd.merge(df_testers, match_counts, on='Kit_Code', how='left')
    part_stats['Match_Count'] = part_stats['Match_Count'].fillna(0).astype(int)

    tester_sort_map = df.drop_duplicates('Kit_Code').set_index('Kit_Code')['Tester_Sort_Key'].to_dict()
    part_stats['Sort_Key'] = part_stats['Kit_Code'].map(tester_sort_map).fillna('zzz')

    def make_admin_row(r):
        tid = str(r['Tester_ID']).strip()
        tname, kcode, mc_val = str(r["Tester_Name"]), str(r["Kit_Code"]), r['Match_Count']
        t_link = f'<a href="https://yates.one-name.net/tng/getperson.php?personID=I{tid}&tree=tree1" target="_blank" style="color:#00838f;text-decoration:underline;font-weight:bold;">{tname}</a>' if tid and tid != 'nan' else f'<b style="color:#333;">{tname}</b>'
        tid_display = f" <span style='color:#777;font-size:0.85em;'>[I{tid}]</span>" if tid and tid != 'nan' else ""
        mc_str = f"<span style='color:#d32f2f;font-weight:bold;'>0</span>" if mc_val == 0 else str(mc_val)
        return f"<tr><td data-sort='{r['Sort_Key']}' style='text-align:center; padding:10px;'>{t_link}{tid_display}<br><span style='color:#666;font-size:0.85em;'>Kit: {kcode}</span></td><td style='text-align:center;font-size:1.1em;vertical-align:middle; padding:10px;'>{mc_str}</td></tr>"

    admin_rows_az = [make_admin_row(r) for _, r in part_stats.sort_values(['Sort_Key', 'Tester_Name'], ascending=[True, True]).iterrows()]
    admin_rows_count = [make_admin_row(r) for _, r in part_stats.sort_values(['Match_Count', 'Sort_Key'], ascending=[True, True]).iterrows()]

    admin_content = f"""<div class="dashboard-grid"><a href="ons_yates_dna_register.shtml" class="dash-card"><span class="dash-icon">📋</span><span class="dash-title">DNA Register</span></a><a href="dna_network.html" class="dash-card"><span class="dash-icon">🕸️</span><span class="dash-title">DNA Network</span></a><a href="proof_consolidator.html" class="dash-card" style="border-color:#4a148c; background:#f3e5f5;"><span class="dash-icon">🎓</span><span class="dash-title" style="color:#4a148c;">Report</span></a><a href="proof_engine.html" class="dash-card"><span class="dash-icon">🔬</span><span class="dash-title">Proof Engine</span></a><a href="dna_dossier.html" class="dash-card"><span class="dash-icon">📁</span><span class="dash-title">Forensic Dossier</span></a><a href="gedmatch_integration.shtml" class="dash-card" style="border-color:#0277bd; background:#e1f5fe;"><span class="dash-icon">🧬</span><span class="dash-title" style="color:#01579b;">GEDmatch Hub</span></a><a href="anchor_frame.htm" class="dash-card" style="border-color:#d84315; background:#fbe9e7;"><span class="dash-icon">⚓</span><span class="dash-title" style="color:#bf360c;">Anchor</span></a><a href="engine_database.csv" download class="dash-card" style="border-color:#b71c1c; background:#ffebee;"><span class="dash-icon">💾</span><span class="dash-title" style="color:#c62828;">Download CSV Database</span></a></div><div class="audit-table-wrapper" style="text-align:center;"><h2 style="color:#004d40;border-bottom:2px solid #004d40;padding-bottom:10px;margin-top:0;text-align:left;">Report - {len(part_stats)} Testers</h2><div style="text-align:center; margin-bottom:20px; font-family:sans-serif;"><button onclick="document.getElementById('table-az').style.display='block'; document.getElementById('table-count').style.display='none'; this.style.background='#006064'; this.style.color='white'; document.getElementById('btn-count').style.background='#e0f7fa'; document.getElementById('btn-count').style.color='#006064';" id="btn-az" style="padding:10px 20px; cursor:pointer; font-weight:bold; background:#006064; color:white; border:1px solid #00acc1; border-radius:4px;">Sort A-Z (Default)</button><button onclick="document.getElementById('table-count').style.display='block'; document.getElementById('table-az').style.display='none'; this.style.background='#006064'; this.style.color='white'; document.getElementById('btn-az').style.background='#e0f7fa'; document.getElementById('btn-az').style.color='#006064';" id="btn-count" style="padding:10px 20px; cursor:pointer; font-weight:bold; background:#e0f7fa; color:#006064; border:1px solid #00acc1; border-radius:4px; margin-left:10px;">Sort by Match Count (Low to High)</button></div><div id="table-az" style="max-height:600px;overflow-y:auto;border:1px solid #ddd;background:#fafafa; display:block; max-width:650px; margin:0 auto;"><table style="width:100%;border-collapse:collapse;"><thead><tr><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:60%;">Participant Kit</th><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:40%;">Matches</th></tr></thead><tbody>{''.join(admin_rows_az)}</tbody></table></div><div id="table-count" style="max-height:600px;overflow-y:auto;border:1px solid #ddd;background:#fafafa; display:none; max-width:650px; margin:0 auto;"><table style="width:100%;border-collapse:collapse;"><thead><tr><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:60%;">Participant Kit</th><th style="background:#004d40;color:white;padding:12px;text-align:center;position:sticky;top:0;width:40%;">Matches</th></tr></thead><tbody>{''.join(admin_rows_count)}</tbody></table></div></div>"""

    # 🌟 5. REGISTRY & TREE DATA 🌟
    df_valid = df[df['Dir_Label'] != 'No Matches'].copy()
    df_valid['sort_key'] = df_valid['Tester_Sort_Key']
    mc = df_valid['Dir_Label'].value_counts()

    def format_reg(r):
        m_id = str(r.get("Match_ID", ""))
        m_name = str(r.get("Match_Name", ""))
        d_label = str(r.get("Dir_Label", "")).split('(')[0].strip()
        lin_len = len(str(r.get("Lineage", "")).split("->"))
        kit_name = str(r["Kit_Name"])
        cm_val = str(r["cM"])
        text = f"<b>{kit_name}</b> is a {cm_val} cM match to <a href='https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(m_id)}&tree=tree1&parentset=0&display=vertical&generations=15' target='_blank'><b>{m_name}</b></a> via {d_label} back {lin_len} generations."
        if mc.get(r['Dir_Label'], 0) == 1: return f"<div style='background-color: #fffde7; padding: 12px; margin: -12px; border-left: 5px solid #fbc02d;'>{text} <span style='float:right; font-size:0.85em; color:#e65100; font-weight:bold; background:#fff8e1; padding:3px 8px; border-radius:4px; border:1px solid #ffe082;'>🌟 Singleton Line</span></div>"
        return f"<div style='padding: 12px; margin: -12px;'>{text}</div>"

    def format_tree(r):
        m_id = str(r.get("Match_ID", ""))
        m_name = str(r.get("Match_Name", ""))
        lin_str = str(r.get("Lineage", ""))
        kit_name = str(r["Kit_Name"])
        if m_name in lin_str: linked_lin = lin_str.replace(m_name, f'<a href="https://yates.one-name.net/tng/verticalchart.php?personID={normalize_id(m_id)}&tree=tree1&parentset=0&display=vertical&generations=15" target="_blank" style="color:#006064;text-decoration:none;font-weight:bold;">{m_name}</a>')
        else: linked_lin = lin_str
        text = f"<b style='color:#4a148c;'>{kit_name}</b>: {linked_lin}"
        if mc.get(r['Dir_Label'], 0) == 1: return f"<div style='background-color: #fffde7; padding: 12px; margin: -12px; border-left: 5px solid #fbc02d;'>{text} <span style='float:right; font-size:0.85em; color:#e65100; font-weight:bold; background:#fff8e1; padding:3px 8px; border-radius:4px; border:1px solid #ffe082;'>🌟 Singleton Line</span></div>"
        return f"<div style='padding: 12px; margin: -12px;'>{text}</div>"

    df_valid['Reg_Narrative'] = df_valid.apply(format_reg, axis=1)
    df_valid['Tree_Narrative'] = df_valid.apply(format_tree, axis=1)

    df_reg_za = df_valid.sort_values(by=['Dir_Label', 'sort_key'], ascending=[False, True]).copy()
    df_reg_za.rename(columns={'Reg_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)
    df_reg_az = df_valid.sort_values(by=['sort_key', 'Dir_Label'], ascending=[True, False]).copy()
    df_reg_az.rename(columns={'Reg_Narrative': 'Participants who tested-Who they matched-Oldest known Yates ancestor'}, inplace=True)

    df_tree_za = df_valid.sort_values(by=['Dir_Label', 'sort_key'], ascending=[False, True]).copy()
    df_tree_za.rename(columns={'Tree_Narrative': 'Visual Lineage Path'}, inplace=True)
    df_tree_az = df_valid.sort_values(by=['sort_key', 'Dir_Label'], ascending=[True, False]).copy()
    df_tree_az.rename(columns={'Tree_Narrative': 'Visual Lineage Path'}, inplace=True)

    toggle_reg_za = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Register:</strong> &nbsp;<span style="color:#006064; font-weight:bold;">By Ancestral Line (Z-A)</span> &nbsp;|&nbsp; <a href="ons_yates_dna_register_participants.shtml" style="color:#00acc1; text-decoration:none;">By Participant (A-Z)</a></div>'
    toggle_reg_az = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Register:</strong> &nbsp;<a href="ons_yates_dna_register.shtml" style="color:#00acc1; text-decoration:none;">By Ancestral Line (Z-A)</a> &nbsp;|&nbsp; <span style="color:#006064; font-weight:bold;">By Participant (A-Z)</span></div>'
    toggle_tree_za = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Trees:</strong> &nbsp;<span style="color:#006064; font-weight:bold;">By Ancestral Line (Z-A)</span> &nbsp;|&nbsp; <a href="just-trees-az.shtml" style="color:#00acc1; text-decoration:none;">By Participant (A-Z)</a></div>'
    toggle_tree_az = f'<div class="no-print" style="text-align:center; margin:15px auto; max-width:1400px; padding:10px; background:#e0f7fa; border:1px solid #b2ebf2; border-radius:4px; font-family:sans-serif; font-size:14px;"><strong>Sort Trees:</strong> &nbsp;<a href="just-trees.shtml" style="color:#00acc1; text-decoration:none;">By Ancestral Line (Z-A)</a> &nbsp;|&nbsp; <span style="color:#006064; font-weight:bold;">By Participant (A-Z)</span></div>'

    # 🌟 6. BASE GLOBALS & PAGE BUILDER FUNCTION 🌟
    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%B %d, %Y %-I:%M %p EST")
    stats_bar_full = f'<div style="background:#f4f4f4;border-top:1px solid #ddd;border-bottom:1px solid #ddd;font-family:sans-serif;font-size:12px;color:#555;padding:8px 15px;text-align:center;margin-bottom:0;"><strong>Study Data Current As Of:</strong> {timestamp} | <strong>Total Autosomal matches:</strong> {len(df):,}</div>'
    current_year = str(datetime.now(est).year)
    LEGAL_FOOTER = r"""<div class="legal-footer no-print" style="margin-top:50px;padding:20px;background:#f4f4f4;border-top:1px solid #ddd;text-align:center;color:#666;font-family:sans-serif;font-size:0.85em;clear:both;"><p style="margin-bottom:5px;font-size:1.1em;color:#333;"><strong>&copy; __YEAR__ Ronald Eugene Yates. All Rights Reserved.</strong></p><p style="margin-bottom:5px;">Generated by <em>The Forensic Genealogy Publisher&trade;</em></p><p style="font-style:italic;color:#888;margin-bottom:0;max-width:800px;margin-left:auto;margin-right:auto;">The terms "Forensic Handshake", "Brick Wall Buster", and "Collateral Saturation" are trademarks of Ronald Eugene Yates.</p></div>""".replace('__YEAR__', current_year)

    def make_page(title, content, nav_b, bar, extra_css=""):
        s_info = SITE_INFO if nav_b else ""
        return f"<!DOCTYPE html>\n<html lang=\"en\">\n<head>\n<meta charset=\"UTF-8\">\n<title>{title}</title>\n<link rel=\"stylesheet\" href=\"partials_unified.css\">\n<link rel=\"stylesheet\" href=\"dna_tree_styles.css\">\n{extra_css}\n</head>\n<body id=\"top\">\n<div class=\"wrap\">\n<h1 class=\"centerline no-print\">{title}</h1>\n<div id=\"nav-slot\">{bar}{NAV_HTML}</div>\n{s_info}{content}\n</div>\n{LEGAL_FOOTER}{JS_CORE}{BTT_BTN}\n</body>\n</html>"

    _contents = globals().get('CONTENTS_CONTENT', '<div class="proof-card"><p>Study Guide Content Placeholder</p></div>')
    _share = globals().get('SHARE_CONTENT', '<div class="proof-card"><p>Share DNA Content Placeholder</p></div>')
    _sub = globals().get('SUBSCRIBE_CONTENT', '<div class="proof-card"><p>Subscribe Content Placeholder</p></div>')
    _theory = globals().get('THEORY_CONTENT', '<div class="proof-card"><p>Theory of the Case Placeholder</p></div>')
    _bust = globals().get('BUST_TMPL', '<div class="proof-card"><p>Brick Wall Buster Placeholder</p></div>')
    _glossary = globals().get('GLOSSARY_INLINE', '<div class="proof-card"><p>Glossary Placeholder</p></div>')
    _gedmatch = globals().get('GEDMATCH_INLINE', '<div class="proof-card"><p>GEDmatch Hub Placeholder</p></div>')

    _theory_page = globals().get('THEORY_PAGE_CONTENT', '<div class="proof-card"><p>Theory Page Placeholder</p></div>')
    _network_page = globals().get('NETWORK_PAGE_CONTENT', '<div class="proof-card"><p>Network App Placeholder</p></div>')

    # 🌟 7. ASSEMBLE ALL PAGES 🌟
    pages_to_upload = {}

    anchor_injected = ANCHOR_CONTENT.replace('__TAB_FRAMEWORK__', _theory).replace('__TAB_MASTER__', tab_master).replace('__TAB_CSS__', tab_css).replace('__TAB_DOCS__', tab_docs)
    pages_to_upload["anchor_frame.htm"] = make_page("ANCHOR Database", anchor_injected, False, stats_bar_full, extra_css=ANCHOR_CSS)

    pages_to_upload["proof_consolidator.html"] = make_page("Master Proof Report", CONSOLIDATOR_HTML, False, stats_bar_full, extra_css=CONSOLIDATOR_CSS).replace('</body>', CONSOLIDATOR_JS.replace('__JS_GLOBALS__', JS_GLOBALS) + '</body>')

    pages_to_upload["proof_engine.html"] = PROOF_ENGINE_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)

    pages_to_upload["dna_dossier.html"] = DOSS_TMPL.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["research_admin.html"] = make_page("Yates Research Admin Hub", admin_content, False, stats_bar_full, extra_css=ADMIN_CSS + "<style>.dashboard-grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(200px,1fr));gap:20px;margin:30px auto;max-width:1200px}.dash-card{background:white;padding:20px;border-radius:8px;text-align:center;box-shadow:0 4px 6px rgba(0,0,0,0.1);text-decoration:none;color:#333;border:1px solid #ddd; transition:transform 0.2s;}.dash-card:hover{transform:translateY(-5px);border-color:#006064;background:#e0f7fa}.dash-icon{font-size:40px;margin-bottom:10px;display:block}.dash-title{font-weight:bold;font-size:1.1em;color:#006064}.audit-table-wrapper{background:white;padding:25px;border-radius:8px;box-shadow:0 4px 6px rgba(0,0,0,0.1);max-width:1400px;margin:0 auto} td{padding:10px;border-bottom:1px solid #eee}</style>")

    pages_to_upload["ons_yates_dna_register.shtml"] = make_page("ONS Yates Study DNA Register", toggle_reg_za + f'<div class="table-scroll-wrapper">{df_reg_za.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)
    pages_to_upload["ons_yates_dna_register_participants.shtml"] = make_page("ONS Yates Study DNA Register", toggle_reg_az + f'<div class="table-scroll-wrapper">{df_reg_az.to_html(columns=["Participants who tested-Who they matched-Oldest known Yates ancestor"], index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)
    pages_to_upload["just-trees.shtml"] = make_page("Ancestor Register (Trees View)", toggle_tree_za + f'<div class="table-scroll-wrapper">{df_tree_za[["Visual Lineage Path"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)
    pages_to_upload["just-trees-az.shtml"] = make_page("Ancestor Register (Trees View)", toggle_tree_az + f'<div class="table-scroll-wrapper">{df_tree_az[["Visual Lineage Path"]].to_html(index=False, border=1, classes="dataframe sortable", escape=False, table_id="reg-table")}</div>', True, stats_bar_full, extra_css=REGISTER_CSS)

    pages_to_upload["data_glossary.shtml"] = make_page("Data Glossary", _glossary, False, stats_bar_full, extra_css="")
    pages_to_upload["gedmatch_integration.shtml"] = make_page("GEDmatch Hub", _gedmatch, False, stats_bar_full, extra_css="")
    pages_to_upload["brick_wall_buster.shtml"] = _bust.replace('__CSS_BASE__', CSS_BASE).replace('__STATS_BAR__', stats_bar_full).replace('__NAV_HTML__', NAV_HTML).replace('__JS_GLOBALS__', JS_GLOBALS).replace('__LEGAL_FOOTER__', LEGAL_FOOTER)
    pages_to_upload["contents.shtml"] = make_page("Yates Study User Guide", _contents, False, stats_bar_full, extra_css="")
    pages_to_upload["share_dna.shtml"] = make_page("Share Your Ancestry DNA Matches", _share, False, stats_bar_full, extra_css="")
    pages_to_upload["subscribe.shtml"] = make_page("Join the Yates Research Community", _sub, False, stats_bar_full, extra_css="")

    pages_to_upload["admin_singletons.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]
    pages_to_upload["admin_singletons_participants.shtml"] = pages_to_upload["ons_yates_dna_register_participants.shtml"]
    pages_to_upload["yates_ancestor_register.shtml"] = pages_to_upload["ons_yates_dna_register.shtml"]

    pages_to_upload["dna_theory_of_the_case.htm"] = make_page("DNA Theory of the Case", _theory_page, False, stats_bar_full, extra_css="")

    # ✨ FIX: Save the new app as .html instead of .shtml to bypass server memory limits!
    pages_to_upload["dna_network.html"] = make_page("DNA Network Visualizer", _network_page.replace('__JS_GLOBALS__', JS_GLOBALS), False, stats_bar_full, extra_css="")

    # ✨ FIX: Overwrite the old broken .shtml file with a silent redirect to the new .html file
    pages_to_upload["dna_network.shtml"] = '<meta http-equiv="refresh" content="0; url=dna_network.html" />'

    # 🌟 8. DISK SAVE AND FTP UPLOAD 🌟
    print("\n[LOCAL] Overwriting Database-Driven Pages on disk...")
    for fn, content in pages_to_upload.items():
        if os.path.exists(fn): os.remove(fn)
        with open(fn, "w", encoding="utf-8") as f: f.write(content)

    print("\n[STEP 3] Uploading via FTP to Live Server...")
    try:
        ftps = FTP_TLS()
        ftps.connect(HOST, 21); ftps.auth(); ftps.login(USER, PASS); ftps.prot_p(); ftps.cwd("ons-study")

        files_to_upload = list(pages_to_upload.keys())
        if os.path.exists(CSV_DB):
            files_to_upload.append(CSV_DB)

        upload_count = 0
        for fn in files_to_upload:
            with open(fn, "rb") as fh: ftps.storbinary(f"STOR {fn}", fh)
            upload_count += 1
            print(f"    [{upload_count}/{len(files_to_upload)}] 📤 Uploaded: {fn}")

        ftps.quit()
        print(f"\n🎉 SUCCESS. Uploaded {upload_count} files directly to the active server.")
    except Exception as e:
        print(f"\n⚠️ FTP SKIP: {e}. ZIP and manual upload required.")

print("✅ Cell 5 (HTML Ext Override) Loaded.")

✅ Cell 5 (HTML Ext Override) Loaded.


In [32]:
# @title [CELL 6] The Master Orchestrator (One-Click Deploy)
print("="*60)
print("      [CELL 6] MASTER ORCHESTRATOR STARTING...")
print("="*60)

try:
    print("\n[1/2] FIRING UP THE DATA ENGINE (Cell 3)...")
    run_engine()

    print("\n[2/2] FIRING UP THE PUBLISHER (Cell 5)...")
    run_publisher()

    print("\n" + "="*60)
    print(" 🎉 FULL SYSTEM DEPLOYMENT COMPLETE! 🎉")
    print("="*60)
    print("Your data has been successfully processed, merged with the")
    print("Template Vault, and deployed directly to the live server.")

except NameError as e:
    print(f"\n❌ ORCHESTRATOR ERROR: {e}")
    print("    It looks like Colab forgot the functions. Please make sure you")
    print("    have clicked 'Run' on Cell 3 (Engine) and Cell 5 (Publisher)")
    print("    at least once during this session to load them into memory.")
except Exception as e:
    print(f"\n❌ UNEXPECTED ERROR: {e}")

      [CELL 6] MASTER ORCHESTRATOR STARTING...

[1/2] FIRING UP THE DATA ENGINE (Cell 3)...
      [CELL 3] ENGINE STARTING (V124 - SORT AUTHORITY)...

[STEP 1] Resolving Files (Local Priority)...
    ✅ Found match_to_unmasked.csv locally. Skipping FTP download.
    👉 Source GEDCOM: yates_study_2025.ged

[STEP 2] Loading Tester Authority CSV...

[STEP 3] Parsing GEDCOM for Study| Tags & Lineages...

[STEP 4] Constructing Database...

[SUCCESS] Engine V124 Complete. Saved 1713 verified matches to engine_database.csv.

[2/2] FIRING UP THE PUBLISHER (Cell 5)...
      [CELL 5] PUBLISHER STARTING (Report Assembly)...
    [+] Parsing GEDCOM Vitals: yates_study_2025.ged
    [+] Calculating ANCHOR Matrices Server-Side...

[LOCAL] Overwriting Database-Driven Pages on disk...

[STEP 3] Uploading via FTP to Live Server...
    [1/23] 📤 Uploaded: proof_consolidator.html
    [2/23] 📤 Uploaded: proof_engine.html
    [3/23] 📤 Uploaded: dna_dossier.html
    [4/23] 📤 Uploaded: research_admin.html
    [5/

In [ ]:
# @title [CELL 7] The Dropbox Archiver (Automated Backup)
print("="*60)
print("      [CELL 7] DROPBOX ARCHIVER STARTING...")
print("="*60)

import os
import shutil
import pytz
from datetime import datetime
from google.colab import userdata

# Install dropbox library if Colab doesn't have it loaded
try:
    import dropbox
    from dropbox.exceptions import AuthError
except ImportError:
    print("    [+] Installing Dropbox API...")
    !pip install -q dropbox
    import dropbox
    from dropbox.exceptions import AuthError

def run_archiver():
    # 1. Fetch Dropbox Token
    try:
        DBX_TOKEN = os.environ.get("DROPBOX_TOKEN") or userdata.get("DROPBOX_TOKEN")
    except Exception as e:
        return print(f"❌ SECRET ERROR: Could not find DROPBOX_TOKEN. Please add it to your Colab keys. ({e})")

    if not DBX_TOKEN:
        return print("❌ ERROR: DROPBOX_TOKEN is empty.")

    # 2. Create a Timestamped ZIP file
    est = pytz.timezone('US/Eastern')
    timestamp = datetime.now(est).strftime("%Y-%m-%d_%H-%M-%S")
    zip_filename = f"Yates_Study_Backup_{timestamp}"
    zip_filepath = f"/tmp/{zip_filename}" # Save to /tmp so we don't zip the zip!

    print(f"    [+] Zipping current working directory...")
    shutil.make_archive(zip_filepath, 'zip', '.')
    final_zip_path = f"{zip_filepath}.zip"
    zip_size_mb = os.path.getsize(final_zip_path) / (1024 * 1024)
    print(f"    [+] Archive created successfully ({zip_size_mb:.2f} MB).")

    # 3. Upload to Dropbox
    print(f"    [+] Connecting to Dropbox...")
    try:
        dbx = dropbox.Dropbox(DBX_TOKEN)
        # Verify connection
        dbx.users_get_current_account()

        dropbox_destination = f"/Yates_DNA_Backups/{zip_filename}.zip"
        print(f"    [+] Uploading to {dropbox_destination}...")

        with open(final_zip_path, "rb") as f:
            # Note: files_upload handles files up to 150MB, which is plenty for our scripts/CSVs/GEDCOM
            dbx.files_upload(f.read(), dropbox_destination, mode=dropbox.files.WriteMode.overwrite)

        print(f"\n🎉 SUCCESS! Full system backup saved to your Dropbox.")

    except AuthError as e:
        print(f"\n❌ DROPBOX AUTH ERROR: Invalid token. Please check your DROPBOX_TOKEN in Colab secrets. ({e})")
    except Exception as e:
        print(f"\n❌ DROPBOX UPLOAD ERROR: {e}")

# Execute the Archiver
run_archiver()